In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Desactiva los mensajes de advertencia

In [2]:
pip install "accelerate>=0.26.0"

In [20]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoConfig, PreTrainedModel, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification

# Cargar el dataset
dataset_path = '/content/combined_shuffled_dataset.csv'
df = pd.read_csv(dataset_path, on_bad_lines='skip')

# Dividir en entrenamiento y validación
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label'].tolist()
)

# Configuración del modelo
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenización
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Dataset personalizado
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item

train_dataset = FakeNewsDataset(train_encodings, train_labels)
val_dataset = FakeNewsDataset(val_encodings, val_labels)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Clase personalizada de BERT que hereda de PreTrainedModel
class AdvancedBERTClassifier(PreTrainedModel):
    def __init__(self, config, num_labels=2):
        super().__init__(config)
        # Cargar modelo base BERT
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            "bert-base-multilingual-cased",  # Usar directamente el nombre del modelo base
            num_labels=num_labels,
            output_hidden_states=True
        )

        # Congelar capas iniciales
        for param in self.bert.bert.embeddings.parameters():
            param.requires_grad = False
        for param in self.bert.bert.encoder.layer[:9].parameters():
            param.requires_grad = False

        # Arquitectura de clasificación avanzada
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Obtener salidas de BERT
        outputs = self.bert.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Usar pooled output
        pooled_output = outputs[1]

        # Pasar por capas personalizadas
        logits = self.classifier(pooled_output)

        # Manejo de pérdida
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.bert.config.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *args, **kwargs):
        # Usamos AutoConfig para cargar la configuración de manera predeterminada
        config = AutoConfig.from_pretrained(pretrained_model_name_or_path)
        model = super().from_pretrained(pretrained_model_name_or_path, config=config, *args, **kwargs)
        return model

# Inicializar modelo
model = AdvancedBERTClassifier.from_pretrained(model_name)

# Métricas personalizadas con múltiples umbrales
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.softmax(torch.tensor(logits), dim=-1)

    # Experimentar con diferentes umbrales
    thresholds = [0.4, 0.45, 0.5]
    best_f1 = 0
    best_threshold = 0.45
    best_metrics = {}

    for threshold in thresholds:
        binary_predictions = (predictions[:, 1] > threshold).int()
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, binary_predictions, average='binary'
        )

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_metrics = {
                "accuracy": accuracy_score(labels, binary_predictions),
                "f1": f1,
                "precision": precision,
                "recall": recall
            }

    return best_metrics

# Argumentos de entrenamiento con warmup y ajustes
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
    learning_rate=2e-5,  # Reducir learning rate
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=15,  # Aumentar épocas
    weight_decay=0.1,   # Ajustar peso de decaimiento
    warmup_ratio=0.05,    # Añadir warmup learning rate
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs'
)

# Early Stopping con mayor paciencia
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0005
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Entrenar
trainer.train()

# Guardar el modelo y el tokenizador
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# Evaluar el modelo
results = trainer.evaluate()
print("Resultados de evaluación:", results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AdvancedBERTClassifier were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.bert.embeddings.LayerNorm.bias', 'bert.bert.embeddings.LayerNorm.weight', 'bert.bert.embeddings.position_embeddings.weight', 'bert.bert.embeddings.token_type_embeddings.weight', 'bert.bert.embeddings.word_embeddings.weight', 'bert.bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.bert.encoder.layer.0.attention.output.dense.bias', 'bert.bert.encoder.layer.0.attention.output.dense.weight', 'bert.bert.encoder.layer.0.attention.self.key.bias', 'bert.bert.encoder.layer.0

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.422317,0.821116,0.828924,0.808255,0.850679
2,0.543100,0.401566,0.835408,0.840125,0.831560,0.848869
3,0.543100,0.387555,0.847856,0.848901,0.859129,0.838914
4,0.379700,0.381673,0.844168,0.838586,0.887765,0.794570
5,0.379700,0.382369,0.845090,0.847826,0.848595,0.847059
6,0.343600,0.384357,0.848317,0.846763,0.872361,0.822624


Resultados de evaluación: {'eval_loss': 0.38755545020103455, 'eval_accuracy': 0.8478561549100968, 'eval_f1': 0.8489010989010989, 'eval_precision': 0.8591288229842446, 'eval_recall': 0.8389140271493213, 'eval_runtime': 14.3422, 'eval_samples_per_second': 151.232, 'eval_steps_per_second': 4.741, 'epoch': 6.0}


In [21]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoConfig, PreTrainedModel, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification

# Cargar el dataset
dataset_path = '/content/combined_shuffled_dataset.csv'
df = pd.read_csv(dataset_path, on_bad_lines='skip')

# Dividir en entrenamiento y validación
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label'].tolist()
)

# Configuración del modelo
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenización
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Dataset personalizado
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item

train_dataset = FakeNewsDataset(train_encodings, train_labels)
val_dataset = FakeNewsDataset(val_encodings, val_labels)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Clase personalizada de BERT que hereda de PreTrainedModel
class AdvancedBERTClassifier(PreTrainedModel):
    def __init__(self, config, num_labels=2):
        super().__init__(config)
        # Cargar modelo base BERT
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            "bert-base-multilingual-cased",  # Usar directamente el nombre del modelo base
            num_labels=num_labels,
            output_hidden_states=True
        )

        # Congelar capas iniciales
        for param in self.bert.bert.embeddings.parameters():
            param.requires_grad = False
        for param in self.bert.bert.encoder.layer[:3].parameters():
            param.requires_grad = False

        # Arquitectura de clasificación avanzada
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Obtener salidas de BERT
        outputs = self.bert.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Usar pooled output
        pooled_output = outputs[1]

        # Pasar por capas personalizadas
        logits = self.classifier(pooled_output)

        # Manejo de pérdida
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.bert.config.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *args, **kwargs):
        # Usamos AutoConfig para cargar la configuración de manera predeterminada
        config = AutoConfig.from_pretrained(pretrained_model_name_or_path)
        model = super().from_pretrained(pretrained_model_name_or_path, config=config, *args, **kwargs)
        return model

# Inicializar modelo
model = AdvancedBERTClassifier.from_pretrained(model_name)

# Métricas personalizadas con múltiples umbrales
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.softmax(torch.tensor(logits), dim=-1)

    # Experimentar con diferentes umbrales
    thresholds = [0.4, 0.45, 0.5]
    best_f1 = 0
    best_threshold = 0.45
    best_metrics = {}

    for threshold in thresholds:
        binary_predictions = (predictions[:, 1] > threshold).int()
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, binary_predictions, average='binary'
        )

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_metrics = {
                "accuracy": accuracy_score(labels, binary_predictions),
                "f1": f1,
                "precision": precision,
                "recall": recall
            }

    return best_metrics

# Argumentos de entrenamiento con warmup y ajustes
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
    learning_rate=2e-5,  # Reducir learning rate
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=15,  # Aumentar épocas
    weight_decay=0.1,   # Ajustar peso de decaimiento
    warmup_ratio=0.05,    # Añadir warmup learning rate
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs'
)

# Early Stopping con mayor paciencia
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0005
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Entrenar
trainer.train()

# Guardar el modelo y el tokenizador
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# Evaluar el modelo
results = trainer.evaluate()
print("Resultados de evaluación:", results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AdvancedBERTClassifier were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.bert.embeddings.LayerNorm.bias', 'bert.bert.embeddings.LayerNorm.weight', 'bert.bert.embeddings.position_embeddings.weight', 'bert.bert.embeddings.token_type_embeddings.weight', 'bert.bert.embeddings.word_embeddings.weight', 'bert.bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.bert.encoder.layer.0.attention.output.dense.bias', 'bert.bert.encoder.layer.0.attention.output.dense.weight', 'bert.bert.encoder.layer.0.attention.self.key.bias', 'bert.bert.encoder.layer.0

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.428718,0.822038,0.826906,0.819556,0.834389
2,0.554600,0.402002,0.839096,0.844958,0.829843,0.860633
3,0.554600,0.387601,0.849239,0.852503,0.849820,0.855204
4,0.348400,0.402589,0.838174,0.831006,0.887860,0.780995
5,0.348400,0.396319,0.842785,0.845070,0.848540,0.841629
6,0.289400,0.430749,0.840941,0.840352,0.859848,0.821719


Resultados de evaluación: {'eval_loss': 0.3876006007194519, 'eval_accuracy': 0.8492392807745505, 'eval_f1': 0.8525033829499323, 'eval_precision': 0.8498201438848921, 'eval_recall': 0.8552036199095022, 'eval_runtime': 14.2866, 'eval_samples_per_second': 151.821, 'eval_steps_per_second': 4.76, 'epoch': 6.0}


In [9]:
import torch
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Cargar el modelo y el tokenizer entrenados
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AdvancedBERTClassifier.from_pretrained("./results", num_labels=2)

# Definir la función para realizar las predicciones
def predict_news(news_list):
    # Tokenizar las noticias
    encodings = tokenizer(news_list, truncation=True, padding=True, max_length=256, return_tensors="pt")

    # Obtener los parámetros necesarios para la predicción
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']

    # Realizar las predicciones (sin token_type_ids)
    with torch.no_grad():
        # Llamamos al modelo
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # La salida es un tensor de logits
        logits = outputs

        # Convertir logits a probabilidades usando softmax
        probabilities = torch.softmax(logits, dim=-1)  # Probabilidades para ambas clases
        predictions = torch.argmax(probabilities, dim=-1)  # Predicción de clase (índice de la clase con la mayor probabilidad)

    return predictions.numpy(), probabilities.numpy()

# Conjunto de noticias de prueba (verdaderas y falsas)
news_to_test = [
    "Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam.  In the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI.  The word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI.  But that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election.  The FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI.  The FBI’s leadership is being warned that the entire left-wing establishment will form a lynch mob if they continue going after Hillary. And the FBI’s credibility is being attacked by the media and the Democrats to preemptively head off the results of the investigation of the Clinton Foundation and Hillary Clinton.  The covert struggle between FBI agents and Obama’s DOJ people has gone explosively public.  The New York Times has compared Comey to J. Edgar Hoover. Its bizarre headline, “James Comey Role Recalls Hoover’s FBI, Fairly or Not” practically admits up front that it’s spouting nonsense. The Boston Globe has published a column calling for Comey’s resignation. Not to be outdone, Time has an editorial claiming that the scandal is really an attack on all women.  James Carville appeared on MSNBC to remind everyone that he was still alive and insane. He accused Comey of coordinating with House Republicans and the KGB. And you thought the “vast right wing conspiracy” was a stretch.  Countless media stories charge Comey with violating procedure. Do you know what’s a procedural violation? Emailing classified information stored on your bathroom server.  Senator Harry Reid has sent Comey a letter accusing him of violating the Hatch Act. The Hatch Act is a nice idea that has as much relevance in the age of Obama as the Tenth Amendment. But the cable news spectrum quickly filled with media hacks glancing at the Wikipedia article on the Hatch Act under the table while accusing the FBI director of one of the most awkward conspiracies against Hillary ever.  If James Comey is really out to hurt Hillary, he picked one hell of a strange way to do it.  Not too long ago Democrats were breathing a sigh of relief when he gave Hillary Clinton a pass in a prominent public statement. If he really were out to elect Trump by keeping the email scandal going, why did he trash the investigation? Was he on the payroll of House Republicans and the KGB back then and playing it coy or was it a sudden development where Vladimir Putin and Paul Ryan talked him into taking a look at Anthony Weiner’s computer?  Either Comey is the most cunning FBI director that ever lived or he’s just awkwardly trying to navigate a political mess that has trapped him between a DOJ leadership whose political futures are tied to Hillary’s victory and his own bureau whose apolitical agents just want to be allowed to do their jobs.  The only truly mysterious thing is why Hillary and her associates decided to go to war with a respected Federal agency. Most Americans like the FBI while Hillary Clinton enjoys a 60% unfavorable rating.  And it’s an interesting question.  Hillary’s old strategy was to lie and deny that the FBI even had a criminal investigation underway. Instead her associates insisted that it was a security review. The FBI corrected her and she shrugged it off. But the old breezy denial approach has given way to a savage assault on the FBI.  Pretending that nothing was wrong was a bad strategy, but it was a better one that picking a fight with the FBI while lunatic Clinton associates try to claim that the FBI is really the KGB.  There are two possible explanations.  Hillary Clinton might be arrogant enough to lash out at the FBI now that she believes that victory is near. The same kind of hubris that led her to plan her victory fireworks display could lead her to declare a war on the FBI for irritating her during the final miles of her campaign.  But the other explanation is that her people panicked.  Going to war with the FBI is not the behavior of a smart and focused presidential campaign. It’s an act of desperation. When a presidential candidate decides that her only option is to try and destroy the credibility of the FBI, that’s not hubris, it’s fear of what the FBI might be about to reveal about her.  During the original FBI investigation, Hillary Clinton was confident that she could ride it out. And she had good reason for believing that. But that Hillary Clinton is gone. In her place is a paranoid wreck. Within a short space of time the “positive” Clinton campaign promising to unite the country has been replaced by a desperate and flailing operation that has focused all its energy on fighting the FBI.  There’s only one reason for such bizarre behavior.  The Clinton campaign has decided that an FBI investigation of the latest batch of emails poses a threat to its survival. And so it’s gone all in on fighting the FBI. It’s an unprecedented step born of fear. It’s hard to know whether that fear is justified. But the existence of that fear already tells us a whole lot.  Clinton loyalists rigged the old investigation. They knew the outcome ahead of time as well as they knew the debate questions. Now suddenly they are no longer in control. And they are afraid.  You can smell the fear.  The FBI has wiretaps from the investigation of the Clinton Foundation. It’s finding new emails all the time. And Clintonworld panicked. The spinmeisters of Clintonworld have claimed that the email scandal is just so much smoke without fire. All that’s here is the appearance of impropriety without any of the substance. But this isn’t how you react to smoke. It’s how you respond to a fire.  The misguided assault on the FBI tells us that Hillary Clinton and her allies are afraid of a revelation bigger than the fundamental illegality of her email setup. The email setup was a preemptive cover up. The Clinton campaign has panicked badly out of the belief, right or wrong, that whatever crime the illegal setup was meant to cover up is at risk of being exposed.  The Clintons have weathered countless scandals over the years. Whatever they are protecting this time around is bigger than the usual corruption, bribery, sexual assaults and abuses of power that have followed them around throughout the years. This is bigger and more damaging than any of the allegations that have already come out. And they don’t want FBI investigators anywhere near it.  The campaign against Comey is pure intimidation. It’s also a warning. Any senior FBI people who value their careers are being warned to stay away. The Democrats are closing ranks around their nominee against the FBI. It’s an ugly and unprecedented scene. It may also be their last stand.  Hillary Clinton has awkwardly wound her way through numerous scandals in just this election cycle. But she’s never shown fear or desperation before. Now that has changed. Whatever she is afraid of, it lies buried in her emails with Huma Abedin. And it can bring her down like nothing else has.  ",  # 0
    "Google Pinterest Digg Linkedin Reddit Stumbleupon Print Delicious Pocket Tumblr  There are two fundamental truths in this world: Paul Ryan desperately wants to be president. And Paul Ryan will never be president. Today proved it.  In a particularly staggering example of political cowardice, Paul Ryan re-re-re-reversed course and announced that he was back on the Trump Train after all. This was an aboutface from where he was a few weeks ago. He had previously declared he would not be supporting or defending Trump after a tape was made public in which Trump bragged about assaulting women. Suddenly, Ryan was appearing at a pro-Trump rally and boldly declaring that he already sent in his vote to make him President of the United States. It was a surreal moment. The figurehead of the Republican Party dosed himself in gasoline, got up on a stage on a chilly afternoon in Wisconsin, and lit a match. . @SpeakerRyan says he voted for @realDonaldTrump : “Republicans, it is time to come home” https://t.co/VyTT49YvoE pic.twitter.com/wCvSCg4a5I  — ABC News Politics (@ABCPolitics) November 5, 2016  The Democratic Party couldn’t have asked for a better moment of film. Ryan’s chances of ever becoming president went down to zero in an instant. In the wreckage Trump is to leave behind in his wake, those who cravenly backed his campaign will not recover. If Ryan’s career manages to limp all the way to 2020, then the DNC will have this tape locked and loaded to be used in every ad until Election Day.  The ringing endorsement of the man he clearly hates on a personal level speaks volumes about his own spinelessness. Ryan has postured himself as a “principled” conservative, and one uncomfortable with Trump’s unapologetic bigotry and sexism. However, when push came to shove, Paul Ryan – like many of his colleagues – turned into a sniveling appeaser. After all his lofty tak about conviction, his principles were a house of cards and collapsed with the slightest breeze.  What’s especially bizarre is how close Ryan came to making it through unscathed. For months the Speaker of the House refused to comment on Trump at all. His strategy seemed to be to keep his head down, pretend Trump didn’t exist, and hope that nobody remembered what happened in 2016. Now, just days away from the election, he screwed it all up.  If 2016’s very ugly election has done any good it’s by exposing the utter cowardice of the Republicans who once feigned moral courage. A reality television star spit on them, hijacked their party, insulted their wives, and got every last one of them to kneel before him. What a turn of events.  Featured image via Twitter",  # 0
    "U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sunday’s unity march against terrorism.  Kerry said he expects to arrive in Paris Thursday evening, as he heads home after a week abroad. He said he will fly to France at the conclusion of a series of meetings scheduled for Thursday in Sofia, Bulgaria. He plans to meet the next day with Foreign Minister Laurent Fabius and President Francois Hollande, then return to Washington.  The visit by Kerry, who has family and childhood ties to the country and speaks fluent French, could address some of the criticism that the United States snubbed France in its darkest hour in many years.  The French press on Monday was filled with questions about why neither President Obama nor Kerry attended Sunday’s march, as about 40 leaders of other nations did. Obama was said to have stayed away because his own security needs can be taxing on a country, and Kerry had prior commitments.  Among roughly 40 leaders who did attend was Israeli Prime Minister Benjamin Netanyahu, no stranger to intense security, who marched beside Hollande through the city streets. The highest ranking U.S. officials attending the march were Jane Hartley, the ambassador to France, and Victoria Nuland, the assistant secretary of state for European affairs. Attorney General Eric H. Holder Jr. was in Paris for meetings with law enforcement officials but did not participate in the march.  Kerry spent Sunday at a business summit hosted by India’s prime minister, Narendra Modi. The United States is eager for India to relax stringent laws that function as barriers to foreign investment and hopes Modi’s government will act to open the huge Indian market for more American businesses.  In a news conference, Kerry brushed aside criticism that the United States had not sent a more senior official to Paris as “quibbling a little bit.” He noted that many staffers of the American Embassy in Paris attended the march, including the ambassador. He said he had wanted to be present at the march himself but could not because of his prior commitments in India.  “But that is why I am going there on the way home, to make it crystal clear how passionately we feel about the events that have taken place there,” he said.  “And I don’t think the people of France have any doubts about America’s understanding of what happened, of our personal sense of loss and our deep commitment to the people of France in this moment of trauma.”",  # 1
    "— Kaydee King (@KaydeeKing) November 9, 2016 The lesson from tonight's Dem losses: Time for Democrats to start listening to the voters. Stop running the same establishment candidates.  — People For Bernie (@People4Bernie) November 9, 2016 If Dems didn't want a tight race they shouldn't have worked against Bernie.  — Walker Bragman (@WalkerBragman) November 9, 2016  New York Times columnist Paul Krugman, who was one of Hillary Clinton’s most outspoken surrogates during the contentious Democratic primary, blamed Clinton’s poor performance on Green Party candidate Jill Stein, who has so far received a negligible number of votes nationally, saying Stein was the Ralph Nader of 2016 in preventing a Clinton victory. The account @BerniesTeachers threw Krugman’s analysis back in his face. Your candidate was the issue. Take responsibility. https://t.co/KHyOuUSrFS  — Teachers for Bernie (@BerniesTeachers) November 9, 2016  Ana Navarro, a Republican who recently endorsed Hillary Clinton, summed up the preposterous nature of the 2016 presidential election in this tweet: GOP nominated the only damn candidate who could lose to Hillary Clinton. Democrats nominated the only damn candidate who could lose to Trump  — Ana Navarro (@ananavarro) November 9, 2016  Popular left-wing Facebook page The Other 98%, which was pro-Sanders during the primary, responded to Trump’s surge by simply posting a meme of Sanders’ face with the text “All this could’ve been avoided. Thanks for nothing, DNC!” The meme has been shared almost 15,000 times in less than an hour:  Posted by The Other 98% on Tuesday, November 8, 2016  While Bernie Sanders endorsed Hillary Clinton just before the Democratic National Convention in July, many of his supporters remained adamant in their refusal to support the DNC-anointed candidate, pointing to WikiLeaks’ revelations that top officials at the DNC had been working behind the scenes to tip the scales in Clinton’s favor by coordinating with media figures to circulate anti-Sanders narratives.  Rather than attribute a potential Trump presidency to the GOP nominee’s perceived popularity among voters, the closeness of this election could be credited to Hillary Clinton’s unfavorable ratings. According to RealClearPolitics, anywhere between 51 and 57 percent of voters had a negative opinion of the Democratic nominee.  As of 11 PM Eastern, Florida, Michigan, Pennsylvania, and Wisconsin remain too close to call. Clinton has 197 electoral votes to Trump’s 187.   Zach Cartwright is an activist and author from Richmond, Virginia. He enjoys writing about politics, government, and the media. Send him an email at [email protected]",  # 0
    "It's primary day in New York and front-runners Hillary Clinton and Donald Trump are leading in the polls.  Trump is now vowing to win enough delegates to clinch the Republican nomination and prevent a contested convention. But Sens.Ted Cruz, R-Texas, Bernie Sanders, D-Vt., and Ohio Gov. John Kasich and aren't giving up just yet.  A big win in New York could tip the scales for both the Republican and Democratic front-runners in this year's race for the White House. Clinton and Trump have each suffered losses in recent contests, shifting the momentum to their rivals.  \"We have won eight out of the last nine caucuses and primaries! Cheer!\" Sanders recently told supporters.  While wins in New York for Trump and Clinton are expected, the margins of those victories are also important.  Trump needs to capture more than 50 percent of the vote statewide if he wants to be positioned to win all of the state's 95 GOP delegates. That would put him one step closer to avoiding a contested convention.  \"We've got to vote and you know Cruz is way, way down in the polls,\" Trump urged supporters.  Meanwhile, Sanders is hoping for a close race in the Empire State. A loss by 10 points means he'll need to win 80 percent of the remaining delegates to clinch the nomination.  Despite a predicted loss in New York, Cruz hasn't lost momentum. He's hoping to sweep up more delegates this weekend while he's talking about how he can win in November.  \"Because if I'm the nominee, we win the General Election,\" Cruz promised his supporters. \"We're beating Hillary in the key swing states, we're beating Hillary with Independents, we're beating Hillary with young people.\"  For now, Cruz, Kasich, and Sanders have all moved on from New York to other states. Trump and Clinton are the only two staying in their home state to watch the results come in.",  # 1
    "   I’m not an immigrant, but my grandparents are. More than 50 years ago, they arrived in New York City from Iran. I grew up mainly in central New Jersey, an American kid playing little league for the Raritan Red Sox and soccer for the Raritan Rovers. In 1985, I travelled with my family to our ancestral land. I was only eight, but old enough to understand that the Iranians had lost their liberty and freedom. I saw the abject despair of a people who, in a desperate attempt to bring about change, had ushered in nationalist tyrants led by Ayatollah Khomeini.  What I witnessed during that year in Iran changed the course of my life. In 1996, at age 19, wanting to help preserve the blessings of liberty and freedom we enjoy in America, I enlisted in the U.S. Navy. Now, with the rise of Donald Trump and his nationalist alt-right movement, I’ve come to feel that the values I sought to protect are in jeopardy.  In Iran, theocratic fundmentalists sowed division and hatred of outsiders — of Westerners, Christians, and other religious minorities. Here in America, the right wing seems to have stolen passages directly from their playbook as it spreads hatred of immigrants, particularly Muslim ones. This form of nationalistic bigotry — Islamophobia — threatens the heart of our nation. When I chose to serve in the military, I did so to protect what I viewed as our sacred foundational values of liberty, equality, and democracy. Now, 20 years later, I’ve joined forces with fellow veterans to again fight for those sacred values, this time right here at home.  “Death to America!”  As a child, I sat in my class at the international school one sunny morning and heard in the distance the faint sounds of gunfire and rising chants of “Death to America!” That day would define the rest of my life.  It was Tehran, the capital of Iran, in 1985. I was attending a unique school for bilingual students who had been born in Western nations. It had become the last refuge in that city with any tolerance for Western teaching, but that also made it a target for military fundamentalists. As the gunfire drew closer, I heard boots pounding the marble tiles outside, marching into our building, and thundering down the corridor toward my classroom. As I heard voices chanting “Death to America!” I remember wondering if I would survive to see my parents again.  In a flash of green and black uniforms, those soldiers rushed into our classroom, grabbed us by our shirt collars, and yelled at us to get outside. We were then packed into the school’s courtyard where a soldier pointed his rifle at our group and commanded us to look up. Almost in unison, my classmates and I raised our eyes and saw the flags of our many nations being torn down and dangled from the balcony, then set ablaze and tossed, still burning, into the courtyard. As those flags floated to the ground in flames, the soldiers fired their guns in the air. Shouting, they ordered us — if we ever wanted to see our families again — to swear allegiance to the Grand Ayatollah Khomeini and trample on the remains of the burning symbols of our home countries. I scanned the smoke that was filling the courtyard for my friends and classmates and, horrified, watched them capitulate and begin to chant, “Death to America!” as they stomped on our sacred symbols.  I was so angry that, young as I was, I began to plead with them to come to their senses. No one paid the slightest attention to an eight year old and yet, for the first time in my life, I felt something like righteous indignation. I suspect that, born and raised in America, I was already imbued with such a sense of privilege that I just couldn’t fathom the immense danger I was in. Certainly, I was acting in ways no native Iranian would have found reasonable.  Across the smoke-filled courtyard, I saw a soldier coming at me and knew he meant to force me to submit. I spotted an American flag still burning, dropped to my knees, and grabbed the charred pieces from underneath a classmate’s feet. As the soldier closed in on me, I ducked and ran, still clutching my charred pieces of flag into a crowd of civilians who had gathered to witness the commotion. The events of that day would come to define all that I have ever stood for — or against.  “Camel Jockey,” “Ayatollah,” and “Gandhi”  My parents and I soon returned to the United States and I entered third grade. More than anything, I just wanted to be normal, to fit in and be accepted by my peers. Unfortunately, my first name, Nader (which I changed to Nate upon joining the Navy), and my swarthy Middle Eastern appearance, were little help on that score, eliciting regular jibes from my classmates. Even at that young age, they had already mastered a veritable thesaurus of ethnic defamation, including “camel jockey,” “sand-nigger,” “raghead,” “ayatollah,” and ironically, “Gandhi” (which I now take as a compliment). My classmates regularly sought to “other-ize” me in those years, as if I were a lesser American because of my faith and ethnicity.  Yet I remember that tingling in my chest when I first donned my Cub Scout uniform — all because of the American flag patch on its shoulder. Something felt so good about wearing it, a feeling I still had when I joined the military. It seems that the flag I tried to rescue in Tehran was stapled to my heart, or that’s how I felt anyway as I wore my country’s uniform.  When I took my oath of enlistment in the U.S. Navy, I gave my mom a camera and asked her to take some photos, but she was so overwhelmed with pride and joy that she cried throughout the ceremony and managed to snap only a few images of the carpet. She cried even harder when I was selected to serve as the first Muslim-American member of the U.S. Navy Presidential Ceremonial Honor Guard . On that day, I was proud, too, and all the taunts of those bullies of my childhood seemed finally silenced.  Being tormented because of my ethnicity and religion in those early years had another effect on me. It caused me to become unusually sensitive to the nature of other people. Somehow, I grasped that, if it weren’t for a fear of the unknown, there was an inherent goodness and frail humanity lurking in many of the kids who bullied and harassed me. Often, I discovered, those same bullies could be tremendously kind to their families, friends, or even strangers. I realized, then, that if, despite everything, I could lay myself bare and trust them enough to reach out in kindness, I might in turn gain their trust and they might then see me, too, and stop operating from such a place of fear and hate.  Through patience, humor, and understanding, I was able to offer myself as the embodiment of my people and somehow defang the “otherness” of so much that Americans found scary. To this day, I have friends from elementary school, middle school, high school, and the military who tell me that I am the only Muslim they have ever known and that, had they not met me, their perspective on Islam would have been wholly subject to the prevailing fear-based narrative that has poisoned this country since September 11, 2001.  In 1998, I became special assistant to the Master Chief Petty Officer of the Navy and then, in 1999, I was recruited to serve at the Defense Intelligence Agency. In August 2000, I transferred to the Naval Reserve.  In the wake of 9/11, I began to observe how so many of my fellow Americans were adopting a fundamentalist “us vs. them” attitude towards Muslims and Islam. I suddenly found myself in an America where the scattered insults I had endured as a child took on an overarching and sinister meaning and form, where they became something like an ideology and way of life.  By the time I completed my military service in 2006, I had begun to understand that our policies in the Middle East,similarly disturbed, seemed in pursuit of little more than perpetual warfare. That, in turn, was made possible by the creation of a new enemy: Islam — or rather of a portrait, painted by the powers-that-be, of Islam as a terror religion, as a hooded villain lurking out there somewhere in the desert, waiting to destroy us. I knew that attempting to dispel, through the patient approach of my childhood, the kind of Islamophobia that now had the country by the throat was not going to be enough. Post-9/11 attacks on Muslims in the U.S. and elsewhere were not merely childish taunts.  For the first time in my life, in a country gripped by fear, I believed I was witnessing a shift, en masse, toward an American fundamentalism and ultra-nationalism that reflected a wanton lack of reason, not to mention fact. As a boy in Iran, I had witnessed the dark destination down which such a path could take a country. Now, it seemed to me, in America’s quest to escape the very demons we had sown by our own misadventures in the Middle East, and forsaking the hallmarks of our founding, we risked becoming everything we sought to defeat.  The Boy in the Schoolyard Grown Up  On February 10, 2015, three young American students, Yusor Abu-Salha, Razan Abu-Salha, and Deah Shaddy Barakat, were executed at an apartment complex in Chapel Hill, North Carolina. The killer was a gun-crazy white man filled with hate and described by his own daughter as “a monster.” Those assassinations struck a special chord of sorrow and loss in me. My mom and I cried and prayed together for those students and their families.  The incident in Chapel Hill also awoke in me some version of the righteous indignation I had felt so many years earlier in that smoke-filled courtyard in Iran. I would be damned if I stood by while kids in my country were murdered simply because of their faith. It violated every word of the oath I had taken when I joined the military and desecrated every value I held in my heart as a sacred tenet of our nation. White nationalists and bigots had, by then, thrown down the gauntlet for so much of this, using Islamophobia to trigger targeted assassinations in the United States. This was terrorism, pure and simple, inspired by hate-speakers here at home.  At that moment, I reached out to fellow veterans who, I thought, might be willing to help — and it’s true what they say about soul mates being irrevocably drawn to each other. When I contacted Veterans For Peace , an organization dedicated to exposing the costs of war and militarism, I found the leadership well aware of the inherent dangers of Islamophobia and of the need to confront this new enemy. So Executive Director Michael McPhearson formed a committee of vets from around the country to decide how those of us who had donned uniforms to defend this land could best battle the phenomenon — and I, of course, joined it.  From that committee emerged Veterans Challenge Islamophobia (VCI). It now has organizers in Arizona, Georgia, New Jersey, and Texas, and that’s just a beginning. Totally nonpartisan, VCI focuses on politicians of any party who engage in hate speech. We’ve met with leaders of American Muslim communities, sat with them through Ramadan, and attended their Iftar dinners to break our fasts together. In the wake of the Orlando shooting , we at VCI also mobilized to fight back against attempts to pit the Muslim community against the LGBTQ+ community.  Our group was born of the belief that, as American military veterans, we had a responsibility to call out bigotry, hatred, and the perpetuation of endless warfare. We want the American Muslim community to know that they have allies, and that those allies are indeed veterans as well. We stand with them and for them and, for those of us who are Muslim, among them.  Nationalism and xenophobia have no place in American life, and I, for my part, don’t think Donald Trump or anyone like him should be able to peddle Islamophobia in an attempt to undermine our national unity. Without Islamophobia, there no longer exists a “clash of civilizations.” Without Islamophobia, whatever the problems in the world may be, there is no longer an “us vs. them” and it’s possible to begin reimagining a world of something other than perpetual war.  As of now, this remains the struggle of my life, for despite my intense love for America, some of my countrymen increasingly see American Muslims as the “other,” the enemy.  My Mom taught me as a boy that the only thing that mattered was what was in my heart. Now, with her in mind and as a representative of VCI, when I meet fellow Americans I always remember my childhood experiences with my bullying peers. And I still lay myself bare, as I did then. I give trust to gain trust, but always knowing that these days this isn’t just a matter of niceties. It’s a question of life or death. It’s part of a battle for the soul of our nation.  In many ways, I still consider myself that boy in the school courtyard in Tehran trying to rescue charred pieces of that flag from those trampling feet. It’s just that now I’m doing it in my own country.  Nate Terani is a veteran of the U.S. Navy and served in military intelligence with the Defense Intelligence Agency. He is currently a member of the leadership team at Common Defense PAC and regional campaign organizer with Veterans Challenge Islamophobia . He is a featured columnist with the Arizona Muslim Voice newspaper. (Reprinted from TomDispatch by permission of author or representative)",  # 0
    "Share This Baylee Luciani (left), Screenshot of what Baylee caught on FaceTime (right)  The closest Baylee Luciani could get to her boyfriend, who’s attending college in Austin, was through video online chat. The couple had regular “dates” this way to bridge the 200-mile distance between them. However, the endearing arrangement quickly came to an end after his FaceTime was left on and caught something that left his girlfriend horrified.  Baylee had been discussing regular things with her boyfriend, Yale Gerstein, who was on the other side of the screen on an otherwise average evening. This video chat was not unlike all the others she had with Yale from his apartment near Austin Community College until the 19-year-old girlfriend heard some scratching sounds after FaceTime had been left on.  According to KRON , Baylee was mid-conversation with Yale when scratches at the door caught both of their attention and he got up from his bed, where the computer was, to see who was at his door. He barely turned the handle to open in when masked men entered the room and beat Yale’s face in and slammed him down on his bed while shoving a pistol in his cheek. The intruders didn’t seem to know or care that FaceTime was still on and Baylee’s face, seen in the corner, was watching everything, terrified that she was about to see her boyfriend murdered in front of her, as she watched him fight for his life.  Admitting that she first thought it was a joke, seconds later, she came to the horrid realization that he was being robbed and called her dad, who was at home with her in Dallas, into the room. “I was scared, because they were saying I’m going to blow your head off, I’m going to kill you,” Baylee explained along with the chilling feeling she got when the intruder finally realized the video chat was running and looked right at her in the camera. “I’m like wow… seriously watching an armed robbery happen to somebody that I care about,” she added. Screengrabs of intruder forcing Yale down on his bed while Baylee and her father watch on FaceTime in horror  With a clear view of at least one intruder’s face, Baylee began taking screenshots of the suspect in the act as she and her dad called the police to report what was going on. She got the pictures right in time since, seconds later, the intruder decided to disconnect the computer as he and the suspects took off with thousands of dollars worth of Yale’s music equipment. Although the boyfriend’s life was spared in the traumatizing ordeal for the two of them, he said that the thieves took something from him that can’t be replaced.  “I had just finished my first album as a solo artist,” Yale said. “That’s all lost,” since they took the recordings on the equipment, which means nothing to the thieves and everything to the victim. It’s not often that you hear of FaceTime solving crimes or potentially saving lives, which is what happened in this case. Although it was difficult to watch, Baylee, being there through technology, was an instrumental part in protecting Yale, who hopefully learned that he better take advantage of Texas’ great gun laws and arm himself with more than just a computer.",  # 0
    "A Czech stockbroker who saved more than 650 Jewish children from Nazi Germany has died at the age of 106. Dubbed “Britain’s Schindler,” Nicholas Winton arranged to transport Jewish youngsters from Prague after Germany annexed Czechoslovakia in March 1939. Though the children were originally set to arrive in Britain by plane, the German invasion forced Winton to transport them by train through Germany before they eventually reached England by boat. Winton arranged eight trains, known as the Kindertransports (children’s transports), to evacuate the children, and died on the anniversary of the 1939 departure of the one carrying the largest number of children: 241. Winton was knighted by Queen Elizabeth II in 2003 for his efforts, despite keeping it secret for nearly 50 years.",  # 1
    "Hillary Clinton and Donald Trump made some inaccurate claims during an NBC “commander-in-chief” forum on military and veterans issues:  • Clinton wrongly claimed Trump supported the war in Iraq after it started, while Trump was wrong, once again, in saying he was against the war before it started.  • Trump said that President Obama set a “certain date” for withdrawing troops from Iraq, when that date was set before Obama was sworn in.  • Trump said that Obama’s visits to China, Saudi Arabia and Cuba were “the first time in the history, the storied history of Air Force One” when “high officials” of a host country did not appear to greet the president. Not true.  • Clinton said that Trump supports privatizing the Veterans Health Administration. That’s false. Trump said he supports allowing veterans to seek care at either public or private hospitals.  • Trump said Clinton made “a terrible mistake on Libya” when she was secretary of State. But, at the time, Trump also supported U.S. action that led to the removal of Moammar Gadhafi from power.  • Trump cherry-picked Clinton’s words when he claimed Clinton said “vets are being treated, essentially, just fine.” Clinton had said the problems in the Department of Veterans Affairs were not as “widespread” as some Republicans claimed, but she went on to acknowledge problems, including the issue of wait times for doctors.  The forum, sponsored by NBC News and the Iraq and Afghanistan Veterans of America, was held Sept. 7 at the Intrepid Sea, Air & Space Museum in New York City. Today show host Matt Lauer, and members of the military and veterans in the audience, questioned the candidates separately.  Trump said he “was totally against the war in Iraq,” while Clinton claimed that he supported the Iraq War before and after it started. The facts don’t support either candidate’s strong assertions.  Our review of Trump’s statements before and after the Iraq War started found no evidence that Trump opposed the war before it started. In fact, he expressed mild support for invading Iraq when asked about it on the Howard Stern radio show on Sept. 11, 2002 — about six months before the war started.  Stern asked Trump if he supported a war with Iraq, and Trump responded, “Yeah, I guess so.”  In the NBC commander in chief forum, Trump cited an Esquire article that appeared in August 2004 to show his opposition to the war. But that article appeared 17 months after the war started.  As for Clinton, who as a senator voted in October 2002 to authorize the war in Iraq, the Democratic nominee claimed that Trump “supported it before it happened, he supported it as it was happening and he is on record as supporting it after it happened.”  But just as there is no evidence that Trump opposed the Iraq War before it started, the Clinton campaign offered no evidence that Trump supported the war “after it happened.”  The Clinton campaign cited Trump’s interview on March 21, 2003, with Neil Cavuto of Fox Business just two days after the war started.  Cavuto asked Trump about the impact of the war on the stock market. Trump said the war “looks like a tremendous success from a military standpoint,” and he predicted the market will “go up like a rocket” after the war. But Cavuto does not ask Trump whether the U.S. should have gone to war with Iraq or whether he supports the war, and Trump doesn’t offer an opinion.  As early as July 2003, Trump expressed concern on Hardball with Chris Matthews about money being spent in Iraq rather than in the U.S. Two months later, Trump told MSNBC’s Joe Scarborough, “I guess maybe if I had to do it, I would have fought terrorism but not necessarily Iraq.”  Clinton invited her audience to read Trump’s comments on the Iraq War. They can read our timeline, “Donald Trump and the Iraq War.”  Trump said President Obama set a “certain date” for withdrawing troops from Iraq, but that date was actually set by President George W. Bush.  NBC’s Matt Lauer asked Trump about his tendency to respond, when pushed for details on his military proposals, that he’s not going to give details because he wants to be “unpredictable.” Trump responded, “Absolutely,” and went on to criticize Obama for revealing the withdrawal date.  As we said then, Republicans and Democrats disagree on whether Obama or Bush is to blame for withdrawing all combat troops from Iraq at the end of 2011. But that date was set when Bush signed the Status of Forces Agreement on Dec. 14, 2008. It said: “All the United States Forces shall withdraw from all Iraqi territory no later than December 31, 2011.”  In the NBC forum, Trump also called the withdrawal of troops “a terrible decision.” As we’ve explained before, Condoleezza Rice, Bush’s secretary of State, later wrote that Bush wanted an agreement for a residual force to remain, but Iraqi Prime Minister Nouri al-Maliki objected.  Once Obama took office in January 2009, he had three years to renegotiate the deal, which his administration tried to do, to leave a residual American troop force. But Maliki still didn’t agree. Negotiations broke down in October 2011 over the issue of whether U.S. troops would be shielded from criminal prosecution by Iraqi authorities. Whether Obama did enough is a matter of opinion: His then defense secretary, Leon Panetta, later wrote that the president didn’t press hard enough for a deal. But some experts say Iraq was more closely aligned at the time with Iran and there wasn’t a deal to be made with Maliki.  So, both presidents had a role in the withdrawal of troops. But Trump wrongly said that Obama was the one who set a “certain date” for withdrawal and let U.S. enemies know about it, when that date was set before Obama was sworn in.  It’s worth noting that Trump said in a March 16, 2007, interview on CNN that the troops should be withdrawn quickly from Iraq.  Trump said that Obama’s visits to China, Saudi Arabia and Cuba were “the first time in the history, the storied history of Air Force One” when “high officials” of a host country did not appear to greet the president.  That’s not true. Other presidents have encountered similar low-key greetings on foreign trips aboard the presidential aircraft.  Trump referred to the fact that Cuba’s president, Raul Castro, did not greet Obama at the airport on his historic visit to Cuba in March, that Saudi Arabia’s King Salman did not meet Air Force One at the start of Obama’s trip to Riyadh in April, and he referred to China’s handling of the president’s arrival in Hangzhou last Saturday for a Group of 20 meeting.  Whether or not those arrivals constituted snubs of a U.S. president as Trump claims is a matter of debate. But Trump is wrong on the facts when he claims it has not happened before. It has.  In 1984, for example, Ronald Reagan landed in Beijing and was received by China’s foreign minister rather than the president, whom he met only later. Similarly, on a 1985 trip to West Germany, Reagan was met by the foreign minister and not Chancellor Helmut Kohl.  These and other examples were dug up by our friend Glenn Kessler, the Washington Post‘s “Fact Checker,” who researched a Trump claim in April that Cuba’s and Saudi Arabia’s handling of Obama’s visits were “without precedent.” Kessler said of Trump, “once again he’s wrong, wrong, wrong.”  Kessler also noted that during Richard Nixon’s historic 1972 visit to China he was greeted at the airport by the country’s number two man, Premier Zhou Enlai. His boss, Chairman Mao, didn’t even agree to meet with Nixon until after he had arrived at a guest house.  Clinton said that her plan to overhaul the Veterans Health Administration would not include privatization, which she said Trump supports.  But Trump refuted that statement when it was his turn to discuss his plan to help veterans. “I would not do that,” Trump said, referring to Clinton’s claim that he supports privatization.  Trump’s campaign published “The Goals Of Donald J. Trump’s Veterans Plan” on its website last October. It doesn’t call for the VA to be completely privatized.  One of the biggest changes that plan would make to the current VA health care system is allowing veterans to get care at any non-VA medical center that accepts Medicare.  “Under a Trump Administration, all veterans eligible for VA health care can bring their veteran’s ID card to any doctor or care facility that accepts Medicare to get the care they need immediately,” the plan states.  “The power to choose will stop the wait time backlogs and force the VA to improve and compete if the department wants to keep receiving veterans’ healthcare dollars,” the plan says.  Trump’s proposal would seemingly go further than the Non-VA Medical Care Program, which allows eligible veterans to access care outside of the VA under certain circumstances, such as when VA medical centers cannot provide services. The program requires pre-approval for veterans to receive care at a non-VA facility in non-emergency situations.  Trump’s proposal would also go further than the bipartisan Veterans Choice Act of 2014 that President Obama signed into law, creating a temporary program, separate from the Non-VA Medical Care Program, that allows eligible veterans to receive health care at a non-VA facility if they would have to wait more than 30 days for an appointment at a VA medical center, or if they live more than 40 miles from the nearest VA hospital.  Trump stuck to the idea of allowing veterans to choose between public and private hospitals when he released his most recent “Ten Point Plan To Reform The VA” in July.  Point 10 of the plan says: “Mr. Trump will ensure every veteran has the choice to seek care at the VA or at a private service provider of their own choice. Under a Trump Administration, no veteran will die waiting for service.”  Trump reinforced that part of his plan during the NBC News forum as well.  To be clear, Trump supports giving veterans a choice between VA hospitals and private ones. That’s not the same thing as supporting the complete privatization of the system that provides care to veterans.  Trump criticized Clinton for making “a terrible mistake on Libya” when she was secretary of State. But, at the time, Trump also supported U.S. action that led to the removal of Moammar Gadhafi from power.  Trump made his claim in response to a question posed by Lauer on whether Trump will be “prepared on Day One,” if elected president, to tackle “complex national security issues.”  This isn’t the first time Trump has ignored his past support for the U.S. intervention in Libya.  During the 10th GOP debate, Trump said he had “never discussed that subject” when Sen. Ted Cruz called him out on supporting U.S. action in the country. But, as we wrote, Trump said in 2011 that the U.S. should go into Libya “on a humanitarian basis” and “knock [Gadhafi] out very quickly, very surgically, very effectively and save the lives.”  Trump made that comment in a video posted to his YouTube channel in February 2011:  Even though Trump now says Clinton’s support for intervention in Libya was a “terrible mistake,” it doesn’t change the fact that five years ago he supported Gadhafi’s removal.  Trump twisted Clinton’s words when he claimed Clinton said “vets are being treated, essentially, just fine.” Clinton said the problems in the Department of Veterans Affairs were not as “widespread” as some Republican supporters of privatization of the VA claim, but she went on to acknowledge problems in the VA system — including the issue of wait times for doctors — and what she would do to address them.  Trump highlighted the issue of wait times to see a doctor as “one of the big problems” in the VA, and then suggested Clinton doesn’t think the VA has problems.  Lauer interrupted, noting that Clinton “went on after that and laid out a litany of problems within the VA.”  Trump insisted his version was accurate, adding, “I’m telling you … she said she was satisfied with what was going on in the Veterans Administration.”  That’s not accurate. The comments in question from Clinton came during an interview with MSNBC’s Rachel Maddow on Oct. 23, 2015. Maddow asked about talk among some Republicans of abolishing the VA and privatizing it. “The reason they are able to propose something that radical is because the problems at the VA seem so intractable,” Maddow said.  Maddow asked if Clinton had any “new ideas for trying to fix” the VA. Here was Clinton’s response, with the part Trump is referring to in bold.  Clinton accused Republicans of underfunding the VA because they “want it to fail” so they can privatize it.  Clinton added, “But we have to be more creative about trying to fix the problems that are the legitimate concern, so that we can try to stymie the Republican assault.”  Indeed, the Clinton campaign website states that Clinton wants to “fundamentally reform veterans’ health care to ensure access to timely and high quality care.” The campaign says Clinton “was outraged by the recent scandals at the VA, and as president, she will demand accountability and performance from VA leadership.” The site specifically mentions Clinton’s dissatisfaction that “[m]any veterans have to wait an unacceptably long time to see a doctor or to process disability claims and appeals” and promises she will “[b]uild a 21st-century Department of Veterans Affairs to deliver world-class care.”  Trump cherry-picked the part of Clinton’s response that said problems in the VA have “not been as widespread as it has been made out to be,” to make the blanket claim that Clinton is “satisfied with what was going on in the Veterans Administration” and that “vets are being treated, essentially, just fine.” But Trump is leaving out the parts of Clinton’s answer that acknowledged problems in the VA — including the wait time issue Trump highlighted as one of his biggest concerns.",  # 1
    "Iranian negotiators reportedly have made a last-ditch push for more concessions from the U.S. and five other world powers as talks on the fate of Iran's nuclear program come down to the final days before a crucial deadline.  The New York Times reported late Sunday that Tehran had backed away from a tentative promise to ship a large portion of its uranium stockpile to Russia, where it could not be used as part of any future weapons program. Western officials insisted to the paper that the uranium did not have to be sent overseas, but could be disposed of in other ways.  The new twist in the talks comes just two days before the deadline for both sides to agree on a framework for a permanent deal. The final deadline for a permanent deal would not arrive until the end of June.  However, if Iran insists on keeping its uranium in the country, it would undermine a key argument made in favor of the deal by the Obama administration. The Times reports that if the uranium had gone to Russia, it would have been converted into fuel rods, which are difficult to use in nuclear weapons. It is not clear what would happen to the uranium if it remained in Iran.  The Associated Press reported Sunday that Iran's position had shifted from from demanding that it be allowed to keep nearly 10,000 centrifuges enriching uranium, to agreeing to keep 6,000. Western officials involved in the talks told the Associated Press that Tehran may be ready to accept an even lower number.  The United States and its allies want a deal that extends the time Iran would need to make a nuclear weapon from the present two months to three months to at least a year. However, The Times reported Sunday that a paper published by Olli Heinonen, former head of inspections for the U.N.'s nuclear watchdog, estimated that Iran could still develop a nuclear weapon in seven or eight months with around 6,500 centrifuges.  Tehran says it wants to enrich uranium only for energy, science, industry and medicine. But many countries fear Iran could use the technology to make weapons-grade uranium.  Officials told the Associated Press that another main dispute involved the length of an agreement. Iran, they said, wants a total lifting of all caps on its activities after 10 years, while the U.S. and the five other nations at the talks — Russia, China, Britain, France and Germany — insist on progressive removal after a decade.  A senior U.S. official characterized the issue as lack of agreement on what happens in years 11 to 15. The official spoke on condition of anonymity in line with State Department rules on briefing about the closed-door talks.  Limits on Iran's research and development of centrifuges also were unresolved, the Western officials said.  Tehran has created a prototype centrifuge that it says enriches uranium 16 times faster than its present mainstay model. The U.S. and its partners want to constrain research that would increase greatly the speed of making enough weapons-grade uranium for a bomb, once limits on Iran's programs are lifted.  One official said Russia opposed the U.S. position that any U.N. penalties lifted in the course of a deal should be reimposed quickly if Tehran reneged on any commitments.  Both Western officials said Iran was resisting attempts to make inspections and other ways of verification as intrusive as possible.  There was tentative agreement on turning a nearly-finished reactor into a model that gives off less plutonium waste than originally envisaged. Plutonium, like enriched uranium, is a path to nuclear weapons.  Iran and the U.S. were discussing letting Iran run centrifuges at an underground bunker that has been used to enrich uranium. The machines would produce isotopes for peaceful applications, the officials said.  With the Tuesday deadline approaching and problems remaining, U.S. Secretary of State John Kerry canceled plans Sunday to return to the United States for an event honoring the late U.S. Sen. Edward Kennedy. French Foreign Minister Laurent Fabius and Frank-Walter Steinmeier, his German counterpart, scratched planned trips to Kazakhstan.  Kerry has been in discussions with Iranian Foreign Minister Mohammad Javad Zarif since Thursday.  The Associated Press contributed to this report.  Click for more from The New York Times.",  # 1
    "CEDAR RAPIDS, Iowa — “I had one of the most wonderful rallies of my entire career right here in 1992,” Bill Clinton said by way of opening to the crowd of more than 1,100 on Saturday night.  Two days before the Iowa caucuses, Cedar Rapids tried to deliver that same old feeling to his wife, Hillary Clinton.  In the crowd, one woman held a sign that said “227 years of men. It’s HER turn!\" Some carried signs and books. Others had traveled from as far as Missouri. They had waited hours, even after the fire marshal told them there was no more room inside the high school gymnasium.  The restive crowd chanted slogans and buzzed with anticipation until finally Bill, Hillary and Chelsea Clinton all appeared on stage hand-in-hand, an hour behind schedule. As they roared, Hillary Clinton beamed.  It has been a long slog in Iowa for the Clinton campaign, which has struggled mightily to shake the label that its supporters can’t muster the enthusiasm of its rival’s backers. As the caucuses near, and with the help of a former president, the energy level at her events are notably dialing up.  \"He's a charismatic speaker,\" said Cigi Ross, 31. \"In general, I'd say he's a bigger draw for people.\"  Monday night will put the campaign's months of their work to the test. Can the campaign’s organization bring out their supporters? Can the candidate energize voters?  Clinton, who seemed to draw on the higher-than-usual energy, stood at the center of it all and delivered a confident closing statement.  “What we need is a plan, and a commitment,” Clinton said at the top of her voice.  “And me, yes, thank you,” Clinton finished.  Eight years later, Clinton is in Iowa once again facing what could be a nail-biting conclusion of a hard-fought campaign. Clinton acknowledges that it isn’t just her campaign that has changed since her devastating loss here in her last run, she too has changed — and improved, she told CNN on Saturday.  \"I think I am a different, and perhaps a better, candidate, so I hope that also shows,\" Clinton said in an interview with the network that morning.  Days ago, Iowa seemed to be slipping from her grasp, but campaign aides are feeling more confident now. A slew of positive news, endorsements, and the latest poll from the Des Moines Register and Bloomberg News indicate the bleeding has at least slowed. That poll -- considered the gold standard in Iowa -- gave Clinton a slim lead over her rival Vermont Sen. Bernie Sanders.  If Democrats are wary of political dynasties, they didn’t show it. Spotting someone in the audience carrying Chelsea Clinton's book, her father remarked: \"Thank you, young woman, for holding up her book.\"  Bill Clinton, who has spent days crisscrossing the state on his wife's behalf, has settled easily into this role as booster-in-chief. He lays off the policy, leaving that to his wife. He focuses instead on what he knows \"about the job.\"  “There are certain, almost intangible qualities that determine whether a president succeeds or not,” Clinton said, his voice raspy, even and low. \"You need a sticker. A sticker: someone who won’t quit on you.”  \"She’s the best at that I’ve ever known,” he added.",  # 1
    "Donald Trump’s organizational problems have gone from bad to worse to flat-out embarrassing. Here’s Politico with the play-by-play from this weekend’s Colorado GOP convention, the latest scene of Trump’s delegate-securing failure:  Unlike in most other states, Republicans in Colorado scrapped plans for a more traditional primary or caucus to award its delegates to this summer’s national convention in Cleveland. Instead, the state GOP selected three delegates from each of the state’s seven congressional districts at individual contests in the days leading up to the state convention, and then the remaining 13 delegates at the statewide event this past weekend. It was a rather convoluted process that favored campaigns that understood the rules and that had the ground game necessary to take advantage of them—or, put another way, not the Trump campaign.  At the first two district-level contests, the billionaire’s team showed up without an approved list of delegates to pass out to attendees, leaving supporters unsure of whom to back. And then at last Thursday’s contest, two of the three delegates on the Trump campaign’s list weren’t actually on the official ballot since they failed to pay the necessary registration fees.  This past weekend’s mix-up, though, was an even bigger embarrassment since at least one of the seven misnumbered names on the Trump-sanctioned list lined up with a Cruz-supporting delegate, according to NBC News. (Even without that mix-up, though, Cruz would have likely swept the contest anyway given his well-oiled delegate-selecting machine.) The Trump camp, though, appeared to blame the state GOP for the mistake, pointing to discrepancies between the delegate guides posted on the party's website and the printed materials distributed at the event. “We'll do whatever it takes to protect the legitimacy of our support in Colorado,” Trump aide Alan Cobb told NBC, suggesting the campaign may challenge the results. “Clearly there are some serious issues with the ballot and balloting.”  Trump remains the favorite to arrive at the convention with the most delegates to his name, but he’s far from assured of the majority of delegates he’d need to win the nomination on the first ballot. Given that reality, the GOP front-runner recently retooled his campaign to address his clear weaknesses in the under-the-radar battle to send loyal delegates to the convention. If those efforts don't start paying dividends soon, though, Trump very well may arrive in Cleveland with the most delegates—and leave without the nomination.",  # 1
    "Click Here To Learn More About Alexandra's Personalized Essences Psychic Protection Click Here for More Information on Psychic Protection! Implant Removal Series Click here to listen to the IRP and SA/DNA Process Read The Testimonials Click Here To Read What Others Are Experiencing! Copyright © 2012 by Galactic Connection. All Rights Reserved.  Excerpts may be used, provided that full and clear credit is given to Alexandra Meadors and www.galacticconnection.com with appropriate and specific direction to the original content. Unauthorized use and/or duplication of any material on this website without express and written permission from its author and owner is strictly prohibited. Thank you.  Privacy Policy  By subscribing to GalacticConnection.com you acknowledge that your name and e-mail address will be added to our database. As with all other personal information, only working affiliates of GalacticConnection.com have access to this data. We do not give GalacticConnection.com addresses to outside companies, nor will we ever rent or sell your email address. Any e-mail you send to GalacticConnection.com is completely confidential. Therefore, we will not add your name to our e-mail list without your permission. Continue reading... Galactic Connection 2016 | Design & Development by AA at Superluminal Systems Sign Up forOur Newsletter  Join our newsletter to receive exclusive updates, interviews, discounts, and more. Join Us!",  # 0
    "October 31, 2016 at 4:52 am  Pretty factual except for women in the selective service. American military is still voluntary only and hasn't been a draft since Vietnam war. The comment was made by a 4 star general of the army about drafting women and he said it to shut up liberal yahoos.﻿",  # 0
    "Killing Obama administration rules, dismantling Obamacare and pushing through tax reform are on the early to-do list.",  # 1
    "As more women move into high offices, they often bring a style and approach that is distinct from men. But do they make better leaders?  Democratic women of the US Senate stand on stage during the final night of the Democratic National Convention in Philadelphia (top). There are 20 women in the Senate – 14 Democrats and six Republicans.  Kelly Ayotte and Joni Ernst take the floor, two dynamic young US senators before a town hall of older military veterans. Between them, they represent a raft of firsts: Senator Ayotte is New Hampshire’s first female attorney general and first female Republican senator. Senator Ernst is Iowa’s first woman ever elected to either house of Congress and the first female military veteran to serve in the Senate.  The gender dimension does not go unnoticed. Please welcome “our beautiful senators,” says the veteran who introduces them. Ayotte mentions Ernst’s military service – she’s also a war veteran. But in most ways, the woman thing doesn’t matter. This is just like any other political town hall in campaign season – one senator trying to help another who’s locked in a tight reelection race.  Yet the stakes are high. Republican control of the US Senate could hinge on Ayotte’s ability to fend off her Democratic challenger, Gov. Maggie Hassan. And across the country, women figure prominently in the Democrats’ strategy to retake the Senate, nominating women in six of the 11 competitive races.  Then there’s Hillary Clinton, the first woman to win a major party’s presidential nomination in the United States.  It’s enough to call 2016 Year of the Woman 2.0, following the original, 1992, when female representation in the Senate jumped from two to six. Today, female senators number 20 out of 100, with potential to reach 24, depending on how the political winds are blowing by Election Day.  Gender diversity among major world powers is also rising, with the ascension of Theresa May to the British prime ministership. If Mrs. Clinton wins in November, three of the world’s top five economies will be headed by women. (German Chancellor Angela Merkel is the other.)  But the numbers and the “firsts” invite a deeper question: Do women really lead differently? From the halls of government to corporate boardrooms, it’s a burning question.  “I think we’re good listeners, and I think that helps,” says Ayotte in an interview. “I don’t want to say my male colleagues aren’t, because plenty of them are. But I think that we listen, and so we’re picking up on where we see the common ground with other people.”  On that point, there is bipartisan agreement.  “There is good research that shows women tend to have different leadership styles,” Democratic Sen. Jeanne Shaheen, New Hampshire’s other senator, says in an interview. “We tend to be more inclusive, we’re less autocratic in our decisionmaking. We like consensus, we like to get people around the table, and so I think that has made a difference.”  Of course, there’s also the example of Rep. Nancy Pelosi (D) of California, the first woman speaker of the House (2007-11). During her tenure, she was known for her ability to wield power – and in a highly partisan way. More than with previous speakers, major legislation came out of her office, not the committees. She was (and is) a master fundraiser for Democrats writ large, a skill she has used to great effect in keeping her members in line.  And as a Roman Catholic woman, Congresswoman Pelosi demonstrated at a crucial moment the power of networking. In 2010, when House passage of the Affordable Care Act appeared in doubt over the abortion issue, she tapped connections her male colleagues had never heard of – two associations of nuns, whose support for the bill proved pivotal.  The research on women’s leadership style has been extensive – and mixed. One academic study, released in 2013, found that women are more attracted to cooperation than men. The reason is that men tend to have greater confidence in their abilities, while women tend to be more optimistic about their prospective teammates’ abilities, according to the study by Peter Kuhn and Marie-Claire Villeval.  New research, released in August, looked specifically at gender differences in the US House of Representatives and found “little evidence to suggest women are inherently more cooperative or bipartisan.” The only difference came with Republican women, who tend to recruit more cosponsors on legislation, including more from the opposite party. That tendency was most pronounced on so-called women’s issues, such as education and social welfare.  “We interpret these results as evidence that cooperation is mostly driven by a commonality of interest, rather than gender per se,” write the authors, Stefano Gagliarducci and M. Daniele Paserman.  The study examined 20 years of data ending in 2008. Since then, female representation in Congress has ticked up a notch, reaching 20 percent in the Senate and nearly that in the House – the rough threshold for a perceived “critical mass” of representation in which women can show perceptible influence.  Indeed, since 2013, female legislators point to multiple examples of how the women of the Senate, in particular, have been instrumental in breaking through congressional gridlock. The best-known example is the end of the 2013 government shutdown, when women senators from both parties met privately over dinner and fashioned a compromise that would form the nucleus of the final deal.  More recently, the women of the Senate – all 20 of them, dubbed “the sisterhood” – backed landmark legislation aimed at combating sex trafficking, which President Obama signed into law in May 2015.  When asked for other examples, Senator Shaheen cites the 2013 reauthorization of the Violence Against Women Act. “It was considered dead,” she says, “and because all the women [senators] got on board and pushed it, we were able to get it through.”  Shaheen also notes the growing number of women who serve in top spots on committees, such as the Senate Energy and Natural Resources Committee, with Sen. Lisa Murkowski (R) of Alaska as chair and Sen. Maria Cantwell (D) of Washington as the ranking member. “It looks like we may get an energy bill done this year, and I think that speaks to their ability to work together, and to be flexible, and move our colleagues in the House,” Shaheen says.  Behind all this female power are the women senators’ regular dinners, a chance to kick back and talk about whatever – their lives, their work, their male colleagues.  “We go by Mikulski’s rules – that what happens in those dinners stays in those dinners,” says Shaheen, referring to Sen. Barbara Mikulski (D) of Maryland, the longest-serving woman in congressional history, who is about to retire. “But yeah, they’ve proved to be a great opportunity to build relationships.”  As it happens, Shaheen had hosted 12 of her female colleagues the night before at her office. On the menu: lobsters and clam chowder. That much she will reveal. Sen. Susan Collins (R) of Maine was there, Shaheen says, “and we had a back and forth about whether the lobsters came from Maine or New Hampshire.”  In 2013, after the Senate’s 20 women helped end the government shutdown, Sen. John McCain (R) of Arizona wondered out loud: “Imagine what they could do if there were 50 of them.”  That may sound logical, but the reality isn’t so simple. While women do bring a distinctive perspective to policy, based on their life experiences, that doesn’t necessarily make them less partisan, experts say. Some of the most polarizing issues are so-called women’s issues, including health care, abortion, and equal pay.  “As long as women’s issues constitute a prominent division between the parties, there will be little bipartisan collaboration among women on these policies,” Michele Swers, a political scientist at Georgetown University, wrote in The Washington Post in 2014.  Ms. Swers also notes that partisan sorting has made the South – a region less hospitable to women candidates – the stronghold of the Republican Party, while the more moderate brand of Republicanism that predominates in parts of the country more willing to elect women has declined.  And because women tend to occupy the “leftward wing” of their respective parties, Democratic women in particular are less inclined to compromise, the Gagliarducci-Paserman study found.  Still, none of this means women shouldn’t be more rigorously recruited for public office, say leaders of both parties.  “What you need is more good people, and the process of excluding women has excluded good people from service,” says former Gov. John Sununu (R) of New Hampshire. “So what you need to do is level the playing field.”  In the business world, boosting female representation on corporate boards – now near 20 percent – has been a long-held goal. Advocates cite benefits to the bottom line. In a famous study, Catalyst Inc., a nonprofit that promotes women leaders in the workplace, found that corporate boards with the highest female representation attained “significantly higher financial performance” than those with the lowest representation.  But whether in the private sector or in government, women as top executives are even rarer than women legislators. Only six of the nation’s 50 governors are women, three Republicans and three Democrats. Among world leaders, fewer than two dozen are female. Among Fortune 500 companies, only 21 chief executive officers – or 4.2 percent – are women.  While legislating is an inherently collaborative process, and therefore seems to play to women’s strength, executive roles hew more toward public expectations of how men should behave, i.e., authoritatively.  “I think the more people see women in executive positions, the more they will see that as a normal course of business, and it’s taken longer for that not just in politics but in the boardroom,” says Shaheen.  But that becomes a Catch-22: Women don’t step up because they don’t see other women in those positions. “Some would call it a confidence gap,” says Liz Shuler, the No. 2 leader at the AFL-CIO. “We need to do more to increase the leadership skills-building opportunities, so that it becomes second nature for women to step up.” (See interview with Ms. Shuler here.)  But do women executives really lead differently from men? Clinton echoes other women leaders on this point: “I just think women in general are better listeners, are more collegial, more open to new ideas and how to make things work in a way that looks for a win-win outcome,” she told Time magazine in January.  Meta-analyses have found that women leaders, on average, are “more likely to be democratic, collaborative, and participative” than their male counterparts, who are more likely to be “autocratic and directive” in their approach, writes Alice H. Eagly at TheConversation.com.  Of course, there are exceptions. Margaret Thatcher of Britain and Golda Meir of Israel were both known for being tough and assertive. Today, Apple CEO Tim Cook is known for his team-oriented style. But for women aspiring to leadership posts, the challenge is to overcome societal expectations for how women are “supposed to” act – i.e., nice and nonconfrontational – while still projecting authority. Clinton has faced this “double bind” in both of her presidential campaigns.  New Hampshire has a strong record of female leadership. The governor, both US senators, and one of the state’s two House members are women. The chief justice of the state Supreme Court is a woman. The chair of the state GOP is a woman.  In 2008, Granite Staters made history by electing the nation’s first majority-female state legislative body, in the state Senate. In 2013, the state made history again by sending an all-female congressional delegation to Washington. That could happen again in January.  What is it about New Hampshire that breeds women leaders? Start with a culture of independent-mindedness, and an enormous “citizens legislature” – 400 in the House, 24 in the Senate – in a state with only 1.3 million people. Members are paid just $100 a year, plus mileage. Sooner or later, the joke goes, everybody ends up serving.  “We have so much local self-governance, women have the opportunity to try things out, to see how they like public service, and then they discover that they like it and that they’re good at it, and one thing leads to the next,” says Governor Hassan in an interview in the State House.  Hassan got her start advocating for her disabled son, which caught the notice of then-Governor Shaheen, who appointed her to a state educational funding commission. That led to regular interactions with state legislators – and an introduction to her next mentor, state Sen. Sylvia Larsen, who encouraged Hassan to run for state Senate. She spent six years there, eventually rising to majority leader, losing reelection, then staging a comeback by winning the governorship.  The common denominator for most women in politics, it seems, is mentors. Ask Ayotte, New Hampshire’s junior senator, about Ruth Griffin, and her eyes light up. “I love Ruth Griffin. She’s my mentor!” Ayotte gushes, referring to the nonagenarian grand dame of the New Hampshire Republican Party. (See interview with Mrs. Griffin here.)  When Griffin served on the state Executive Council – an elected board that serves as a check on the governor – she advocated strongly for Ayotte, insisting the Democratic governor reappoint the Republican Ayotte as attorney general.  Shaheen, now a mentor to others, cites as her inspiration Marilla Ricker, the first woman to run for governor of New Hampshire – before women even had the right to vote. Shaheen also speaks fondly of the late Susan McLane, a Republican state senator “who was always very good to me.” (Ms. McLane’s daughter is Rep. Ann McLane Kuster (D) of New Hampshire.) Shaheen broke two gender barriers in N.H., first with her election as governor (1996), then as a US senator (2008).  Men, too, have championed women politicians here. At a recent Republican women’s chili fest in Stratham, N.H., Mr. Sununu recalls fighting to make Vesta Roy the first woman state Senate president, in 1983. “There’s that classic picture of me standing at the state Senate door, cracked open, peeking in to make sure the 13 votes I had lined up for her stayed with her,” Sununu says.  Whether New Hampshire’s success at electing women to higher office can be duplicated elsewhere remains an open question. Only two other states have two female US senators, and five others have female governors. New Hampshire is no longer the champ at electing women state legislators; that crown belongs to Colorado, with 42 percent. The top 10 states for female representation in state legislatures are mostly Northern, and the bottom 10 mostly Southern, suggesting cultural factors.  Overall, women hold a higher percentage of state legislative seats (24.6 percent) than seats in Congress (19.4 percent, House and Senate combined). At least the female “farm team” has more players than the “major league,” though those numbers still fall far short of matching the overall female population of the country, 50.8 percent.  For many women, reaching parity with men in government – and in business, the judiciary, the labor movement, and other spheres of life – is a deeply held goal. But a quick glance at the graphs charting female representation in state and national legislatures reveals a stark truth: Growth has nearly hit a plateau.  If progress continues at the current rate of change since 1960, women will not achieve equal representation in Congress until 2117, according to the Institute for Women’s Policy Research. Women’s representation in state legislatures, which more than quintupled between 1971 and 2015, has also essentially plateaued. What’s going on?  “It’s not that women are running in droves and losing,” says Debbie Walsh, director of the Center for American Women and Politics at Rutgers University in New Brunswick, N.J. “We know that when women run, they win at about the same rate as men do.”  Women also do just as well in fundraising as men, even if they have to work harder to raise the same amount. Part of the problem, Ms. Walsh says, comes in the recruitment process. Much of the recruiting is done by white men, and they recruit who they know – other white men.  That is particularly so among Republicans: Only 17 percent of Republican state lawmakers are women, a rate that hasn’t changed since the early 1990s. In contrast, Democrats doubled their rate of female state representation during the same period from 15 percent to 30 percent.  In Congress (both houses), the disparity is even more stark: Only 9 percent of Republicans are female, versus 33 percent of Democrats. In the Senate alone, only six of the 20 women members are Republican.  Walsh cites several factors behind this gap: Republican women are perceived to be more moderate than their male counterparts, and can have a harder time making it out of the primaries, where turnout is low and skews conservative. The GOP also has no counterpart to Emily’s List – a well-funded political action committee that helps pro-abortion-rights Democratic women get elected. And then there’s the simple fact that more women identify as Democrats, so their recruitment pool is bigger.  In New Hampshire, where both legislative chambers are controlled by the Republicans, Democratic women legislators still outnumber Republican women legislators, 73 to 49. State GOP chair Jennifer Horn says the party – nationally, not just in New Hampshire – needs a message that’s more inclusive.  “I’m being diplomatic – I think there are times when some of the folks in our party are not as sensitive to the fact that men and women see the world differently,” says Ms. Horn. “When we’re talking about jobs and the economy, for example, we need to be cognizant of the fact that women are an economic engine in and of themselves.”  And then there’s the fact that, in both parties, women are less likely than men to run for office without being recruited.  In the Monitor’s conversations with Ernst and Ayotte, both mentioned how they were asked several times to run for the Senate before saying yes – and both brought up family issues. Without using the word “multitasking,” they both show that they have mastered that classic skill of the working mom.  “It’s about getting the message out about empowering women and saying, you can do this,” says Ernst. “It is important that you have a supportive family, even more so for women. I have a teenage daughter right now – we’re looking at colleges – and sometimes it’s hard for a woman to be away. But there are ways of making it work.”  Before running for Senate, Ayotte had been asked a couple of times to run for Congress, but declined – in part, she says, because she was pregnant, but also because she had plans in her work as a prosecutor.  Ayotte then excuses herself. “I’ve got to pick up my daughter before 5:30 or I’m in trouble!”",  # 1
    "Shocking! Michele Obama & Hillary Caught Glamorizing Date Rape Promoters First lady claims moral high ground while befriending rape-glorifying rappers Infowars.com - October 27, 2016 Comments  Alex Jones breaks down the complete hypocrisy of Michele Obama and Hillary Clinton attacking Trump for comments he made over a decade ago while The White House is hosting and promoting rappers who boast about date raping women and selling drugs in their music.  Rappers who have been welcomed to the White House by the Obama’s include “Rick Ross,” who promotes drugging and raping woman in his song “U.O.N.E.O.”  While attacking Trump as a sexual predator, Michelle and Hillary have further mainstreamed the degradation of women through their support of so-called musicians who attempt to normalize rape. NEWSLETTER SIGN UP Get the latest breaking news & specials from Alex Jones and the Infowars Crew. Related Articles",  # 0
    "0  Hillary Clinton has barely just lost the presidential election and here she is already getting herself caught in another one of her tangled web of lies.  The day after losing, a picture was posted of her taking her dog out for a walk. The picture was reportedly from a random hiker she ran into and showed that she was getting back to public life, or that’s what she wants you to believe.  Now, I want you to do me a favor and take a very close look at this picture, specifically the “hiker” she met. Memorize her face…  Got it? Okay, good.  Now what I am about to show you is something Hillary Clinton was hoping nobody else would notice. Tell me if the girl in the 2nd picture, taken in the 2000’s, reminds you of anyone:  Yeah, it is the EXACT SAME GIRL!  Her name is Margot Gerster and her mother just happens to be one of the big fundraisers for Hillary. That’s actually where the older picture was taken.  Wow! That doesn’t seem random at all, does it?  It looks like this is yet another example of Hillary Clinton trying to mislead the public using her inner circle and pretending like she doesn’t know them.  All I gotta say is, give it the Hell up, Hillary! You lost, nobody needs to see your “chance encounters” or other promotional propaganda anymore. This is exactly why Americans ended up hating you!  Oh well, I guess you really cannot teach an old dog new tricks. Well, in that case all that’s left to do is share this all across Facebook and the internet and expose yet another one of her Serial Lies! ",  # 0
    "Washington (CNN) For months, the White House and Congress have wrangled over a bill that would give lawmakers a greater say in the Iran nuclear deal the administration is hammering out along with other world powers.  And now, less than two weeks after a framework agreement with Tehran was reached, the Senate Foreign Relations Committee is planning to take up the measure.  On Tuesday, the committee will consider amendments to Tennessee Republican Sen. Bob Corker's Iran Nuclear Agreement Review Act before deciding whether to have the full Senate vote on it.  For many in Congress, it's a very simple premise: Congress should weigh in on the terms of a final nuclear deal with Iran.  But for the White House, and some Democrats in Congress, that's an explosive idea that could derail the talks — and therefore must be stopped.  So what's in the bill?  Supporters of the legislation have emphasized that it would allow Congress to give a thumbs up or down on the deal President Barack Obama is aiming to finalize with Iran by the negotiations' June 30 deadline. Backers think that's only fair since the United States is contemplating a major nuclear pact with a decadeslong enemy.  The bill would give Congress a chance to hold hearings, host briefings and pave the way to a vote on a joint resolution that could express approval or disapproval of the deal. It also requires the Obama administration to quickly report to Congress on details of the deal and regularly assess whether Iran is keeping its commitments. And it spells out what the President needs to do vis-a-vis Congress if Iran is found violating the terms.  But most important, it would keep Obama from waiving any congressional sanctions on Iran during Congress' 60-day review period. And if Congress passes a joint resolution of disapproval, then sanctions could not be lifted even after that time.  Does this hurt the diplomatic effort to reach a final deal?  Iran's primary reason for agreeing to the deal's limits on its nuclear program is to get sanctions relief.  Iranian Supreme Leader Ayatollah Ali Khamenei and President Hassan Rouhani have insisted sanctions must be lifted on Day One of the deal. While the Obama administration has said it would wait several months before lifting sanctions -- to verify that Iran has taken major steps to roll back its program — the President is counting on having some latitude to waive sanctions in order to give Iran an incentive to sign the deal and then keep its commitments.  The Corker bill makes it less likely sanctions would be lifted, and lifted in a timely way, which gives less encouragement to Iran.  Without the bill, could Obama lift all sanctions on Iran?  Not all, but most — at least for a limited time.  Without the Corker-Menendez bill (named after Corker and lead Democratic co-sponsor Sen. Bob Menendez), Iran could see nearly all nuclear-related sanctions at least temporarily lifted. Those enacted through the United Nations and by presidential executive order could be fully removed, and most of those passed by Congress could be waived by Obama until he leaves office, when they would go back into effect unless the next president continues to waive them.  If the Corker-Menendez bill becomes law and a deal is finalized, Obama wouldn't be able to lift the congressional sanctions for at least 60 days.  During that 60-day review phase, the bill says Obama can't \"waive, suspend, reduce, provide relief from, or otherwise limit the application of statutory sanctions.\"  How can Obama waive sanctions passed by Congress anyway?  The biggest sanctions package approved by Congress, widely credited with crippling Iran's economy and bringing the country to the negotiating table, was the 2010 Comprehensive Iran Sanctions, Accountability, and Divestment Act.  It included a provision that gives the President leverage to waive sanctions for a limited time if he determines it's in the U.S. national security interest.  He wouldn't be able to permanently sunset the sanctions, though, unless Iran cuts its ties with and stops funding terror organizations around the world, an aspect not being addressed in the current negotiations with Iran and not predicted to happen any time soon.  The administration says sanctions are where Congress could eventually weigh in on the Iran deal -- by repealing them -- but that would only happen if lawmakers deem the deal a good one.  How would Congress weigh in during the 60-day review period?  Congress could pass a joint resolution approving of the deal or disapproving of the deal and forbidding any sanctions relief. Or it could do nothing, allowing Obama to implement the sanctions, and by implication the deal, after the 60 days are up.  A joint resolution disapproving of the deal would need to muster a two-thirds majority, though, to override the veto Obama has promised -- an unlikely, but not entirely unthinkable, scenario.  Are there other reasons that Obama opposes this bill?  First, he's facing a combative, Republican majority in both houses of Congress. Those Republicans have sought to undermine his negotiations with Iran at every turn. Giving those lawmakers a legislative avenue to slam the terms of an eventual deal would, at best, be a political blow to Obama and his administration's efforts to broker an agreement.  There's also the White House's argument that the executive branch has the power to broker international agreements without Congress meddling, while congressional oversight of the Iran deal would set a dangerous precedent. Most international agreements aren't treaties ratified by Congress, and Obama has argued that U.S. allies need to know they can count on those agreements holding up.  Virtually every Senate Republican supports the bill and nine Democrats have signed on as co-sponsors, including independent Sen. Angus King, who caucuses with Democrats. Several more on the left are leaning toward supporting the bill.  That means backers will likely pass the bill with a filibuster-proof majority. It's unclear, though, if they can muster enough Democrats willing to vote to override Obama's veto.  It's not just the numbers, though. Some powerful Democrats are supporting the legislation. Sen. Chuck Schumer of New York, the No. 3 Senate Democrat, who will become the chamber's Democratic leader when Minority Leader Harry Reid of Nevada retires at the end of his term, is a big proponent of the legislation.  Menendez, of New Jersey, who has been working to rally Democrats around legislation on Iran throughout the negotiations, remains the most vocal Democrat on the issue. He was the top-ranked Democrat on the Senate Foreign Relations Committee until stepping aside to face corruption charges handed down earlier in the month.  Also, Sen. Tim Kaine of Virginia, one of the earliest Obama backers during the 2008 campaign, has been insistent on the need for Congress to weigh in and has worked behind the scenes to make Corker's bill more palatable to Democrats -- and the White House. But so far that effort hasn't succeeded, as the White House remains opposed.",  # 1
    "While paging through Pew's best data visualizations of 2014 (it's awesome), I came across what I believe to be one of the best and most revealing charts about the state of American politics. Here it is:  The chart, which comes from Pew's amazing political polarization project, shows how partisans of both parties have grown both increasingly unified amongst themselves and increasingly far apart from their partisan others over just the last 20 years. As recently as 1994, seven in 10 Democrats were more consistently liberal than the median Republican.  As of 2014, it's a whopping 94 percent. Same goes for Republicans; 64 percent of GOPers were more consistently conservative than the median Democrat in 1994 while 92 percent are today. In addition, \"the overall share of Americans who express consistently conservative or consistently liberal opinions has doubled over the past two decades from 10 percent to 21 percent,\" according to Pew.  To me, this chart is so important -- particularly in a week where a new Congress arrives in Washington -- because it reveals that the polarization of our elected officials isn't some sort of \"only in Washington\" thing.  The increasing partisanship of Congress is a direct reflection of the increasing partisanship of the country. After all, that's who elects these people to Congress, right?  So, when you hear people decry the partisanship of their elected officials in Washington, don't believe it.  We have the Congress we want -- even if we aren't totally honest with ourselves all the time about what that is.  And, we get the results -- not many -- from our elected officials that you have to expect when you have a country as polarized as ours.",  # 1
    "With little fanfare this fall, the New York developer who had planned to build an Islamic community center north of the World Trade Center announced that he would instead use the site for a 70-story tower of luxury condos.  Those who had rallied in opposition to the building because of its religious affiliation back in 2010 were exultant. “The importance of the defeat of the Ground Zero Mosque cannot be overstated,” Pamela Geller, president of the American Freedom Defense Initiative, wrote on the website Breitbart in September. “The Ground Zero Mosque became a watershed issue in our effort to raise awareness of and ultimately halt and roll back the advance of Islamic law and Islamic supremacism in America.”  It’s all well and good that so many Republicans have condemned Donald Trump’s reprehensible call for “a total and complete shutdown of Muslims entering the United States.” House Speaker Paul D. Ryan (Wis.) was particularly forceful, calling proper attention to the “many Muslims serving in our armed forces, dying for this country.”  When he was president, George W. Bush honorably put a lid on right-wing Islamophobia. He regularly praised American Muslims and stressed that the United States needed Muslim allies to fight violent extremism. Once Bush was gone, restraint on his side of politics fell away.  Thus, Trump’s embrace of a religious test for entry to our country did not come out of nowhere. On the contrary, it simply brought us to the bottom of a slippery slope created by the ongoing exploitation of anti-Muslim feeling for political purposes.  You don’t have to reach far back in time to see why Trump figured he had the ideological space for his Muslim ban. Last month, it was Jeb Bush who introduced the idea of linking the rights of Syrian refugees to their religion. He said he was comfortable granting admission to “people like orphans and people who are clearly not going to be terrorists. Or Christians.” Asked how he’d determine who was Christian, he explained that “you can prove you’re a Christian.”  Sen. Ted Cruz (Tex.) took a similar view, saying , “There is no meaningful risk of Christians committing acts of terror.”  Trump took limits on Muslim access to our country to their logical — if un-American and odious — conclusion. Vice President Biden said that Trump was serving up “a very, very dangerous brew,” but the brew has been steeping for a long time. This is why the “Ground Zero Mosque” episode is so instructive.  The demagoguery began with the labeling of the controversy itself. As PolitiFact pointed out, “the proposed mosque is not at or on Ground Zero. It does not directly abut it or overlook it.” It was “two long blocks” away. And while a mosque was part of the proposed cultural center, the plans also included “a swimming pool, gym and basketball court, a 500-seat auditorium, a restaurant and culinary school, a library and art studios.”  This didn’t stop opponents from going over the top, and Newt Gingrich deserved some kind of award for the most incendiary comment of all. “Nazis,” he said, “don’t have the right to put up a sign next to the Holocaust museum in Washington.”  When President Obama defended the right of developers to build the project, he was — surprise, surprise — accused of being out of touch, and Republicans were happy to make the Muslim center and Obama’s defense of religious rights an issue in the 2010 campaign.  “I think it does speak to the lack of connection between the administration and Washington and folks inside the Beltway and mainstream America,” said Sen. John Cornyn (Tex.), who was then chairman of the committee in charge of electing Republicans to the Senate. Voters, he said, felt they were “being lectured to, not listened to.” Sound familiar?  At the time, John Feehery, the veteran Republican strategist, put his finger on why Republicans were so eager to lambaste Obama’s response to the Ground Zero issue. “This will help drive turnout for the GOP base,” he said.  The Republican establishment is now all upset with Trump, but he is simply the revenge of a Republican base that took its leaders’ pandering — on Islam and a host of other issues — seriously.  You can’t be “just a little” intolerant of Muslims, any more than you can be “just a little” prejudiced against Catholics or Jews. Once the door to bigotry is opened, it is very hard to shut.  Read more from E.J. Dionne’s archive, follow him on Twitter or subscribe to his updates on Facebook.",  # 1
    "November 13, 2016 By 21wire Leave a Comment  Episode #160 of SUNDAY WIRE SHOW resumes this November 13, 2016 as host Patrick Henningsen brings a 3 HOURS special broadcast of LIVE power-packed talk radio on ACR…  LISTEN LIVE ON THIS PAGE AT THE FOLLOWING SCHEDULED SHOW TIMES:  SUNDAYS – 5pm-8pm UK Time | 12pm-3pm ET (US) | 9am-12pm PT (US)  This week’s edition of THE SUNDAY WIRE is on the road broadcasting LIVE from the Valley of the Sun. This week host Patrick Henningsen covers this week’s top stories in the US and internationally. In the first hour we’ll conduct a post-mortem on the incredible US Election which has produced President Elect Donald J Trump , and the aftermath – a nation divided punctuated by numerous street protests in part fuelled by Soros and the Democratic Party Machine . Later, we’re joined by our roving everyman , ACR Boiler Room contributor, Randy J , for an on the ground take on Election events from the West Coast, and beyond…  SUPPORT 21WIRE – SUBSCRIBE & BECOME A MEMBER @21WIRE.TV  Strap yourselves in and lower the blast shield – this is your brave new world…  *NOTE: THIS EPISODE MAY CONTAIN STRONG LANGUAGE AND MATURE THEMES* ",  # 0
    "Hillary Clinton told a Staten Island crowd today that she was the candidate who could reach across party lines to get things done as president—pointing to her experience representing the borough in the Senate and even giving public thanks to Republican President George W. Bush.  “I have no time for people who are partisan for the sake of being partisan,” Ms. Clinton, a Democrat and former secretary of state, told a crowd that welcomed her enthusiastically in a historic building at Snug Harbor Cultural Center.  Ms. Clinton was the second leading presidential candidate to speak on Staten Island today, after Republican Donald Trump delivered a typical stump speech at a Republican brunch. But Ms. Clinton, perhaps sensing or recalling Staten Island’s ever-present desire for politicians to pay attention to the forgotten borough, delivered a speech that seemed to be especially tailored for the borough, which is more conservative than the rest of New York City. The majority of its voters are registered Democrats, but the borough routinely swings Republican.  “When I ran in 2000, I didn’t carry Staten Island. When I ran in 2006, I did,” Ms. Clinton said today. “I’ve got no problem with people having political disagreements. That’s in America’s DNA, isn’t it?”  Instead, Ms. Clinton decried “deliberate efforts to set Americans against each other.” She harkened back to representing the borough of cops and firefighters on September 11, 20001, when nearly 300 Staten Islanders were killed in the terror attacks. She noted the city’s mayor, the state’s governor, and the president were all Republicans.  “I did not for one minute stop and say to myself: ‘Well, I don’t know, can we work together?'” Ms. Clinton said. “How absurd is that?”  Ms. Clinton said with 3,000 people murdered, people “pulled together” and “politics was totally left behind.” She had criticized President Bush for his handling of a strong economy he “inherited” from her husband, President Bill Clinton, and for other reasons. But she praised him for approving billions of dollars to rebuild lower Manhattan after the terror attacks, even as others in Washington didn’t want to spend it.  “I publicly say, ‘Thank you President George W. Bush,’ for making sure that we got the money that we needed to rebuild our city,” Ms. Clinton said.  She also touted her record working for the borough after the attacks, including sounding the alarm on air quality and pushing for the passage of the Zadroga Act. Ms. Clinton went on to discuss her foreign policy experience, saying anyone who wants to president has to be able to offer specifics on what they would do—an allusion to her Democratic rival Vermont Sen. Bernie Sanders—but also has to be able to keep people safe.  “Maybe we take that more seriously here in New York, but we should,” she said. “And we are going to do everything we can to keep America safe.”  She cited her work building a coalition that “brought Iran to the negotiating table,” without mentioning how controversial that final Iran deal was for President Barack Obama.  Ms. Clinton’s rally had little in common with Mr. Trump’s Staten Island event, except one thing: the crowd at both seemed to hate Republican Texas Sen. Ted Cruz. Ms. Clinton mentioned his plan to spy on Muslim neighborhoods, which has also been dismissed by Police Commissioner Bill Bratton.  At the mention of Mr. Cruz’s name, the crowd offered a chorus of boos.  The crowd had been hyped up for Ms. Clinton before she even arrived, and gave a nice response to several local pols as they urged people to get out the vote on Tuesday.  “Hillary is the only one with the experience,” said Councilwoman Debi Rose, a Democrat who represents the borough’s North Shore. “She knows what it’s like to be in the White House. She knows the realities. She knows that living and running the White House in this country is not a reality show.”  Ms. Clinton, at the end of her speech, offered the kind of promise Staten Island always loves—and the kind it holds politicians to, if the way the Staten Island Advance repeatedly agitated and inquired about a town hall Mayor Bill de Blasio will finally hold here on Wednesday night is any indication.  “I want you to hold me accountable,” Ms. Clinton said. “I will be coming back to Staten Island when I am your president.”",  # 1
    "Mitch McConnell has an unusual admonition for the new Republican majority as it takes over the Senate this week: Don’t be “scary.”  The incoming Senate majority leader has set a political goal for the next two years of overseeing a functioning, reasonable majority on Capitol Hill that scores some measured conservative wins, particularly against environmental regulations, but probably not big victories such as a full repeal of the health-care law. McConnell’s priority is to set the stage for a potential GOP presidential victory in 2016.  “I don’t want the American people to think that if they add a Republican president to a Republican Congress, that’s going to be a scary outcome. I want the American people to be comfortable with the fact that the Republican House and Senate is a responsible, right-of-center, governing majority,” the Kentucky Republican said in a broad interview just before Christmas in his Capitol office.  It’s a far cry from his defiant declaration in 2010 that his “single most important” goal was to make President Obama a one-term president, an antagonizing oath that Democrats frequently invoke to embarrass the GOP leader — Obama won reelection comfortably in 2012, and McConnell’s party lost seats.  Now in charge at both ends of the Capitol, Republicans aim to avoid the worst excesses of the past four years and make sure the public isn’t fearful of the GOP’s course.  “There would be nothing frightening about adding a Republican president to that governing majority,” McConnell said, explaining how he wants voters to view the party on the eve of the 2016 election. “I think that’s the single best thing we can do, is to not mess up the playing field, if you will, for whoever the nominee ultimately is.”  But McConnell, who will become majority leader Tuesday, is not planning to avoid conflict altogether. He wants to use the annual spending bills to compel Obama to accept conservative policy riders that will divide Democrats, similar to the December spending bill’s inclusion of a provision benefiting Wall Street firms involved in risky derivative trades. That rider brought a liberal outcry but did not end up torpedoing the bill, which had Obama’s support.  McConnell has been coaching his members to understand that, in the initial rounds, they will have to almost unanimously support the budget outline and the spending bills, because few Democrats will support their policy riders.  With 54 Republicans in his caucus, McConnell knows that he’s a long way from getting 67 votes to override an Obama veto and that it won’t even be easy getting six Democrats to regularly support legislation so that he can overcome likely filibusters led by the incoming minority leader, Sen. Harry M. Reid (D-Nev.). Still, on some issues, such as energy and taxes on the health industry, McConnell thinks there’s enough bipartisan support to get bills onto Obama’s desk.  “They’d like to be relevant. They’d like to be part of the process,” he said of discussions with some rank-and-file Democrats. “Assuming we will have on most issues a largely unified conference — I don’t expect that on everything — it wouldn’t take a whole lot of Democrats to actually pass legislation in the Senate.”  But McConnell said those who are “craving some grand deal as a way to measure the next two years” should lower their expectations. He’s very skeptical of such bargains with Democrats on tough issues such as immigration and entitlement reform. Instead, he believes three issues have potential common ground: international trade deals, an overhaul of the tax code and new revenue streams for infrastructure projects.  “Could the country use a lot more? You bet. But there’s no way you can overcome a reluctant president on something really large,” McConnell said. The best he can do on some of those bigger issues is force Obama to break out his veto pen so there is a clear set of Democratic policy stances Republicans can campaign against in 2016.  Democrats are dubious of McConnell’s pledge to avert edge-of-the-cliff moments. They believe he will run into the same problems that have bedeviled House Speaker John A. Boehner (R-Ohio) during the past four years — including the inability to corral rabble-rousers such as Sen. Ted Cruz (R-Tex.) to support an agenda that conservative critics will probably view as not bold enough in challenging Obama. Appeasing those far-right conservatives will lead to an agenda that Democrats hope to exploit in 2016.  “What Senator McConnell wants people to think and what they will think when they see the results for themselves are two very different things,” said Reid’s spokesman, Adam Jentleson. “Senator McConnell heads a caucus that is obsessed with rigging the game against working people in favor of wealthy special interests. That’s a scary fact indeed, and he won’t be able to hide it.”  While McConnell’s office is adorned with a portrait of Kentucky’s only other Senate majority leader — Alben W. Barkley, whose tenure was often marked by clashes with his own party’s president, Franklin D. Roosevelt — his model more resembles that of George Mitchell, the Democratic majority leader from 1989 through 1994.  Without ever shutting down the government, Mitchell stared down a president of the opposite party, George H.W. Bush, and won several liberal victories in a 1990 budget showdown. Mitchell also delivered a slew of legislation to Bush’s desk that drew a veto, helping frame the debate for the 1992 election on domestic policy.  McConnell has recently studied other two-term presidents on how often they vetoed legislation in their last two years in office. Obama has issued only two minor vetoes over technical matters, largely because the previously Democratic-run Senate prevented anything from reaching his desk that he opposed.  McConnell suggested that a veto strategy would help clarify issues for voters. “I think his bureaucracy across the board has done a lot of damage to the country. . . . I’d at least like for him to have to personally take responsibility for it, even if he in the end decides to veto the bill over some restriction on EPA regulations.”  McConnell is keenly aware of the challenges of reining in some of the impulses that his fellow Republican lawmakers have developed over eight straight years in the minority.  The first test will come on an energy debate, which could begin by the end of this week. McConnell is trying to keep his side from offering amendments not related to energy issues. In recent years, when some rank-and-file Republicans wanted to stop the Senate in its tracks, they would threaten amendments related to how Congress gets its health care.  “I’ve asked my members to restrain themselves,” he said.  Restraint has been hard to come by in this political era, particularly because a small army of conservative groups has made it a mission to push Republicans to the most strident stands, even if it means shutting down the government or risking default on the national debt.  McConnell faced some of those groups firsthand when they supported his 2014 primary challenger, and he faced a grinding general election against a Democrat who was well financed.  It cost nearly $30 million from his campaign war chest and tens of millions of dollars more from outside groups. Winning both races handily, McConnell is delivering a message to Republicans on how to behave heading into 2016.  “Don’t try to reinvent yourself. Be yourself, number one. And don’t be afraid of a primary. We will win all the primaries. We did it in ’14. We will do it in ’16,” McConnell said.  Shortly after Election Day this fall, McConnell sat down with Obama for a rare one-on-one meeting in the Oval Office, a huddle that both sides kept largely secret. In the pre-Christmas interview, McConnell acknowledged that the meeting’s purpose was only to discuss areas where the president and Republicans might find bipartisan agreement — because their disagreements on other issues were too in­trac­table.  That meeting, he said, focused on trade deals, a tax code overhaul and infrastructure funding. “Can we get there on tax reform, trade, infrastructure?” he said.  He eschewed the idea of holding the debt ceiling hostage to gain spending-cut concessions from Obama, a tool Republicans had to use before because they were in the minority.  “We are in the majority. We can pass a budget. We can determine how much we’re going to spend,” he said.  Avoiding those moments could make for a less “scary” Congress, giving the Republicans a better chance in 2016 to hit the trifecta and gain the White House. His own horse is already chosen, Sen. Rand Paul (R-Ky.), but don’t expect McConnell to be out on the campaign trail or doing any other favors for his home-state colleague.  “I’m going to be supporting Rand Paul. But he knows that beyond that, I won’t be involved in presidential politics. I’ve got a big job here,” he said.",  # 1
    "Mises.org November 1, 2016 Inferno is a great thriller, featuring Tom Hanks reprising his role as Professor Robert Langdon. The previous movie adaptations of Dan Brown’s books ( Angels and Demons and The Da Vinci Code ) were a success, and I expect Inferno will do well in theaters, too. Langdon is a professor of symbology whose puzzle solving skills and knowledge of history come in high demand when a billionaire leaves a trail of clues based on Dante’s Inferno to a biological weapon that would halve the world’s population. The villain, however, has good motives. As a radical Malthusian, he believes that the human race needs halving if it is to survive at all, even if through a plague. Malthus’s name is not mentioned in the movie, but his ideas are certainly there. Inferno provides us an opportunity to unpack this overpopulation fear, and see where it stands today. Thomas Malthus (1766–1834) thought that the potential exponential growth of population was a problem. If the population increases faster than the means of subsistence, then, “The superior power of population cannot be checked without producing misery or vice.” Is overpopulation a problem? The economics of population size tell a different, less scary, story. While it is certainly possible that some areas can become too crowded for some people’s preferences, as long as people are free to buy and sell land for a mutually agreeable price, overcrowding will fix itself. As an introvert who enjoys nature and peace and quiet, I am certainly less willing to rent an apartment in the middle of a busy, crowded city. The prices I’m willing to pay for country living versus city living reflect my preferences. And, to the extent that others share my preferences or even have the opposite preferences, the use and construction of homes and apartments will economize in both locations. Our demands and the profitability of the varied real estate offerings keep local populations in check. But what about on a global scale? The Inferno villain was concerned with world population. He stressed the urgency of the situation, but I don’t see any reason to worry. Google tells me that we could fit the entire world population in Texas and everybody would have a small, 100 square meter plot to themselves. Indeed, there are vast stretches of land across the globe with little to no human inhabitants. Malthus and his ideological followers must have a biased perspective, only looking at the crowded streets of a big city. If it’s not land that’s a problem, what about the “means of subsistence”? Are we at risk of running out of food, medicine, or other resources because of our growing population? No. A larger population not only means more mouths to feed, but also more heads, hands, and feet to do the producing. Also, as populations increase, so does the variety of skills available to make production even more efficient. More people means everybody can specialize in a more specific and more productive comparative advantage and participate in a division of labor. Perhaps this question will drive the point home: Would you rather be stranded on an island with two other people or 20 other people? Malthus wouldn’t be a Malthusian if he could see this data The empirical evidence is compelling, too. In the graph below, we can see the sort of world Malthus saw: one in which most people were barely surviving, especially compared to our current situation. Our 21st-century world tells a different story. Extreme poverty is on the decline even while world population is increasing. Hans Rosling, a Swedish medical doctor and “celebrity statistician,” is famous for his “Don’t Panic” message about population growth. He sees that as populations and economies grow, more have access to birth control and limit the size of their families. In this video , he shows that all countries are heading toward longer lifespans and greater standards of living. Finally, there’s the hockey stick of human prosperity. Estimates of GDP per capita on a global, millennial scale reveal a recent dramatic turn. The inflection point coincides with the industrial revolution. Embracing the productivity of steam-powered capital goods and other technologies sparked a revolution in human well-being across the globe. Since then, new sources of energy have been harnessed and computers entered the scene. Now, computers across the world are connected through the internet and have been made small enough to fit in our pockets. Goods, services, and ideas zip across the globe, while human productivity increases beyond what anybody could have imagined just 50 years ago.  I don’t think Malthus himself would be a Malthusian if he could see the world today.  Note: The views expressed on Mises.org are not necessarily those of the Mises Institute. The Best of Jonathan Newman Tags: Jonathan Newman was a 2013 and 2014 Summer Fellow at the Mises Institute and teaches economics at Auburn University.",  # 0
    "Washington (CNN) The faction of the GOP that is unhappy with Donald Trump as the party's presumptive nominee has one last plan to stop the mogul: staging an all-out delegate revolt at the Republican National Convention.  The far-fetched idea is the latest reflection of a campaign cycle that has been anything but ordinary, and stems from a continuing dissatisfaction among some conservative stalwarts with how Trump is behaving and running his campaign. But two longtime GOP veterans says they wouldn't bet on the effort working.  The effort comes at a rough time for the GOP. As the Democratic Party's heaviest hitters, including President Barack Obama, Vice President Joe Biden and Massachusetts Sen. Elizabeth Warren, line up behind Hillary Clinton and against Trump, Republicans have been forced to criticize their own nominee. Recent comments from Trump about a federal judge's Mexican heritage have drawn widespread rebuke and put GOP leaders in a corner as they defend their endorsement of Trump while disavowing his comments.  One of the vocal advocates for a delegate revolt is conservative commentator and Weekly Standard editor Bill Kristol, who has also been actively seeking a candidate to mount an independent bid against Trump, thus far to no avail.  Kristol tweeted late Thursday that the idea of a \"conscience convention,\" where delegates are free to vote for whomever they want to, is also appealing.  \"I've been focused on independent candidacy, & still am. But struck by sudden level of interest in possible delegate revolt at convention,\" Kristol tweeted. He added: \"A Convention of Conscience in Cleveland would be quite something. Made easier by fact Trump only won minority of total primary votes anyway.\"  Bob Vander Plaats, the head of The Family Leader, an influential social conservative group in Iowa, told CNN's Kate Bolduan Friday morning that \"everything does need to be on the table\" at the convention, though he stopped short of calling for a revolt on the convention floor.  \"We want a principled conservative and disciplined candidate who is the standard-bearer of this party,\" said Vander Plaats, who backed Texas Sen. Ted Cruz during the primaries. He said Trump has time before the convention to \"have the concerns laid to rest.\"  \"All I'm doing is adapting to the circumstances,\" Kendal Unruh told ABC. \"I certainly believe Trump's demagogic racist comments are hurting him.\"  The rules enacted by the previous convention, which govern in 2016 until delegates pass a new set of rules, state that even if a delegate casts a ballot for a candidate other than one they are bound to, the convention secretary will record their bound vote.  In order to change that rule, the 112 delegates (two from each state and territory) on the Rules Committee would have to pass different rules and bring those to the floor of the convention, where a majority of delegates present would have to approve them.  Rules expert and RNC veteran Jim Bopp, an Indiana delegate who serves as special counsel to the RNC Rules Committee, said he has spoken with people who want to \"keep the option open to manipulate the rules in some way to deny Trump the nomination,\" but he said he wouldn't bet on any changes.  \"I would put money on no rules changes that would affect the outcome of the nominating process,\" Bopp told CNN. \"I think it's highly likely that no rules changes would be adopted that would affect the nomination.\"  Bopp said there's also a counter movement within Rules insiders to pass a rule that would prevent any other rules changes from going into effect until the close of the convention.  Rules Committee and Oregon RNC member Solomon Yue is behind that effort, and has been pushing the RNC this year to adopt rules that give less power to the party and more to the delegates. He tried but failed to get the party to adopt rules that would require bigger majorities to pass business at the convention.  Yue says with roughly 80% of the convention delegates being either Trump or Cruz backers, the anti-Trump forces don't have much strength.  \"The common denominator of the delegates is anti-establishment, anti-Washington,\" Yue said. \"And if you think about 'Never Trump' people, they are representing Washington and the establishment.\"  Part of the philosophy for a delegate revolt comes from longtime RNC veteran Curly Haugland, from North Dakota, and a book he co-wrote with public policy consultant Sean Parnell. \"Unbound\" uses the history of the RNC to make the case that RNC rules dictate that delegates be allowed to vote their conscience.  \"What Curly and I are contending is that because of RNC rules, there is no such thing as binding,\" Parnell told CNN, saying the binding rules that currently are in place are in a part of the rules package that govern pre-convention during the delegate selection process.  But Parnell also acknowledged that any effort to make the interpretation stick would require at the very least a handful of state delegations if not a majority of delegates on the floor.  \"It would be messy. Good television though,\" Parnell said. \"I would not call it likely. My hope is that delegates are free to vote however they want to vote, and it's going to be up to the chair whether or not to allow that. But I think unless Donald Trump actually does go shoot somebody on Fifth Avenue, I don't think this is going to cost him the nomination.\"  Neither the Trump campaign nor Republican National Committee immediately responded to a request for comment on the chatter about a delegate revolt.  There has been intense focus on the Rules Committee for months, going back to when there was a possibility of no Republican candidate getting enough delegates to clinch the nomination outright. With the prospect of a contested convention, the Cruz campaign made a concerted effort to stock the Rules Committee and state delegations with loyalists who would support Rules that would benefit Cruz in a bid to win the nomination on multiple ballots.  But after Cruz lost Indiana soundly and suspended his campaign, the prospect of a contested convention vanished and Trump rolled to the magic number to clinch the nomination.  Meanwhile, the Cruz campaign urged supporters to continue to become delegates and earn leadership spots to influence the platform at the convention.  Though some in the party have never warmed to Trump, the intensity of finding a way to prevent his formal nomination has grown in recent days after Trump's comments about a federal judge inflamed even the leaders of his own party.  Trump questioned the impartiality of the district court judge overseeing a lawsuit related to his venture Trump University, saying the Indiana-born judge's Mexican ancestry could bias him against Trump. The mogul cited his campaign promise to build a wall along the border with Mexico in making the comments.  Though the presumptive nominee has repeatedly stood behind and doubled down on the comments, his stance has drawn outrage from the likes of Senate Majority Leader Mitch McConnell and House Speaker Paul Ryan, who called the remarks \"the textbook definition of a racist comment.\"  Still, only a small handful of Republicans have withdrawn or withheld their endorsements of Trump. Vulnerable Illinois Sen. Mark Kirk disavowed Trump this week and said he could not endorse the party's nominee after all, but Ryan, McConnell and others have stood by their endorsements, saying Clinton would be a worse choice.",  # 1
    "Meanwhile, Democrat Bernie Sanders picked up more delegates in the two states than Hillary Clinton.  The Vermont senator's still way behind, but says he's not giving up, calling his win in West Virginia \"tremendous.\"  Clinton still holds a commanding delegate lead, but Sanders still has the fight in him.  \"We are in this campaign to win the Democratic nomination!\" he declared.  But Sanders' quest appears to be almost impossible, with Clinton 94 percent of the way to winning the nomination.  \"I am, if I am so fortunate enough as to be the nominee - I am looking forward to debating Donald Trump come the fall,\" she said.  Still, Clinton faces the FBI investigation of her email scandal.  In addition, her loss in West Virginia, a state she took in 2008, was payback for her statements in March that a lot of coal miners and coal companies would be out of business.  She's also polling badly among whites, men, and young people, with many loyal Sanders supporters vowing to never vote for her.  Meanwhile, on the Republican side, Trump kept rolling, winning West Virginia and Nebraska.  The billionaire told the Associated Press he's looking at five or six people as his running mate, all with deep political experience.  One person that Trump will likely not be considering is Sen. Marco Rubio, who told CNN he disagrees with Trump's \"America first\" foreign policy.  \"I have well defined differences with the presumptive nominee of the Republican Party,\" Rubio said. \"And like millions of Republicans, you try to reconcile those two things.\"  Trump will be meeting on Capitol Hill Thursday with Republican leaders, a group he has railed against, to try to patch up the public differences they've had.  Unifying the Democrats is going to be an issue for Clinton too, as many Sanders voters have said they wouldn't vote for her.  So, after tough primary battles, both parties are facing the question of how to work together against each in the general election this fall.",  # 1
    "After a week of nonstop criticism from Democrats and Republicans alike for comments many condemned as racially charged, Donald Trump claims to be altering his campaign to be a little more inclusive. While the presumptive G.O.P. has long promised to “make America great again,” Trump now says he’s adding two words to slogan to illustrate just how non-racist he really is.  “You know, I have the theme ‘make America great again,’ and I've added a couple of things,” Trump announced to supporters at a campaign rally in Richmond, Virginia, on Friday night. “Right now I’m adding make America great again—I’m adding ‘for everyone,’ because it’s really going to be for everyone. It’s not going to be for a group of people, it’s going to be for everyone. It’s true.”  The allegedly amended slogan, which has yet to appear on any official signage or Trump merchandise, comes after the presidential candidate spent the first half of June repeatedly denouncing Gonzalo Curiel, the federal judge of Mexican heritage presiding over the Trump University class action lawsuit, as inherently biased against him. (Curiel was born in Indiana.) His comments were widely condemned by the Washington political establishment, including Senate Minority Leader Mitch McConnell, who suggested he may be an idiot, and House Speaker Paul Ryan, who called Trump’s statement the “textbook definition of a racist comment.”  Trump, who hasn’t apologized or taken back any of his comments, indicated on Friday that he realized his words have had a negative effect on his campaign and declared he is not a racist.  “I am the least racist person. The least racist person that you’ve ever seen. I mean give me a break,” he said at the rally. “I am the least racist person that you’ve ever looked at, believe me.”",  # 1
    "If you want a glimpse into a presidential candidate’s governing style, take a look at his campaign. How does he build an organization? How does he manage personnel? How does he delegate responsibility? Does he value loyalty or honesty in advisers? It’s hardly conclusive, but how a candidate deals with these challenges says a lot about what they’d prioritize as president.  The presumptive Republican nominee has managed his campaign the way he manages his casinos or his realty TV program: haphazardly and with an unearned arrogance. Everything’s about the brand and non-sycophants are cast aside. Getting it right is far less important than being right. Thus, when prompted by advisers to dial down the rhetoric and think more about his long-term viability, Trump has resisted. He’s yet to understand how different a primary and a general election are, and he’s too cocksure to listen to anyone who tries to explain it.  Trump isn’t as stupid as he pretends to be, but his confidence seems to scale with his ignorance, and that’s a dangerous trait in a president, given how consequential each decision can be. As a candidate, a confident idiot can make a lot of noise and fool a lot of voters. But you can’t lead that way. As president, Trump would need the sober advice of serious professionals. Considering how little he understands about the job and the world, this is especially true in his case.  Based on the latest behind-the-scenes reports on Trump’s campaign (as well as his entire history in real estate and television), it’s unlikely he would govern with the humility and self-awareness required. The campaign has undergone several shake-ups in recent months, with key staffers like Corey Lewandowski being demoted to make way for veteran operatives like Paul Manafort. Trump’s approach has remained very much the same, however. Everyone he brings in bends to his will or is replaced by someone who does.  Things have gotten more chaotic in the last week or so, as Rick Wiley was fired as the campaign’s national political director. Wiley was known to assert himself in ways that alienated Trump and his loyalists. The Washington Post’s Sean Sullivan and Robert Costa summarized the broader situation over the weekend: “For the last two months, Donald Trump has presided over a political team riddled with turf wars, staff reshuffling and dueling power centers. But tensions are more than typical campaign chaos: They illustrate how Trump likes to run an organization, whether it’s a real estate venture or his presidential bid. Interviews with current and former Trump associates reveal an executive who is fond of promoting rivalries among subordinates, wary of delegating major decisions, scornful of convention and fiercely insistent on a culture of loyalty around him.” These are the management techniques of a brand-hustling magnate, not a president of the United States. Trump’s dictatorial approach works well on the campaign trail, but it’d be a disaster in office. A president has to persuade and compromise. The capacity to admit ignorance is equally important. Trump, by all accounts, has no interest in any of these things. Trump doesn’t know what he needs to know in order to be president, nor does he care that he doesn’t know. If his campaign is any indication, he’ll hire people to ensure his veil of ignorance is never cracked. That kind of megalomania is why Trump is who he is – it’s part of his outsized persona. It’s also the clearest indicator we have of the kind of president he’d be.",  # 1
    "Syrian War Report – October 31, 2016: Al-Nusra-led Forces Failed to Break Aleppo Siege ‹ › South Front Analysis & Intelligence is a public analytical project maintained by an independent team of experts from the four corners of the Earth focusing on international relations issues and crises. They focus on analysis and intelligence of the ongoing crises and the biggest stories from around the world: Ukraine, the war in Middle East, Central Asia issues, protest movements in the Balkans, migration crises, and others. In addition, they provide military operations analysis, the military posture of major world powers, and other important data influencing the growth of tensions between countries and nations. We try to dig out the truth on issues which are barely covered by governments and mainstream media. Syrian War Report – November 1, 2016: Syrian Military Deploys Advanced T-90 Battle Tanks to Aleppo By South Front on November 1, 2016 …from SouthFront  The Syrian military has deployed advanced Russian-made T-90 main battle tanks to western Aleppo, according to the video released on October 31. T-90 MBTs were observed in the Minyan area where they were participating in operations against Jaish al-Fatah, a Jabhat al-Nusra-led coalition of militant groups. Last weekend, elite units of the Syrian Army’s Tiger Forces and the Desert Hawks Brigade were deployed in Aleppo to counter the militants’ offensive operation to capture the al-Assad Military Academy and the nearby areas. Both formations operate T-90 MBTs supplied by Moscow over the last year. The government forces massively use tanks, artillery, warplanes and helicopters to attrit Jaish al-Fatah’s manpower in non-populated urban areas.  Experts note that the jihadists have also concentrated a high number of experienced troops, artillery, rocket launchers and military equipment at a restricted front in western Aleppo. To do this, they had been pushed to use almost all their resources from the rear bases in Idlib province. If Jabhat al-Nusra is not able to achieve a decided success in clashes with the government forces soon, this will lead to its total collapse as a powerbroker in the war. The group’s material and technical base will be destroyed and experienced troops and field commanders killed in the clashes. We’ve been already able to observe signs of this tendency since the failed al-Nusra attempt to dig in the Ramouseh Artillery Acandemy in southern Aleppo.  The Kurdish YPG and the Ankara-led forces (Turkish-backed militant groups and the Turkish Armed Forces) have been competing in the northeastern countryside of Aleppo city. Both forces recently captured a bunch of villages from ISIS in the direction of Al-Bab. In this case, the ongoing coordination between the Syrian army and the YPG has once again become reality on the ground. The recent Kurdish operations were coordinated and supported by the Russian and Syrian military and military sources say that Moscow increased military supplies to the YPG in the area. Moscow and Domascus believe that the Kurdish buffer zone plays an important role, preventing Harakat Nour al-Din al-Zenki and other pro-Turkish groups from attacking the Syrian army and its allies in Aleppo city. In mid-October Harakat Nour al-Din al-Zenki officially announced that the next stage of Turkey’s Operation Euphrates Shield will include an advance on the ‘regime forces’ in Aleppo.  Dozens of al-Nusra Front and linked-groups members were killed during their failed attempt to break the Syrian army’s defenses at the abandoned al-Mahjoorah Battalion military camp near the militant-controlled town of Ibtaa in the province of Daraa on October 31. According to pro-government forces over 40 militants were killed. Pro-militant media outlets confirm 26 killed in action terrorists. Related Posts: No Related Posts The views expressed herein are the views of the author exclusively and not necessarily the views of VT, VT authors, affiliates, advertisers, sponsors, partners, technicians, or the Veterans Today Network and its assigns. LEGAL NOTICE - COMMENT POLICY Posted by South Front on November 1, 2016, With 727 Reads Filed under World . You can follow any responses to this entry through the RSS 2.0 . You can skip to the end and leave a response. Pinging is currently not allowed. FaceBook Comments  You must be logged in to post a comment Login WHAT'S HOT",  # 0
]

# Etiquetas esperadas (0 = falsa, 1 = verdadera)
true_labels = [
    0,
    0,
    1,
    0,
    1,
    0,
    0,
    1,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    1,
    0,
    0,
    1,
    1,
    1,
    0,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
]

# Realizar las predicciones
predictions, probabilities = predict_news(news_to_test)

# Contadores para las verdaderas y falsas
correct_true = 0
correct_false = 0
incorrect_true = 0
incorrect_false = 0

# Mostrar los resultados con las etiquetas reales
for i, news in enumerate(news_to_test):
    pred_label = 'Verdadera' if predictions[i] == 1 else 'Falsa'
    true_label = 'Verdadera' if true_labels[i] == 1 else 'Falsa'

    # Comparar con las etiquetas reales
    if true_labels[i] == 1 and predictions[i] == 1:
        correct_true += 1
    elif true_labels[i] == 0 and predictions[i] == 0:
        correct_false += 1
    elif true_labels[i] == 1 and predictions[i] == 0:
        incorrect_true += 1
    elif true_labels[i] == 0 and predictions[i] == 1:
        incorrect_false += 1

    print(f"Noticia: {news}")
    print(f"Predicción IA: {pred_label}")
    print(f"Etiqueta Real: {true_label}")
    print(f"Probabilidad de ser falsa: {probabilities[i][0]:.4f}")
    print(f"Probabilidad de ser verdadera: {probabilities[i][1]:.4f}")
    print("")

# Calcular las métricas de evaluación
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
accuracy = accuracy_score(true_labels, predictions)

# Mostrar las métricas
print(f"Exactitud: {accuracy:.4f}")
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Contador final
print("\nContador final:")
print(f"Verdaderas correctas: {correct_true}")
print(f"Falsas correctas: {correct_false}")
print(f"Verdaderas incorrectas: {incorrect_true}")
print(f"Falsas incorrectas: {incorrect_false}")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Noticia: Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam.  In the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI.  The word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI.  But that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election.  The FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI.  The FBI’s leadership is being warned that the en

In [11]:
import torch
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Cargar el modelo y el tokenizer entrenados
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AdvancedBERTClassifier.from_pretrained("./results", num_labels=2)

# Definir la función para realizar las predicciones
def predict_news(news_list):
    # Tokenizar las noticias
    encodings = tokenizer(news_list, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Obtener los parámetros necesarios para la predicción
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']

    # Realizar las predicciones (sin token_type_ids)
    with torch.no_grad():
        # Llamamos al modelo
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # La salida es un tensor de logits
        logits = outputs

        # Convertir logits a probabilidades usando softmax
        probabilities = torch.softmax(logits, dim=-1)  # Probabilidades para ambas clases
        predictions = torch.argmax(probabilities, dim=-1)  # Predicción de clase (índice de la clase con la mayor probabilidad)

    return predictions.numpy(), probabilities.numpy()

# Conjunto de noticias de prueba (verdaderas y falsas)
news_to_test = [
    "Home / Badge Abuse / Shots Fired, 117 Arrests Made as Militarized Police Remove Pipeline Protesters Shots Fired, 117 Arrests Made as Militarized Police Remove Pipeline Protesters The Free Thought Project October 28, 2016 1 Comment  Standing Rock, ND — (RT) Hundreds of police in riot gear and with heavy military equipment have moved Dakota Access Pipeline protesters from their encampment on private land in North Dakota. Police have reportedly arrested at least 117 demonstrators so far.  “117 protesters have been arrested [as of 8:15 pm Thursday]. Morton County will be utilizing other jails in this mass arrest operation,” said Donnell Hushka, a spokeswoman for the Morton County, North Dakota, Sheriff’s Department.  Police have entered the sacred ground camp and surrounded protesters there. Livestream videos showed more than one hundred protesters on the front line. Police have protesters more or less surrounded. #noDAPL pic.twitter.com/G4xGQuXpZM  — Jason Patinkin (@JasonPatinkin) October 27, 2016  The Morton County Sheriff’s Office confirmed to the Associated Press that it had cleared the private land of protesters around 6:30pm EST Thursday.  Shortly after 7:00pm EST Thursday, the sheriff’s office posted on its Facebook page that “one woman has been arrested for firing a weapon at the police line near Highway 1806.”  The sheriff’s office just after 8:00pm EST stated in a Facebook post that it received reports of “several incidents involving firearms.” One “involved a private individual who was run off the road by protestors. The victim was shot in the hand and is being treated. An investigation in underway. No law enforcement was involved with this shooting.”  With regard to the woman arrested, the sheriff’s office said she “pulled a .38 caliber revolver and fired three shots at law enforcement, narrowly missing a sheriff’s deputy.” The sheriff’s office claims it did not fire any shots when taking her into custody.  “Ten shots were reported in the area. It’s possible it is local hunters. This incident is under investigation,” the Facebook post concluded. BREAKING: Sheriff: Dakota Access pipeline protesters cleared from private property.  — The Associated Press (@AP) October 27, 2016  According to KFYR, police made 16 arrests by evening time Thursday.  “The protesters are not being peaceful or prayerful. Law enforcement has been very methodical in moving ahead slowly as to not escalate the situation. However, the protesters are using very dangerous means to slow us down. Their aggressive tactics include using horses, fire and trying to flank us with horses and people,” said Cass County Sheriff Paul Laney in a press release Thursday. 16 arrests at DAPL today…. #kfyrkafe  — KFYR AM 550 (@KFYR550) October 27, 2016  “Move to the south,” police said over the loudspeaker.  “We see some of you have bows. Do not shoot bows and arrows at us,” they added, as protesters laughed.  National Guard troops and police removed tents and other possessions from the new encampment, which the Standing Rock Sioux tribe says is sacred land granted to them via treaty in 1851.  Police also dismantled the barricade across the road.  As police corralled the protesters southward, some activists remained facing the police and even engaged with them while walking backwards. Atsa E’sha Hoferer was livestreaming on Facebook, telling officers that they should be on the side of the protests and that they too would suffer from contaminated water should the pipeline burst. Police are pulling down tents now. Just saw them spray paint an X on a teepee. pic.twitter.com/nNrWRPDggP  — Caroline Grueskin (@cgrueskin) October 27, 2016  The stand-off with police had lasted for several hours before authorties moved closer to the camp and started arresting people. The number of arrested is currently unclear.  The protesters are currently blocking state Highway 1806, which law enforcement is attempting to clear using cop cars and military – style vehicles. Police said they will defend themselves against horses if necessary #DAPL #NoDAPL pic.twitter.com/5yWqKBUeGW  — wes enzinna (@wesenzinna) October 27, 2016  Police have said that they will defend themselves against horses, journalist Wes Enzinna tweeted.  “Do not ride horses at the police,” law enforcement announced.  “You’re endangering people’s lives by what you’re doing,” police told the protesters, telling them to move south or be arrested. People will not be arrested if they return to the main camp, one officer said at the barricade.  Protesters are burning tires, logs and other objects, sending smoke onto the roadway. A car has also been torched, journalist Jason Patinkin tweeted.  — Jason Patinkin (@JasonPatinkin) October 27, 2016  Although the Federal Aviation Administration has restricted flights over the Cannon Ball area until November 5, there is a police helicopter above the faceoff, journalist Antonia Juhasz tweeted.  More than 260 protesters have been arrested in the weeks leading up to Thursday’s operation, AP reported.  In the morning police made a line in front of the fence and the company workers are behind them. The tents of the so-called North Camp were located a stone’s throw away from the police line. Authorities insist that this camp should be removed, according to activist Atsa E’sha Hoferer live-streaming from the site.  People on the barricades are retreating peacefully amid chants of “hands up, don’t shoot!”  People are moving from the road towards the construction site. “We might as well get arrested defending the sacred,” a protester told the Bismarck Tribune’s Caroline Grueskin. We need to band together and keep the pressure up against them. One is a map of Native American reservations the other is the DAPL. #NoDAPL pic.twitter.com/VQ5MXgXkpY  — Bart Starr Mistrot (@bartmistrot) October 27, 2016  Construction on the pipeline has resumed within sight of the confrontation on the state highway. While police came from north and things riled up, pipeline construction restarted to west. False flag much? pic.twitter.com/XxgC7vH11Z  — Jason Patinkin (@JasonPatinkin) October 27, 2016  “Protesters’ escalated unlawful behavior this weekend by setting up illegal roadblocks, trespassing onto private property and establishing an encampment, has forced law enforcement to respond at this time. As I said yesterday, I visited the protesters roadblock and requested them to remove the barricade and have protesters vacate the private property. However, the spokesperson at the roadblock, Mekasi Camp-Horinek of Oklahoma, told law enforcement they ‘were not moving’,” Morton County Sheriff Kyle Kirchmeier said in a statement . “I can’t stress it enough, this is a public safety issue. We cannot have protesters blocking county roads, blocking state highways, or trespassing on private property.” So @potus , this means you're going to remove the National Guard from Standing Rock and cancel DAPL, right? Right? #NoDAPL https://t.co/FVV5KsgUom",  # 1
    "RIO DE JANEIRO  —   Proud and beautiful as this seaside city is, glorious as the feats of athleticism and sportsmanship were these past two weeks, these Games underline that the Olympic model is fractured. As we depart, Rio de Janeiro is left with stadiums for which it has little use and swimming pools far removed from the working class who could dearly use them. Tens of thousands of residents were displaced, a golf course sits atop a former nature preserve, and the towers of the athletes’ village will have a second life as luxury housing. Rio’s state government turns its pockets inside out looking for money to pay salaries and to keep hospitals open. The army withdraws, and fears rise that crime will spiral. The International Olympic Committee has taken steps toward reform. But what it demands of nations is basically unchanged: Build us expensive monuments with a shelf life of two weeks. The I. O. C. has no hammer to bring down when cities fail to deliver on environmental promises. There is a more sustainable and less ruinous path, and I will get to that. First, I should wrestle with the challenge tossed down by my colleague Roger Cohen. He is an incisive, humane observer, and he argued that critics have the Rio Olympics all wrong. His perspective should be reckoned with, even as I disagreed with nearly every word. “I am tired, very tired,” Cohen wrote last week, “of reading negative stories about these Brazilian Olympics  —   the anger in the slums, the violence that continues (including the armed robbery of four American swimmers) the enduring gulf between rich and poor, the occasional organizational hassles, the Russian doping and the Brazilian mosquito, money that could supposedly have been spent better. ” He argued that the developed world harbors an antipathy for grand sporting events held in the developing world. I would turn his formulation on its head. The greatest honor we can pay to a nation is to take its maladies seriously. Brazil overthrew a dictatorship and remains a gloriously messy democracy. Yet police and paramilitary violence and human rights violations accompany its grand sporting events. Brazilian officials, desperate to tamp down the vicious drug gangs before the Olympics, let the army and the police ratchet up violence, to the point that last year the police committed   of the homicides in Rio. Such problems are not unique to Rio de Janeiro. “A major sporting event,” Amnesty International wrote of Rio, “tends to increase the human rights violations that already occur in a host city. ” Corruption scarred these Games, as contractors and politicians sluiced hundreds of millions of dollars into their coffers. Brave journalists here exposed this, and Brazilian prosecutors, to their credit, investigated the metastasizing scandals. Earlier this year, the scion of the nation’s largest construction companies, Odebrecht, an investor in the athletes’ housing, was sentenced to 19 years in prison for his role in another political scandal. We would do well to stanch any North American impulse toward  . The United States walked this twisted path, too. Our Salt Lake 2002 Winter Olympics were a bacchanalia of bribes, doping and corruption. My colleague Cohen claimed to detect another double standard he could recall no reporters poking around impoverished corners of London during its Summer Olympics. If that is so, that is our failure. London remains one of the globe’s more grossly unequal cities, and terrific activism was afoot during those Olympics. I recall walking through London’s East End before those Games and talking with the imams, bakers, construction workers, students and pastors who were the backbone of East London Communities Organization. They successfully pressured the London organizers to pay living wages to all who worked inside the Olympic perimeter. “They said they had obligations to their contractors,” Lina Jamoul, an organizer told me at the time. “We said: ‘What you have is a bureaucracy. That isn’t our problem. ’” Activists in London also applied concerted pressure to get the Olympic corporation to create 2, 000 units of affordable housing after the Games. The Rio Olympics offer nods toward a sustainable future. The handball courts, an example of nomadic architecture, will be dismantled and used to build schools. The media center will become dorms, and sections of the Olympic center could become a public park. There was, however, no movement to ensure livable wages for workers. And not a single flat in the towers where the athletes were housed will be reserved for   and poor Cariocas. The developers obtained the public land for free. A recessionary housing market does threaten their profit margin, which is a pity. More to the point, the loudest protests about these Olympics came not from Berkeley or Oxford but from within Rio de Janeiro. Labor unions, teachers and activists on either side of the coastal mountains that divide this city joined in marches. Their anger, coupled with a wounded economy, might account for the thousands of empty seats seen here these past two weeks. “We liked the talk of the Olympics at first,” noted Carla Maria Avesani, who runs a respected nutrition institute at Rio de Janeiro’s state university, which has suffered grievous cuts in pay and services. “Then Brazilians awoke from this dream. It became a nightmare. ” So what’s the solution? We could place the Games in Los Angeles, London or Paris, or seek a permanent home in Athens, which has historical and emotional resonance. “I very much understand the I. O. C. ’s desire to be more equitable in delivering the Olympics to the developing world,” said Allen Hershkowitz, the founder of Green Sports International, who is an environmental consultant to pro sports leagues and the Olympics. “But the Sochi and Rio Olympics indicate that there are such huge environmental, transportation, water and   questions. ” To retreat to the cities of the Northern Hemisphere, however, strikes me  —   and Hershkowitz  —   as a defeat. What if the I. O. C. instead had awarded Rio de Janeiro three cycles of Olympics: 2016, 2020 and 2024? City officials would have time to learn and grow, to ride out a bad recession and market Rio as a true Olympic city. Activists might have forced the Olympics to truly integrate into the lifeblood of the city, incorporating favelas no less than the beach community of Barra. They might have demanded that the athletes’ towers become   housing. They might demand living wages and true apprenticeship programs. Sewage treatment plants could come on line over the course of a decade. (I opposed the efforts of the former mayor Michael R. Bloomberg to bring the Olympics to New York, the cacophonous echo chamber that is my native city. But I admired his equitable proposal to put Olympic swimming pools and kayak routes in   city parks.) Olympic officials would also possess a stick: If city leaders did not offer measurable progress, the I. O. C. could withdraw the next two rounds of Games. Such a plan offers much for prospective Winter Olympics sites as well. Argentine or Chilean developers in the Andes  —   or Russians in Sochi, for that matter  —   would have time to build out   and environmentally sensitive winter sports resorts over 12 years, cushioned by the security of Olympic branding. (Tennis’s biggest events, from Wimbledon to the United States, French and Australian Opens, build out their businesses in this fashion year after year. The United States Tennis Association recently added renewable energy and recycling at Flushing Meadows.) “You can’t just do this for a   event it’s got to be a   planning,” Hershkowitz said. “Otherwise, you’re left with empty stadiums and a wrecked environment. ” In the end, the Olympics must change. Travel to Athens or Sochi and you will find a shattered vase of a model. Fewer and fewer cities desire that. In the past few years, residents of   host cities revolted, and Oslo and Boston, among other cities, canceled bids to host the Games. I’m no dour Puritan I love great athletics. My heart leapt as I watched Usain Bolt fly down the track and Simone Biles spin and flip as if exempt from the laws of gravity. To stand in the legendary Maracanã stadium for the gold medal men’s soccer game and listen as Brazilians transformed it into a joyous asylum was a wonder. Cariocas were gracious, from Copacabana baristas to an impoverished    Marie Auxiliadora, who invited me into her home in Favela do Mandela to listen to her upset with these   Games. One day, perhaps even next time, we can deliver a sustainable Olympics worthy of them.",  # 0
    "LJUBLJANA (Reuters) - Slovenians vote in a referendum on Sunday on whether the government should pursue construction of a 1-billion-euro ($1.20 billion) railway to its main Adriatic port - a test of the government s popularity ahead of elections next year. Given the opposition s lead in opinion polls and criticism of the project as too expensive, analysts see possible changes in the center-left coalition government ahead of the elections in June or July 2018 if the railway gets a thumbs-down in the referendum.  If (it) is rejected that would be a strong blow to the government, particularly the leading party SMC (Party of the Modern Centre),  Meta Roglic, political analyst at the daily Dnevnik, told Reuters. Opinion polls have forecast varying outcomes of the referendum but suggest turnout might not be sufficient to block the railway project. A poll published by daily Delo said voters would narrowly reject the project, while another released by Dnevnik said voters would back it by a 53-29 percent margin. Under Slovenian law, a minimum 20 percent of some 1.7 million eligible voters would be needed to sink the project, in addition to a majority of those voting. Civil society group Taxpayers Don t Give Up and the leading opposition group, the center-right Slovenian Democratic Party (SDS), pushed for the referendum, saying the railway line as projected by the government is too costly. The government says the 27-km (17-mile) rail link between Koper and the town of Divaca is necessary to speed up freight traffic to and from Slovenia s only port and improve the port s competitiveness. The government has received 44.3 million euros in EU funding for preparation of the track while neighboring Hungary said it is willing to invest about 200 million euros as it relies on Koper for much of its seaborne freight. The opposition SDS has led in opinion polls for most of the past year. But government parties hope to improve their ratings by advancing the railway project, raising family subsidies and the lowest tier of wages, and injecting more money into the state health sector in the coming months. ",  # 0
    "21st Century Wire says If you ve been following the protest and federal standoff outside of Burns, Oregon, you ll have seen 99% of the news coverage has been about the presence of  armed militia  and endless rumors of FBI  Waco Siege  plots   and not about the core legal issues surrounding the Hammond family.For more in-depth analysis on minimum sentencing laws, jury nullification and property rights, watch the following Liberty Report hosted by Congressman and former presidential candidate Ron Paul is joined by the executive director of the institute for peace and prosperity, Daniel McAdams, along with Jacob Sullum, senior editor from Reason.com  ",  # 1
    "CHARLOTTE, N.C. (Reuters) - Republican presidential nominee Donald Trump apologized on Thursday for past remarks that “may have caused personal pain” as he sought to refocus his message in the face of falling opinion poll numbers in his first speech since shaking up his campaign team this week. “Sometimes, in the heat of debate and speaking on a multitude of issues, you don’t choose the right words or you say the wrong thing,” Trump told a crowd in Charlotte, North Carolina. “I have done that, and I regret it, particularly where it may have caused personal pain. Too much is at stake for us to be consumed with these issues.” Trump did not cite any examples of such remarks. The New York businessman has made his “tough talk” and brash style a selling point of his campaign for the Nov. 8 election, rarely apologizing in the face of criticism even from within his own party for own party for comments insulting women, Muslims and Mexican immigrants. In his presidential announcement speech last year, he described some Mexican immigrants as “criminals and rapists.” He recently faced a barrage of criticism for belittling the family of a Muslim American soldier who died in Iraq in 2004, after the soldier’s father spoke out against Trump at the Democratic National Convention last month. The campaign for his Democratic rival, Hillary Clinton, quickly dismissed Trump’s apology, saying: “Donald Trump literally started his campaign by insulting people. “We learned tonight that his speechwriter and teleprompter knows he has much for which he should apologize,” the campaign said in a statement. “But that apology tonight is simply a well-written phrase until he tells us which of his many offensive, bullying and divisive comments he regrets — and changes his tune altogether.” Trailing Clinton in national opinion polls, Trump has tried to reset his campaign, announcing on Wednesday a shake-up of his senior campaign staff for the second time in less than two months. In the past week, he has abandoned his free-wheeling style of campaigning, instead using a teleprompter at every rally. Trump also began adding non-rally events to his campaign schedule, visiting a police lodge on Thursday afternoon and hosting a roundtable on Wednesday morning. Previously, Trump had eschewed such events that historically comprise a significant portion of a candidate’s time. Trump’s speech on Thursday night, echoing remarks earlier in the week in Wisconsin, reached out to minority voters and accused Clinton of being dishonest. “So while sometimes I can be too honest, Hillary Clinton is the exact opposite: She never tells the truth,” Trump said. “In this journey, I will never lie to you. I will never tell you something I do not believe.” Trump’s new campaign manager, Kellyanne Conway, promised on Thursday he would stick to a more disciplined and uplifting message to voters in the final dash to Election Day without crimping his freewheeling style. Conway said the candidate’s White House bid could preserve his “authenticity” and still move past a long string of controversies to focus on issues. “We would like to take an uplifting, optimistic, policy-centric message directly to the American people,” Conway told CBS’s “This Morning,” adding she was confident the former reality television star could stick to a sharper message. Wednesday’s campaign reshuffle also brought on Steve Bannon, a combative conservative who headed the Breitbart News website, as chief executive officer, in a move seen as giving Trump a chance to emphasize his unconventional style. Conway said the more disciplined approach by Trump, who has never held elective office, would not mean jettisoning his more off-the-cuff and unpredictable style. “We’re going to make sure Donald Trump is comfortable about being in his own skin - that he doesn’t lose that authenticity that you simply can’t buy and a pollster can’t give you,” she said on CNN. “Let him be him, in this sense.” Trump has so far been unable to extend his appeal beyond white middle-class voters who pack his rallies. Conway said her rise to campaign manager showed Trump valued a more policy-driven approach that would appeal to women and independent voters. “The way to speak to women and all Americans is through issues. We’ve got to get away from this content-free campaign and on to the substance,” she said.     The campaign will put two ads on the air in five states this week, the first campaign ads Trump has aired in the general election, Conway said. He will also continue a series of policy-focused speeches after giving speeches in the past two weeks on jobs, national security and law and order. “He’s going to give these policy speeches,” she said on CNN. “You’ll see more of those. Next week is immigration week, followed by education week.” ",  # 0
    "(CNN) On Tuesday, students went to classes as they usually do. Football players intended to take the field in preparation for their game against Brigham Young University on Saturday.  But something was very different at the University of Missouri campus.  Students on Tuesday woke up to what protesters call a small but important victory: a weeks-long protest movement that ousted both the university president and the school's chancellor.  African-American students at Missouri have long complained of a mealy-mouthed response by school leaders in dealing with racism on the overwhelmingly white Columbia campus. Black student leaders have conveyed their displeasure over students openly using racial slurs and other incidents.  \"This is just a beginning in dismantling systems of oppression in higher education, specifically the UM system,\" said Marshall Allen, a member of the protest group Concerned Student 1950.  The speed of Wolfe's resignation shocked many. As late as Sunday, Wolfe didn't sound like a man who planned to leave his job, putting out a statement expressing a desire to have an \"ongoing dialogue to address these very complex, societal issues.\"  But the tide had already turned against him Saturday night, when about 30 black members of the Missouri Tigers football team declared in a tweet that they wouldn't play until Wolfe was gone. By Sunday, more members of the team, black and white, and head coach Gary Pinkel publicly backed the players, and the media started paying attention.  By Monday morning, student groups were calling for walkouts and some faculty offered protesting students their support. The calls for his resignation grew louder.  So Wolfe -- who had presided over the university system, which includes the main University of Missouri campus in Columbia, along with the University of Missouri-St. Louis, University of Missouri-Kansas City and Missouri University of Science and Technology -- stepped down, saying he took \"full responsibility for the inaction that has occurred\" and urged the university community to listen to each other's problems.  \"It is my belief we stopped listening to each other; we didn't respond or react,\" he said. \"Use my resignation to heal and start talking again.\"  Students, faculty and staff converged on the Carnahan Quad after Wolfe's announcement. There, they linked arms and swayed side to side, singing, \"We Shall Overcome.\"  Though the protesting students and some faculty say racial problems on campus go back decades, the current crisis took flight back in September, when Student Government President Payton Head took to Facebook to complain about bigotry and anti-homosexual and anti-transgender attitudes at the school after people riding in the back of a pickup truck screamed racial slurs at him.  \"For those of you who wonder why I'm always talking about the importance of inclusion and respect, it's because I've experienced moments like this multiple times at THIS university, making me not feel included here,\" he wrote.  In early October, a drunken white student disrupted the Legion of Black Collegians, an African-American student group, while the group prepped for homecoming and used a racial slur when he was asked to leave.  Later that month, Concerned Student 1950 -- named for the year African-American students were first admitted to the university -- issued a list of demands, including an apology from Wolfe, his removal from office and a more comprehensive racial awareness and inclusion curriculum overseen by minority students and faculty.  L'Damian Washington, a former wide receiver on the football team, said that he was happy the team was able to add leverage to Butler's hunger strike and that the protest wasn't just about Missouri or being black. It was about discrimination in all forms everywhere, he said.  \"Only a minority knows what it feels like to be a minority on campus,\" he said.  It's difficult to put yourself in others' shoes, he said, explaining that even though he and Butler are black, his experience -- as a football player -- on campus was different from Butler's.  A statement from head coach Pinkel and Missouri athletic director Mack Rhoades, released after Wolfe's announcement, said football activities would resume Tuesday.  The two men addressed the media Monday afternoon.  \"There's no playbook. There's no script for what all of us have been dealing with. And I think, certainly, it's been also a great learning experience for everyone involved,\" said Rhoades.  \"As we move forward, it's paramount as a campus and a community that this not divide us, but rather bring us together to listen, to grow, to understand and to create positive change,\" the athletic director said.  If the Tigers had failed to take the field Saturday against the Brigham Young University Cougars at Kansas City's Arrowhead Stadium, the home of the NFL's Chiefs, the team would have been forced to pay a cancellation fee of $1 million, according to a copy of the contract published in The Kansas City Star earlier this year.  \"Our team's excited about getting going again and playing, and we're looking forward to our game against BYU this weekend,\" Pinkel told reporters, saying he got involved because he supports his players and because Butler's life was \"on the line.\"  \"My support of my players had nothing to do with anyone losing their job. With something like this, football became secondary,\" Pinkel said. \"I just know my players were suffering and they felt awful, and again, I'm like their dad, and I'm going to help them in any way I can.\"  The University of Missouri's Columbia campus has a population of 35,000 students. The undergraduate student body is about 79% white, while African-Americans make up roughly 8% of undergraduates. The school's faculty is also more 70% white with black representation of just over 3%, according to the university.",  # 0
    "A caller to British radio host Katie Hopkins whose name is Jessica, called into Katie s show to relay a story to her about her formerly  bleeding heart liberal  husband who has come to have a  hatred of a certain demographic  after helping migrants who are arriving in migrant rescue boats. Jessica told Hopkins,  My husband is turning into a man I don t recognize anymore because of what he has physically seen. Jessica added that it was the way male migrants behaved in particular, that led to his change of heart.She told Katie:  It was a really, really traumatic time for him. He was doing three-week stints and they were picking bodies out of the sea. Babies, women, it was awful. But at the same time, he said they were processing men, adults, in their 30s who all had phones with Isis stuff on the phones, they were being separated. When they were being handed over to the authorities with the information explaining that these guys had all this on their phones, and what their ages were, and they were just being processed through. There was nothing, they were just being separated out, they just all went through with everything else, it broke his heart, because he was dealing with families who were in such a terrible state with young babies who so clearly needed rescuing. And yet you had adult males that were pushing women and children to the back of the queue they felt that they were more important. It s just an awful situation, and this was last year. He can t handle the way these women and children are treated by these men. There were hundreds and hundreds of women, children, men who were being picked out of the sea. Not all of them were refugees. He came home heartbroken. Absolutely heartbroken, because he s saying  I don t know how Europe is going to cope .She continued:  How are we supposed to process that as individuals? You know, he still has images that he has to deal with over a year later. We forget about the medics who deal with this, the soldiers that are dealing with this. The people that are processing these people. It s not just how are we going to process all these people, but how are we going to deal with the trauma of having to process these people? Having to make these decisions? Who do you treat first? A special thank you to Paul Joseph Watson for this video: Man who helped rescue  refugees  said many were men with ISIS content on phones who shoved women & children out of the way. pic.twitter.com/wH08TLf4xY  Paul Joseph Watson  (@PrisonPlanet) May 3, 2017Katie asked the caller how her husband is doing currently after seeing the vast number of migrants rescued this year already.Jessica said:  My husband is turning into, it s why I stopped him doing it, he s turning into a man I don t recognise anymore. Because my family think I m a bleeding heart liberal, and you know, I would just wave everyone in and say  come on, we ll help , and my husband was of a very similar opinion that people needed help. He s now turning into someone I don t recognise, because of what he has physically seen. He is starting to say this is not all people looking for help, there are people here who are out for their own self interest. She added:  It s just awful. And now he is building a hatred of a certain demograph[ic], that he has been exposed to because he s seen the way they behave.   LBCIs it too late for Europe? Have they allowed this unabated invasion of migrants from majority Muslim nations to their once  culturally distinct nations for too long? What will these bleeding heart nations look like in 5 years? How long will Americans sit back and allow bleeding heart liberals to call them  racists  or  xenophobes  because they don t want to see the fundamental transformation of the United States that Barack Obama promised would happen if we elected him as our president. Can President Trump stop this nightmare without the help of a Republican majority in both the House and Senate?",  # 1
    "During Thursday night s undercard debate on Fox Business Channel, candidate Rick Santorum, who s somehow still in the race, asked viewers for a very special favor.The former Pennsylvania senator said,  Go and Google Rick Santorum and Hillary Clinton. You ll see a five-minute debate. I ll let you decide who won the debate.  He continued,  I ll tell you who won, I know I m out of time, I m going to take some of Rand Paul s time here for a second. Yes, Rick Santorum wants us to  Google  him. Why is this so goddamn funny?There was a time when the absolute last thing Rick Santorum ever wanted you to do was to  Google  him. Not too long ago, LGBT commentator Dan Savage produced one of the greatest political pranks of all time. In response to Santorum saying publicly that same-sex marriage will lead to man-on-dog sex, Savage re-defined the word  Santorum  to mean the  frothy mixture of lube and fecal matter that is sometimes the byproduct of anal sex. Worse, enemies of Santorum rigged it so that googling the Republican s name would only produce results that led to that definition. Hence, Santorum probably didn t spend much time asking people to Google him.Until Thursday night. Brave of him because, fortunately, Google no longer produces the new definition as a search result. But people who despise the Republican haven t forgotten. Twitter erupted with reactions to the inadvertent reference to Google.Historic moment: Rick Santorum just told people to go Google Rick Santorum.  Daniel Dale (@ddale8) January 14, 2016People who should never say  Google me  -Rick Santorum, and well, me. ?  Sydney Leathers (@sydneyelainexo) January 15, 2016 Hi, I m Rick Santorum. Please don t Google me.  Andrew Katz (@katz) September 16, 2015 Go and Google Rick Santorum    nooooooooooooo  Brett LoGiurato (@BrettLoGiurato) January 14, 2016DO NOT GOOGLE SANTORUM  T. Becket Adams (@BecketAdams) January 14, 2016Hi I m Rick Santorum. I don t need to introduce myself. Just google me. Just make sure your kids aren t in the room.  Donald J. Trump (@realDonaIdTromp) September 16, 2015Couldn t happen to a nicer guy, huh?Featured image via video screen grab.",  # 1
    "MELBOURNE, Australia  —   The sibling rivalry, at least on the tennis tour, started right here at the Australian Open for the Williams sisters. It was 1998, and older sister Venus beat younger sister Serena,   (4)   in a   match that  —   as intrusive as it felt to watch  —   surely drew more attention than any   match in history between a pair of Australian Open debutantes. The fascination in their dynamic and their futures was there from the start in Melbourne Park, known then as Flinders Park when it had only one stadium with a retractable roof instead of three. A picture of Venus consoling Serena after the match was on the front page of The New York Times. Though it would be tempting to label their Australian Open final on Saturday as a   moment and to speculate that it might be their last meeting at this late a stage of a Grand Slam tournament, it seems best to resist the temptation. The Williams sisters have taught us a lot about the limits of conventional tennis wisdom through the years. And so, even if 19 years have passed and Serena is now 35 and Venus 36, it is wise to avoid fencing them in again after they have run roughshod over so many other preconceptions. “I watched Venus today celebrating after she won the semifinal like she was a    girl, and it made you want to cry for joy just watching her,” said Marion Bartoli, a former Wimbledon champion. “Such a powerful image, and it makes you think about all those questions she was getting: ‘When are you retiring? Have you thought about retiring? How much longer?’ “You must let the champions decide when the right moment comes. ” The Williamses are both great champions, even if Serena is clearly the greater player with her 22 Grand Slam singles titles and her long run at No. 1, a spot she can reclaim from Angelique Kerber with a win Saturday. Serena has been the most prolific Grand Slam winner after age 30 in tennis history, and she is back in rare form again after another extended break at the end of 2016. She disconnected completely from the game and physical training initially and had to push hard to get back in shape in November and December. It worked. She has not dropped a set here despite a challenging draw, nor has she even been pushed to a tiebreaker. Newly engaged to the American technology entrepreneur Alexis Ohanian, who has watched her matches from the players box, and seemingly refreshed, Serena deserves to be the favorite to win her 23rd major singles title and break her tie with Steffi Graf for the highest total in the Open era. In this tournament, Serena has beaten two former members of the top 10  —   Belinda Bencic and Lucie Safarova  —   and one current member, the   No. 9 seed Johanna Konta. Venus’s draw has been soft by comparison, devoid of top 10 players  —   past or present  —   and including only one seeded player: No. 24 Anastasia Pavlyuchenkova. On Thursday, she had to scrap and come back to win,   (3)     against the powerful unseeded American CoCo Vandeweghe, while Serena cruised past the unseeded Croat Mirjana      . Serena, who already holds a   edge over her sister, could be the fresher player, too, on Saturday. But the psychology remains complex and the fallout unpredictable, even after all these years. “When I’m playing on the court with her, I think I’m playing the best competitor in the game,” Venus said. “I don’t think I’m chump change either, you know. I can compete against any odds. No matter what, I can get out there, and I compete. ” They have not played since the 2015 United States Open, when Serena won,       in a quarterfinal in which Venus attacked, often successfully, from the start but had no answer in the end for Serena’s ultimate weapon: her first serve. It was an intense match in which the big crowd in Arthur Ashe Stadium seemed more reflective than fully engaged one in which Serena’s celebration was understandably subdued with her sister across the net, even if their matches are no longer the awkward, constricted affairs of their early years. Saturday’s final in Melbourne could be intriguing on multiple levels, in part because of the Australian public. Venus is viewed here, as elsewhere, as a sympathetic figure: the older sister who has handled the younger’s greater tennis success unselfishly and with dignity. And though both sisters have had to cope with major health problems and family tragedy, with the murder of their half sister Yetunde Price in 2003, Venus is the one whose tennis fortunes dipped more dramatically. A   Grand Slam singles champion and a former No. 1, she did not advance past the third round in any major event in singles from late 2011 to the end of the 2014 season. She was a major star reduced to a minor role, largely because of an autoimmune disorder  —   Sjogren’s syndrome, diagnosed in 2011  —   that sapped her strength and endurance. When Russian hackers breached the World   Agency’s databases last fall, it was revealed that Venus had needed 13   exemptions for drugs in recent years. The retirement questions to which Bartoli referred started during that period. But Venus’s ability to cope with her condition has improved, and after rejoining the top 10 in 2015, she reached the semifinals at Wimbledon last year and then the final here. “She never even thought of the word retire,” said David Witt, her coach and hitting partner of 10 years. “I just think when she got diagnosed, it was a step back, a shock. She’s learned a lot about how to deal with it and her body, how to eat, how to manage it. “There are days she can’t work as hard as she wants to work. Some days it’s maybe not smart to do it because it will then hurt you for two or three more days. Where she is now in her career, she has to listen to her body, and I don’t think she really needs to go out and hit balls for two hours. ” Witt said there were no more    sessions in the   or in time off tour: just one session in the morning and then gym work, primarily sprints, core strengthening and flexibility. “It took her years and years to realize that stretching is important and can keep you healthier,” he said. “The more flexible you are, you’re not going to strain or pull anything. I’ve been with her 10 years, and I think it took seven years to get her to stretch. She likes to do a lot of dancing, and that consists of a lot of stretching and being flexible, so I think that’s helped. ” Her dance skills were in evidence Thursday as she pirouetted after beating Vandeweghe, but what will linger longest in memory were her screams of delight at having conquered an inspired young opponent in a semifinal. It was a moment she described as “just joy. ” “You could really see the happiness on her face,” Serena said. “I’ve been there when she was down and out of it, and back and in it. I’ve been there for all those moments, so I just really was oh so happy. ” As visceral as her reaction on court was, she was nothing but considered in the interview room. “I think why people love sport so much is because you see everything in a line,” Venus said. “In that moment, there is no  . There’s no retake. There is no  . It’s triumph and disaster witnessed in real time. “This is why people live and die for sport, because you can’t fake it. You can’t. It’s either you do it or you don’t. People relate to the champion. They also relate to the person who didn’t win, because we all have those moments in our life. ” This will be Venus’s first major singles final since she lost to Serena in straight sets in the 2009 Wimbledon final, and her first match against Serena in Melbourne since the 2003 final when Serena won her fourth Grand Slam title in a row, having defeated Venus in all four finals. “It’s just amazing,” said Rennae Stubbs, the Australian star who first met the sisters before they joined the tour. “They came onto the scene at age 15 and 16 with the beads and the hair and the exuberance, and here they are: mature, remarkable young women at 35 and 36. No matter what anyone says to me, their story from start to finish is the greatest sports story ever. ” And if the Williamses have taught us anything along the way, it is that the story is not finished until they say it is.",  # 0
    "Wow! Poor Kathy Griffin The toxic  comedian  found herself facing a tsunami-sized backlash after she held a severed, and bloodied head of President Trump, during a comedy skit earlier in the year. And now, the former  funny girl  is telling anyone who will listen, that she s furious no one wants to hire her. Kathy Griffin starts her homemade 5-minute video by whining to viewers about all of the online  hate  by  trolls  that she s receiving. Griffin then goes on to trash the other  not funny  comedian Lena Dunham, who wrote about sexually assaulting her little sister, for defending a Hollywood actor who s been accused of sexually assaulting a female actress. Her comments that follow the Dunham remarks make no sense, although she seems to be attempting to make the point that the equally trashy liberal comedian, Lena Dunham, is still getting work, and how Griffin just doesn t think that it s fair. (After all, Dunham hates Trump as much as her right?)Griffin tells the camera that appears to be set up in her living room,  I just want you guys to know that I am fully in the middle of a blacklist like I m in the middle of a Hollywood blacklist. It is real. I m not booked on any talk shows. Griffen continued to whine about her lack of jobs,  I just want you guys to know, that when I get home, I do not have one single day of paid work in front of me, and the people who want me to go back and do clubs and do 10 minutes again I don t mean to be an asshole, but I m not gonna do that. I ve worked way too hard to go back and work for free, and do the club scene again because this is some bullshit. Because I ve been blacklisted.Griffin then goes on to tell her fellow Hollywood libs to stop sending her requests for donations to their fundraisers because she doesn t have any money. She goes on to explain that she s spent all of her money on lawyers because she believes in something. Griffin can t quite find a way to articulate what that something is, but she s apparently learned her lesson and has refrained from exposing her true obsession and desire, which was to destroy Donald Trump.Watch: ",  # 1
    "DAR ES SALAAM (Reuters) - A Tanzanian minister who was named in reports on the results of an investigation into the diamond mining industry has resigned, state-run television broadcaster TBC1 said on Thursday. TBC1 said that George Simbachawene, the Minister of State in the President s Office, had relinquished his post. ",  # 0
    "SEOUL, South Korea  —   Bowing to public pressure, an official in South Korea’s   city said on Friday that activists could put up a statue representing Korean women forced into sexual slavery by Japan during World War II, a year after the two countries said they had put that emotional issue behind them. The bronze,   statue, of a girl in traditional Korean dress sitting in a chair, had been raised without permission on Wednesday on a sidewalk near the Japanese Consulate in Busan. The police removed the statue, dispersing activists who tried to stop them, but on Friday an official said it could be reinstated. “I apologize to many citizens,” Park   mayor of the ward in Busan where the consulate sits, said at a news conference as he announced the statue’s return. “This is an issue between the two nations, and I realize it’s too much for a local office like mine to handle. ” Activists quickly put the statue back in place. Since the statue’s removal on Wednesday, the ward’s office had been overwhelmed with angry phone calls, and its website temporarily froze because so many people were visiting to leave hostile comments. Some called Mr. Park a “  collaborator,” a grave insult in South Korea, where bitter memories of Japan’s colonial rule in the first half of the 20th century still run deep. The consulate had strongly objected to the presence of the statue, which the activists installed near one of its gates, where diplomats would see it as they left their offices. On Friday, the consulate lodged a protest and request to Busan city to remove the statue. Japan registered complaints with the governments of South Korea and Busan. Shinsuke Sugiyama, Japan’s vice minister for foreign affairs, told the South Korean ambassador in Japan that the statue “went against the spirit of the   Korea agreement concluded at the end of last year and is extremely regrettable,” adding that it would have an “unfavorable impact on the relationship between Japan and South Korea, as well as disturb the security of the consulate. ” Mr. Sugiyama said the Japanese government would continue to strongly request that South Korea remove the statue. Dozens of identical statues have been put up in South Korea since 2011, when the first one, placed near the Japanese Embassy in Seoul, caused a diplomatic uproar. But the statue in Busan was only the second to be installed near a Japanese diplomatic mission. Wednesday was the first anniversary of a landmark agreement between South Korea and Japan to resolve their dispute over the extent of Tokyo’s responsibility for what happened to the wartime sex slaves, or “comfort women,” as they were euphemistically known. Both sides called that deal  —   in which Japan apologized and promised $8. 3 million to care for the surviving women, in return for South Korea’s promise not to press any future claims  —   a “final and irreversible resolution” to the dispute, which had become a serious obstacle in their relations. But the deal fell short of the survivors’ demand that Japan pay formal reparations and accept legal responsibility for what they endured. And it proved to be one of the most unpopular decisions made by President Park   whose powers have been suspended since the National Assembly voted for her impeachment this month over a corruption scandal. In the year since the agreement was reached, students have been camping out by the statue near the Japanese Embassy to ensure it is not removed, and five more statues have been installed around the country, with money raised through donations. Opposition parties have long denounced the deal, and since Ms. Park’s powers were suspended, they have increased pressure on the government to reconsider it, along with some of her other key policy decisions. If the Constitutional Court ratifies Ms. Park’s impeachment in the coming months, formally removing her from office, a presidential election will be held. Two leading contenders to succeed her have said that they will seek changes to the deal with Japan if elected. Japan’s defense minister, Tomomi Inada, stirred bitter wartime memories in South Korea on Thursday by visiting the Yasukuni Shrine in Tokyo, which commemorates Japan’s war dead, including a number of officers convicted of war crimes during World War II.",  # 0
    "TUNIS  —   Since their revolution in 2011, Tunisians have tussled among themselves to define their new identity after successive dictatorships. Belatedly, they have started reclaiming their history as well. Dictatorships have a way of manipulating historical narratives. So alongside any of the most pressing issues of the day, the past, too, is in play. The struggle to shape the past, and give it new authenticity, can be witnessed all around the Tunisian capital. Last summer, the Tunisian government restored a statue of Habib Bourguiba, the founder and first president of the republic, to its original place on the capital’s main avenue. The famous symbol of Mr. Bourguiba  —   astride his horse, arm aloft rallying Tunisians to independence  —   had been banished to a seaside neighborhood in 1987, after he was ousted by his prime minister, Zine   Ben Ali. (Mr. Ben Ali was himself overthrown decades later during the Arab Spring uprising.) The current president, Beji Caid Essebsi, who served in Mr. Bourguiba’s cabinet, was the main driver behind reinstating the statue, which he said was intended to reinforce his call for national unity and a strong state. Mr. Bourguiba’s statue had replaced a humiliating symbol of colonialism: an image of the colonialist politician Jules Ferry with a Tunisian woman at his feet proffering an olive branch, he reminded Tunisians. “That used to be the symbol of colonialism, and Bourguiba is the symbol of freedom, of independence and of the modern state,” he said at the unveiling. Many Tunisians do indeed respect Mr. Bourguiba as the father of the independent nation, yet he was also a dictator who imprisoned his opponents and clung to power for over 30 years. The young revolutionaries who spearheaded the Arab Spring uprising greeted the return of the statue with a flurry of jokes on social media about the waste of money. Far from cementing his reputation, moving the statue opened a new debate around Mr. Bourguiba’s legacy. “People talk of a ‘new Bourguibism,’ but it does not exist,” Med Dhia Hammami, a journalist and activist, said. “There are no ideas coming out of that. ” With the new freedom gained since their revolution, some of Mr. Bourguiba’s victims are being given a chance to set the record straight at public hearings of the Truth and Dignity Commission. One of them, Hamadi Ghars, 85, a journalist in the independence movement, gave a damning portrayal of Mr. Bourguiba at the commission’s first public hearing. Mr. Ghars described how Mr. Bourguiba had betrayed thousands of his fellow   fighters when he accepted an interim deal with the French government that allowed the colonizers’ security forces to continue controlling the country. “We were happy with independence and were waiting for reconciliation,” Mr. Ghars said. Yet Mr. Bourguiba then ordered them to surrender to the French, who bombed the fighters in the mountains. “They killed hundreds of young Tunisian men,” he recalled. “I don’t doubt that their bodies are still in the mountains and rivers today. ” Mr. Ghars was captured and sentenced by the French to 20 years imprisonment and 10 years hard labor, which he served under Mr. Bourguiba. “They destroyed a dream of unity,” he said. “Usually history is written by the victors, but this is the opposite,” said Adel Maizi, the president for preservation of memory at the commission. “These testimonies will reveal the truth. ” “Dictatorship always tries to keep things secret,” he explained. “These kinds of testimonies are against forgetting. They will preserve memory for the country and serve as a way to guard such things happening in the future. ” Tunisians are discovering other forgotten chapters of their past that the two dictators tried to obscure. Crowds have flocked in recent weeks to an exhibition on the opulent, and at times profligate,   reign of tribal princes when Tunisia was part of the Ottoman Empire. The exhibit is housed in the   Ksar   palace. Once a royal residence, it was used as a barracks and a hospital and has been shuttered since the 1980s, its collection of French paintings and Ottoman magnificence hidden from view. For a year, the Rambourg Foundation has been restoring a part of the palace and its collections to unveil the history of Tunisia between 1830 and 1881, the period when princely rulers known as beys ruled the country before it was colonized by France. Ridha Moumni, a curator of the exhibit, insists it is not political, but a matter of history. Yet he is displaying events that Tunisia’s dictators sought to suppress. “We have a very rich heritage that no one knew about,” Mr. Moumni said. “Our goal was to show that Tunisian modernity did not start with independence or colonization. ” The exhibition offers many Tunisians their first glimpse of the   court through royal portraits and rich costumery. More important, it provides a history lesson on the significant reforms of the era  —   the founding of the army, the drafting of a constitution and development of diplomatic relations  —   that helped forge a nation. Among the original documents on display, one abolished slavery in 1846  —   before the United States did so, Mr. Moumni pointed out. Others include a constitution drafted in 1860 that recognizes the rights of all citizens, including Christian and Jewish minorities, and census registers, in Hebrew and Arabic, belonging to Tunisia’s ancient Jewish community. Students and a university teacher touring the exhibition said it was the first time they had seen images of Tunisian royalty. Another discovery is the diversity of Tunisia’s leaders  —   from the Christian foreign minister, Giuseppe Raffo, to a Circassian general, Kheireddine Pasha, and the former slave Mustapha Khaznadar, who married into the royal family and rose to become the bey himself. One of the most popular displays  —   in the throne room, and beside the very table upon which it was signed  —   is the Treaty of Bardo, under which the beys, facing a crumbling dynasty and mounting debts, surrendered to become a French protectorate. The beys were subsequently blamed for clearing the way for seven decades of French rule and were condemned to obscurity by Mr. Bourguiba after Tunisia’s independence. “He wanted to erase this from the national memory,” Mr. Moumni said.",  # 0
    "Obama Leaves U.S.A $9,335,000,000,000 Deeper in Debt",  # 1
    "BERLIN (Reuters) - Activists from the online campaign group Avaaz tore down a cardboard wall near the site of the former Berlin Wall on Friday to urge American expats in Germany to vote against Republican nominee Donald Trump in the U.S. election on Nov. 8. Many identify Trump with his proposal to build a huge border wall between the United States and Mexico to keep migrants out; moreover, the event was staged in front of the Brandenburg Gate, near the place where U.S. president Ronald Reagan publicly urged Soviet leader Mikhail Gorbachev in 1987 to “tear down this wall”. Holding red and blue placards saying “Stop Trump” and “Tear Down Trump’s Wall”, dozens of cheering activists toppled the 2.5 meter (8.2 foot) structure, emblazoned with a picture of Trump, with their hands and rubber mallets. Many of the 150 or so onlookers chanted “The wall must come down!” as it was knocked over.  Around 8 million Americans live abroad but only 12 percent of them voted in the 2012 election, Avaaz said in a statement. “Americans abroad are the ‘Swing State’ that nobody knows about, who could, however, decide the vote this year. A large number of them live in Germany,” said Emma Ruby-Sachs, deputy director of Avaaz. In a Reuters/Ipsos poll published last week, 42 percent of likely voters supported Clinton, a former secretary of state, and 38 percent backed the Republican real estate tycoon. ",  # 0
    "They are   Donald J. Trump’s disrupters. Seven men and one woman named by Mr. Trump to run vast government agencies share a common trait: once they are confirmed, their presence is meant to unnerve  —   and maybe even outright undermine  —   the bureaucracies they are about to lead. Some of those chosen  —   17 picks so far for federal agencies and five for the White House  —   are among the most radical selections in recent history. Other presidents’ nominees, even when controversial, were often veterans of the Washington bureaucracy and generally believed in it. But a number of Mr. Trump’s most important selections have no experience in federal government and a great drive to undo it. Scott Pruitt, the Oklahoma state attorney general who was picked to lead the E. P. A. rejects the established science of   climate change and has built his career on fighting environmental regulations. At the Education Department, Betsy DeVos wants to steer government money away from traditional public schools. Rick Perry was picked to head the Energy Department  —   unless he eliminates it, as he once promised. Representative Mick Mulvaney of South Carolina, the conservative Republican who was chosen as White House budget director, refused to back the 2011 deal to raise the federal debt limit and helped to bring the United States to the brink of default. “Donald Trump ran to make the governing people uncomfortable,” said Andrew H. Card Jr. who served as chief of staff to former President George W. Bush and as transportation secretary for his father. “He clearly picked people to lead some of these departments who will be challenging to the insiders. ” As Mr. Trump finishes announcing his choices for cabinet and senior White House aides, a picture is emerging of an administration with little ideological cohesion and no single animating purpose. They are neither all   conservatives, nor all mainstream conservatives. They are not all insiders or outsiders. Some have known Mr. Trump for years. Others met him in the weeks since he was elected. Instead, Mr. Trump has recruited people to lead his government who can be roughly grouped into categories that appeal to the different parts of his psyche. In addition to the disrupters, there are the dealmakers, the loyalists, the establishment and the generals. Mr. Trump’s administration so far reflects the people who worked for and advised him during the campaign. It is mostly men, and mostly white. Many of his picks are extraordinarily rich, though not all. Loyalty is at a premium  —   except when it isn’t. (Lt. Gen. Michael T. Flynn, Mr. Trump’s choice for national security adviser, was an early supporter who was rewarded. Todd Ricketts, who may become deputy commerce secretary, was a leader of the “Never Trump” movement.) There is “no overarching theory,” said Pete Wehner, who was deputy director of speechwriting for Mr. Bush and a fierce critic of Mr. Trump during the campaign. “He’s not being driven by the usual impulses, which would be policy or ideology or political philosophy. ” Here is a look at the groups: Few things made Mr. Trump’s campaign supporters roar with approval more than when he hammered the “political class in Washington,” telling people at his rallies that the government had betrayed them, and vowing that change was coming. At many agencies, it may soon arrive. In addition to Ms. DeVos, Mr. Pruitt, Mr. Perry and Mr. Mulvaney, the disrupters include Ben Carson, chosen to lead the Housing Department Andrew Puzder as labor secretary Representative Tom Price of Georgia to oversee the Health and Human Services Department and Senator Jeff Sessions of Alabama as attorney general. The disrupters appear to have been picked in part for their singular ability to shake up the normal course of business at their agencies. Mr. Sessions, whose 1986 nomination as a federal judge was rejected by the   Senate because of testimony from colleagues that he was a racist, is likely to push for wholesale changes at the Justice Department by imposing conservative positions on immigration, terrorism, crime, drugs and guns. He is almost certain to reverse   efforts to confront civil rights violations, sentencing disparities and police abuses. Mr. Carson, a surgeon who has no experience in housing or running a large government agency, has already put housing advocates on notice that he does not think much of current programs for the poor. Mr. Puzder, who owns   chains and is opposed to raising the minimum wage, is a longtime critic of his department’s approach. Mr. Price, who waged a crusade against the Affordable Care Act in Congress, could soon be in a position at H. H. S. to lead the drive to repeal and replace it. Mr. Trump demonstrated his desire for a disruptive force when he named Mr. Pruitt to head the nation’s top environmental agency. While saying that his administration “strongly believes in environmental protection,” he previewed the kind of change he is looking for. “For too long, the Environmental Protection Agency has spent taxpayer dollars on an      agenda that has destroyed millions of jobs,” he said, adding that Mr. Pruitt “will reverse this trend. ” As he creates an administration in his own image, perhaps no category is more natural for the author of “The Art of the Deal” than the dealmakers. Mr. Trump, who puts a premium on the ability to negotiate in business and finance, has chosen Wilbur Ross, a billionaire investor, to lead the Commerce Department Steven Mnuchin of Goldman Sachs to lead the Treasury Department and Rex W. Tillerson, the chairman of Exxon Mobil, who has a longtime relationship with President Vladimir V. Putin of Russia, to be secretary of state. “To me, a great advantage is he knows many of the players, and he knows them well,” Mr. Trump said last week during an interview about Mr. Tillerson on “Fox News Sunday. ” “He does massive deals in Russia. He does massive deals for the company  —   not for himself  —   for the company. ” Mr. Mnuchin made a fortune through financial deals and now becomes the nation’s top financial official. Mr. Ross, a “vulture” investor on Wall Street, bought nearly dead companies and made huge profits when his turnaround strategies worked. Like most future presidents, Mr. Trump has also positioned loyal, trusted aides to be close to him at the White House. Four of the loyalists  —   Reince Priebus, who will be White House chief of staff Stephen K. Bannon, chief strategist Donald F. McGahn II, White House counsel and Mr. Flynn, national security adviser  —   will occupy the most coveted real estate in the West Wing. The president has also tapped Linda McMahon, the former chief executive of World Wrestling Entertainment, to oversee the Small Business Administration. Ms. McMahon has known Mr. Trump for years, long before he considered running for president. In 2007 on W. W. E. Mr. Trump took down Vince McMahon, Ms. McMahon’s husband and the chairman of the organization, in what was billed as the “Battle of the Billionaires. ” All five loyalists stuck by Mr. Trump during the toughest of times in the campaign. Mr. Priebus, the departing chairman of the Republican Party, came late to joining the Trump team, but refused to waver publicly when the Washington establishment was rushing to condemn Mr. Trump during various scandals, particularly a leaked “Access Hollywood” recording that captured Mr. Trump boasting about grabbing women by their genitals. Mr. Bannon, who has been accused of embracing white nationalist views as the head of Breitbart News, was always there, whispering advice to Mr. Trump in the background, if not officially in charge of campaign strategy. Mr. Flynn and Mr. McGahn are veterans of Mr. Trump’s campaign world, having secured their posts in the administration through longevity. Loyalty goes only so far, however. Four of Mr. Trump’s most avid supporters so far have no role in the new administration: Chris Christie, the governor of New Jersey Newt Gingrich, the former House speaker Mike Huckabee, the former governor of Arkansas and Rudolph W. Giuliani, the former mayor of New York City. If there was one group Mr. Trump repeatedly derided on the campaign trail, it was the establishment. It was filled with the “stupid” people who ran the government poorly, Mr. Trump said. It represented the Washington swamp that he pledged to drain. When it looked like he might not win, he blamed them. But Mr. Trump hasn’t shunned the establishment as he sets up his government. He has chosen Nikki Haley, the governor of South Carolina, as his envoy to the United Nations Representative Michael Pompeo of Kansas for C. I. A. director and Elaine Chao as his secretary of transportation. Ms. Haley, a rising star in the Republican establishment, endorsed Senator Marco Rubio of Florida during the primaries. Mr. Pompeo, a member of the House Intelligence Committee, is   in the nation’s capital. And there are few people who define the Republican establishment more than Ms. Chao, who, in addition to having served in cabinet posts during both Bush administrations, is married to Senator Mitch McConnell of Kentucky, the majority leader. Ultimately there may be no more direct expression of Mr. Trump’s psyche in his administration picks than the generals. Despite saying during the campaign that he knew more about the Islamic State than generals did, Mr. Trump is now embracing the retired leaders of America’s military. That explains Gen. James N. Mattis, who was responsible for American military operations in the Middle East as head of United States Central Command, or Centcom, and is now Mr. Trump’s pick for defense secretary. It also explains Gen. John F. Kelly, who oversaw American military operations in Central and South America as head of United States Southern Command and is now Mr. Trump’s choice for homeland security secretary. It also may help frame his choice of Representative Ryan Zinke of Montana, a former Navy SEAL commander, to be interior secretary. While not a general, Mr. Zinke served in Iraq, Bosnia, Kosovo and the Pacific. His autobiography, “American Commander,” was published recently. Mr. Trump particularly likes “Mad Dog Mattis,” the former Centcom commander’s nickname. “Mad Dog Mattis!” Mr. Trump said to a roar of applause at a recent rally in Fayetteville, N. C. “Mad Dog plays no games, right?” Mr. Trump called General Mattis, who led a Marine division to Baghdad during the 2003 invasion of Iraq, one of “the most effective generals that we’ve had in many, many decades. ”",  # 0
    "TOKYO (Reuters) - President Donald Trump and Japanese Prime Minister Shinzo Abe agreed to work together on steps to counter North Korea s nuclear and missile development, ahead of the U.S. leader s visit to Asia, the Japanese government said late on Monday. In a 20-minute phone call, Trump and Abe discussed the schedule of the president s coming visit, which includes a Nov. 5-7 stop in Japan, and agreed to remain in close contact over North Korea, Deputy Chief Cabinet Secretary Yasutoshi Nishimura told reporters. Trump told Abe  he is looking forward to his visit to Japan, that Japan and America are 100 percent together and there is no room to doubt the Japan-U.S. alliance,  Nishimura said.   They agreed to deepen their discussions on the North Korean situation and other matters  during Trump s visit, he said. ",  # 0
    "A military commander of the Iranian Revolutionary Guard Corps has reportedly told the country’s state controlled media that Iran will soon be sending elite forces into the United States and Europe in an effort to thwart potential plots against the rogue Islamic Republic. As the Washington Free Beacon has reported, the commander stated that at the direction of Supreme Leader Ali Khamenei, the IRGC will soon send their elite forces to operate in both the US and Europe under the goal of bolstering Iran’s hardline regime. “The IRGC is [the] strong guardian of the Islamic Republic… The Fedayeen of Velayat [fighting force] are under the order of Iran’s Supreme leader. Defending and protecting the Velayat [the Supreme Leader] has no border and limit.” Salar Abnoush, deputy coordinator of Iran’s Khatam-al-Anbia Garrison stated. Abnoush’s provocative comments come at a time where Iran is actively upgrading their military hardware and attempting to strengthen their presence throughout the Middle East with funds provided by the Obama Administration. Via Washington Free Beacon  “The whole world should know that the IRGC will be in the U.S. and Europe very soon,” Salar Abnoush, deputy coordinator of Iran’s Khatam-al-Anbia Garrison, an IRGC command front, was quoted as saying in an Iranian state-controlled publication closely tied to the IRGC. Iranian military and government officials have continued to advocate violence against the U.S. and its allies, despite the nuclear deal and several secret side agreements that gave Iran $1.7 billion in cash. Iran accuses the U.S. of violating its end of the agreement by not helping the Islamic Republic gain further access to international banks and other markets. “Our enemies have several projects to destroy our Islamic revolution, and have waged three wars against us to execute their plans against our Islamic Republic,” Abnoush said. “The IRGC has defeated enemies in several fronts. The enemy surrendered and accepted to negotiate with us.” “And now all of our problems are being solved and our country is becoming stronger in all fronts. Some believe the holy defense ended,” the military leader added. “They are wrong; the holy defense continues, and today, it is more complicated than before.” Another source who advises congressional leaders on Iran sanctions issues told the Free Beacon that the Obama administration is blocking Congress from taking action to stop this type of infiltration by Iranian forces. “Iran is ideologically, politically, and militarily committed to exporting the Islamic revolution through terrorism, which is why even the Obama administration says they’re the world’s foremost state sponsor of terrorism,” the source said. “Congress wants to act, but Obama officials keep saying that new laws are unnecessary because the U.S. has enough tools to block Iranian terror expansion. Instead of using those tools, though, they’re sending Iran billions of dollars in cash while Iran plants terror cells in Europe and here at home.” ",  # 1
    "(Reuters) - U.S. intelligence agencies have alleged that Russia interfered in the 2016 presidential election to benefit President Donald Trump. Law enforcement agencies and congressional committees are investigating Russian meddling and possible collusion with members of Trump’s campaign. Here is what is publicly known and not known:   How did the investigations begin? Former President Barack Obama ordered U.S. intelligence agencies to assess whether Russia tried to intervene in the  election after a cyber attack on the Democratic National Committee in July 2016 and the publication of thousands of hacked personal emails from Hillary Clinton’s campaign manager in the month before the Nov. 8 election. Obama told intelligence officials to deliver a report on possible foreign interference before he left the White House in January 2017. What did the intelligence agencies find?  The Central Intelligence Agency, the Federal Bureau of Investigation and the National Security Agency concluded in a report declassified in January that Russian President Vladimir Putin ordered a campaign not just to undermine confidence in the U.S. electoral system but to affect the outcome. The agencies said Putin and the Russian government had a clear preference for Trump to win the White House. Putin’s  associates hacked information, paid social media “trolls” and backed efforts by Russian government agencies and state-funded media to sway public opinion, the agencies said.  The report stopped short of assessing whether Russia succeeded in swaying the election result. Putin and other Russian officials have repeatedly denied interfering in the U.S. election. How many U.S. investigations are there into Russian election meddling? Comey told the House Intelligence Committee on March 20 that the FBI was investigating Moscow’s role in the election, including possible collusion with Trump’s campaign. The Justice Department announced on May 17 that it had appointed Robert Mueller, a former FBI director, as special counsel to lead an independent Russia probe. Mueller would, if the evidence merits, work in tandem with the FBI to handle any related criminal prosecutions. In addition, there are multiple committees in the Senate and House of Representatives investigating various aspects of Russian election meddling. What has Trump said about Russia’s role in the election? He has not taken a clear public position. At a July 2016 news conference, Trump addressed an FBI probe into Clinton’s use of a private email system when she was secretary of state and emails that had possibly been deleted by saying: “I will tell you this, Russia: if you’re listening, I hope you’re able to find the 30,000 emails that are missing.” He subsequently dismissed reports, including from U.S. intelligence officials, that Russia had attempted to intervene in the election on his behalf.    The first time Trump said he accepted the findings of the intelligence agencies was at a Jan. 11 news conference ahead of his inauguration. “As far as hacking, I think it was Russia,” Trump said, although he added: “It could have been others also.” Trump said in May that China may have hacked the emails of Democratic officials to meddle with the election, offering no evidence and countering the view of intelligence officials. Trump has made clear on multiple occasions he believes the Russia investigations have run their course and should be closed. “The Russia-Trump collusion story is a total hoax, when will this taxpayer funded charade end?” he wrote on Twitter on May 8. On May 30, Trump tweeted: “Russian officials must be laughing at the U.S. & how a lame excuse for why the Dems lost the election has taken over the Fake News.” Did the FBI probe continue after Comey’s dismissal? There is no evidence that the FBI ended or paused its investigation after Comey’s departure. Andrew McCabe, who is leading the agency as acting director, promised the Senate Intelligence Committee that Comey’s firing would not affect the investigation and that he will notify the committee of any attempt to delay or derail it.  Trump on June 6 chose former U.S. Justice Department official Christopher Wray, who represented New Jersey Governor Chris Christie in the so-called Bridgegate scandal, to be the next FBI director. Wray will need Senate confirmation. Why was Comey fired? The White House cited a May 9 letter to Trump from Attorney General Jeff Sessions that attached a memo from Rod Rosenstein, the deputy attorney general, on “Restoring Confidence in the FBI” that recommended Comey’s dismissal. Rosenstein’s memo said Comey erred in July 2016 by announcing the FBI had been examining Clinton’s use of a private email server and that the case should be closed without prosecution. Rosenstein’s view was that Comey’s decision to make a public statement broke with longstanding FBI precedent and should have been handled by the then-U.S. attorney general, Loretta Lynch. Trump called Comey a “showboat” and “grandstander” in an interview with NBC News on May 11, saying that he would have fired Comey regardless of Rosenstein’s recommendation. Was Comey’s firing related to the FBI’s Russia probe? Comey told the Senate Intelligence Committee he believes his dismissal was directly related to the FBI’s Russia probe. “I know I was fired because of something about the way I was conducting the Russia investigation, was in some way putting pressure on him, in some way irritating him, and he decided to fire me because of that,” Comey said in his June 8 testimony. Trump asked Comey to end the agency’s investigation of Michael Flynn, his former national security adviser, and also to make some sort of public statement that he was not personally under investigation in order to clear “the cloud” hanging over his presidency, according Comey’s testimony. Comey said he found it concerning that Trump would ask him to drop the agency’s probe of Flynn and declined to state publicly that Trump himself was not under investigation, in part because it would create a “duty to correct” if that changed. “I need loyalty, I expect loyalty,” Trump told Comey at a Jan. 27 dinner, according to Comey. Less than a week before he was fired, Comey told a congressional panel that it made him “mildly nauseous” that he may have affected the outcome of the election by publicly re-opening and then re-closing the Clinton email probe days before the election. In an unorthodox move, Great America Alliance, an offshoot of a pro-Trump super PAC devoted to promoting the White House agenda, developed a television ad called “Showboat” to air as Comey testified. The ad claims Comey was “consumed with election meddling” at the expense of fighting terrorism. Has there been any fallout for Trump associates over contacts with Russia before, during or after the election campaign? Flynn was fired in February. The White House said he had misled Vice President Mike Pence about the contacts he had with Russia’s ambassador to the United States, Sergei Kislyak, before Trump took office. On May 9, federal prosecutors issued grand jury subpoenas seeking business records from people who worked with Flynn when he was a private citizen. On May 10, the Senate Intelligence Committee issued the first subpoena in its Russia investigation, demanding documents from Flynn. He provided the first batch on June 6. Sessions had to recuse himself from involvement in Russia-related probes at the Justice Department because he had not told Congress of his own contacts with Kislyak in 2016. Rosenstein is handling matters related to Russia and it is he who appointed Mueller as special counsel. Is Trump himself under investigation?  Mueller’s probe has expanded to include examining whether Trump obstructed justice by attempting to curtail the FBI’s probe of Russian meddling, the Washington Post reported on June 14. In the short letter Trump sent to Comey dismissing him from the FBI, he had thanked Comey for informing him that he was not under investigation. Comey has confirmed that Trump was not personally under investigation at any point before his dismissal from the FBI. The Post reported the scope of the probe expanded to include Trump shortly after Comey’s dismissal, citing government officials. Trump’s personal lawyer, Marc Kasowitz, denied Trump ever told Comey he needed and expected his loyalty. ",  # 0
    " (Corrects this August 1st story to show Tillerson did not say European countries would be reluctant to re-impose sanctions in 11th paragraph) By Yeganeh Torbati WASHINGTON (Reuters) - U.S. Secretary of State Rex Tillerson acknowledged on Tuesday that he and President Donald Trump disagree over the 2015 Iran nuclear deal, and said the two men discuss how to use the international agreement to advance administration policies. Trump at times vowed during the 2016 presidential election campaign to withdraw from the agreement, which was signed by the United States, Russia, China and three European powers to curb Iran’s nuclear program in return for lifting most Western sanctions.  Trump has preserved the deal for now, although he has made clear he did so reluctantly after being advised to do so by Tillerson. “He and I have differences of views on things like JCPOA, and how we should use it,” Tillerson said at a State Department briefing, using the acronym for the deal, formally called the Joint Comprehensive Plan of Action (JCPOA). Tillerson said that Washington could “tear it up and walk away” or stay in the deal and hold Iran accountable to its terms, which he said would require Iran to act as a “good neighbor.” Critics say the deal falls short in addressing Iran’s support for foreign fighters in Iraq and Syria, arms shipments around the Middle East and ballistic missile tests. The White House did not immediately respond to a request for comment on Tillerson’s remarks. Trump said in an interview with the Wall Street Journal last month that he predicts Iran will be judged “noncompliant” with the Iran deal at the next deadline in October, and that he would have preferred to do so months ago. Tillerson expressed a more nuanced view of the deal’s potential benefits on Tuesday. “There are a lot of alternative means with which we use the agreement to advance our policies and the relationship with Iran, and that’s what the conversation generally is around with the president as well,” Tillerson said. European officials would likely be reluctant to re-impose sanctions, especially the broader measures that helped drive Iran to negotiate over its nuclear program in the first place. New U.S. sanctions on Iran in July were a breach of the nuclear deal and Tehran had lodged a complaint with the body that oversees the pact’s implementation, a senior Iranian politician said. Tillerson acknowledged that the United States is limited in how much it can pressure Iran on its own and said it was important to coordinate with the other parties to the agreement. “The greatest pressure we can put to bear on Iran to change the behavior is a collective pressure,” he said. ",  # 0
    "PARIS (Reuters) - French President Francois Hollande on Wednesday congratulated Donald Trump on his shock victory in the U.S presidential election, but warned the result would open up a period of uncertainty. “I congratulate him as is natural between two democratic heads of state,” said Hollande. “This American election opens a period of uncertainty.” France would be vigilant and frank in its talks with the new administration on international issues, he said, adding that Trump’s victory showed that France needed to be stronger and that Europe needed to be united. A presidential official said Hollande had spoken to German Chancellor Angela Merkel before making his declaration. ",  # 0
    "Militia Fighters to Advance on Areas West of City by Jason Ditz, October 28, 2016 Share This  After a week and a half of assuring everyone that their role in the invasion of Mosul would be very limited, and removed from the Sunni population, Iraq’s Shi’ite militias have been announced to have launched an offensive west of Mosul, advancing on Tal Afar .  According to officials, the main goal of this offensive is to cut the city of Mosul off from ISIS territory in Syria, preventing the ISIS fighters within the city from fleeing west if the battle begins to turn sour. The offensive is said to start “within a few days or hours.”  The involvement of Shi’ite militias in the “liberation” of Sunni Arab cities in Iraq has been controversial because they often end up carrying out extrajudicial executions, looting and torture of locals they suspect of being secretly in league with ISIS.  An effort to block flight from Mosul to ISIS territory is Syria would be an obvious step, but the militias’ involvement again risks targeting of civilians, as the population of from Mosul is already overrunning the minimal camps set up for the displaced.  Civilians fleeing these cities are usually not welcomed in Iraqi government-held territory, and wind up fleeing into other ISIS-held territory. With little left in Iraq, this likely means fleeing into Syria, and the militias’ presence means the civilians displaced by the attack are likely to be targeted for fleeing “with” ISIS. Last 5 posts by Jason Ditz",  # 1
    "“Lifeline” is a   thriller directed by an Academy Award winner about a man’s search for his missing girlfriend. In the film, set in Shanghai with a plot driven by corporate malfeasance, punches are thrown, shots are fired and people are killed. At one point, the actress Olivia Munn stands over a dead woman, blood on her hands. But this is no ordinary movie. It is an online advertisement for the mobile technology company Qualcomm and, in particular, its Snapdragon 820 chip set, a smartphone processor. As more people skip or block ads when streaming shows or browsing websites, advertisers are trying to find new ways to deliver their messages. The internet has long been a place where companies have tried to break out of the   and   ad model, but as it has become easier to present   videos online  —   and as top directors and actors have shown a willingness to be involved  —   these efforts have become more sophisticated. The goal of “Lifeline” and similar ads is “to make something you want to see  —   and the holy grail is if people seek you out,” said Teddy Lynn, chief creative officer for content and social at Ogilvy  Mather, which produced the film. “This is a piece of entertainment that can compete in a very crowded marketplace. ” “Lifeline” was released in May and pushed out on multiple social media channels in the United States and China. “Inside Lifeline,” a      look at the film, is also available on the “Lifeline” website. It emphasizes the importance of the cellphone to the action in the film and the phone’s various features the film hopes to highlight, like its long battery life and improved photo capability. In many ways, “Lifeline” is just an extension of product placement and show sponsorship by advertisers that goes back to the early days of radio and television, said Lou Aversano, chief executive of Ogilvy  Mather NY. “I think we continue to push, not just in terms of length, but in terms of the line between entertainment and brand message,” he said. Many companies are looking for ways to promote their brand through longer storytelling, such as Johnnie Walker, Nike and Prada. It is an impulse that dates to at least 2001 or 2002, when a series of eight   films for BMW starring Clive Owen appeared online. “That was at the time the internet was still ” said Steve Golin, founder and chief executive of Anonymous Content, a multimedia development company that produced the BMW ads and “Lifeline. ” “It would take all night to download. ” “Lifeline,” which stars Ms. Munn, Leehom Wang and Joan Chen, is directed by Armando Bo, who won an Oscar for best original screenplay for “Birdman” in 2015. The film has attracted 20 million views. There have been an additional 100 million combined views of the film’s trailers and the    video, Mr. Lynn said. Eighty percent of the views came from China, which was the primary market. The dialogue is 70 percent in Chinese and 30 percent in English. Mr. Lynn said Qualcomm would not disclose the cost of the ad, but noted that with less money needed to buy time on television, more was available for the production. “You can create content that is compelling and you don’t have to spend money to place it on TV,” said Mr. Golin, whose company has been involved with movies like “Spotlight” and “The Revenant” and TV shows like “Mr. Robot. ” “We think this is the direction advertising is headed. As long as sports exists, we will still do   and   commercials, but with most other entertainment there is a lot of resistance to watching    advertising. ” Mark Crispin Miller, a professor of media studies at New York University, said the disappearing boundaries between advertising and entertainment could be troubling. Ads, by their nature, often exaggerate “the benefits or virtues of the products and, even more troubling, downplay the dangers or risk of a product,” Mr. Miller said. And using big stars “makes the commercial intent even harder to perceive and blurs the true purpose behind the work. ” Still, more advertisers are eager to experiment. Take “The Ballad of the Dreadnought,” a   documentary about the distinctive guitar body originally manufactured by C. F. Martin  Company. It is narrated by Jeff Daniels and includes interviews with musicians like Rosanne Cash, Stephen Stills and Steve Miller. The film appears solely on Martin’s website, but was selected to appear at several film festivals. It has received 40, 000 views since it first streamed on May 5, said Scott Byers, a managing partner at Lehigh Mining  Navigation, the advertising agency that developed the film. The documentary idea developed, Mr. Byers said, when Martin came to his agency wanting to celebrate the dreadnought guitar, which was developed in the early 20th century but never trademarked, enabling many other manufacturers to copy it over the years. “They asked, ‘What can we do to reclaim ownership of the shape? ’” said Denis Aumiller, also a managing partner at the agency. “The initial thought was that we would produce a short,   product video, or maybe a magazine article. ” As enticing as it may be to think of every commercial as a potential short film that could play on the festival circuit, creating something that attracts viewers and promotes a product is not easy. Entertainment, after all, is not the ultimate goal. “At some point,” Mr. Aversano said, “there’s a responsibility to deliver the message of the brand. Otherwise it’s just empty calories. ”",  # 0
    "LOS ANGELES (Reuters) - Editors Note: Attention to language that may be offensive to some readers in paragraph 3 The National Mall in Washington could become a sea of bright pink the day after Donald Trump is inaugurated as U.S. president if the vision of a pair of Los Angeles women is realized.   For two months, Krista Suh and Jayna Zweiman have called on people around the world to make 1.17 million pink “pussyhats” for those attending the Women’s March, a rally on Jan. 21 organized with hopes of bringing attention to civil and human rights issues. The name of the hats comes partly from President-elect Trump’s comments in an infamous 2005 tape that came to light during his campaign in which, discussing women, he said: “Grab them by the pussy. You can do anything.” The project is meant to “create a sea of pink hats which is going to be a strong collective visual statement,” Zweiman said.  It is also meant to involve “people across the country and the world who want to be a part of the march but can’t physically be there and want to support the marchers,” she said.  The women have asked volunteers around the world to help sew, crochet or knit pink hats with ears by using simple patterns available on the project’s website. They say it is easier than knitting a scarf, the typical starter project for novices. “It’s a rectangle folded in half, stitched up the sides and when you put it on your round head, these ears come out,” said Zweiman, surrounded by women stitching away at a knitting store on Friday night. Marchers can get a hat by contacting a maker through an online distribution system, through social media or at sites in Washington. “So it’s a distribution system that’s very grassroots and our aim is to get a hat on every marcher’s head who wants one,” Zweiman said. Organizers have said the protest could draw around 200,000 people, but Suh and Zweiman decided to aim for the 1.17 million people that could feasibly fit in the Mall. For some knitters at Friday’s “pussyhat party,” it was hard to put the needles down as the deadline nears. “Doing this is taking my mind off things,” said Marina Mont’Ros.  “This will be my 35th hat. Every time I think ‘OK, that’s enough. I’ve made enough, I’m going to stop,’ I read something in the news and, give me that yarn! I’m making more.” ",  # 0
    "WASHINGTON (Reuters) - U.S. Senate Democrats on Thursday pressed Ben Carson, Donald Trump’s nominee to lead housing policy, on potential conflicts of interest between the agency he is set to run and properties in which the president-elect may hold a financial stake. The Department of Housing and Urban Development (HUD) hands out billions of dollars each year to developers and landlords. Democrats grilling Carson during his confirmation hearing for the post expressed concerns that some of that taxpayer money could end up directed to Trump or members of his family. “The president-elect is hiding his family businesses interests from you, from me, and the rest of America,” Senator Elizabeth Warren of Massachusetts, one of Trump’s most vocal critics, told Carson. “He can divert taxpayer money into his own pockets without the American people knowing about it.” Trump, a New York businessman who got his start in real estate, has refused to release his tax returns or provide detailed information about his holdings across the globe. On Wednesday, he announced that he would turn control of his business empire over to his two oldest sons and move his assets into a trust. Those moves, however, have not assuaged some critics who argue that, as president, Trump will still have undisclosed financial interests that could affect his policies. Pushed on the issue by Democrats, Carson at one point vowed he would monitor any potential conflicts of interest but said  he was unsure about how to go about it. Asked how he would prevent conflicts, the retired neurosurgeon and former Republican presidential candidate told the Senate Banking Committee, “I would hope what would happen with this committee is that we could come up with a suggestion that might be acceptable to all sides.” Trump reportedly has an interest in the massive Starrett City housing project in the New York City of Brooklyn, which has received HUD funds. But Trump’s real estate career has focused mainly on luxury housing, commercial real estate and resort properties. Earlier in the hearing, Carson was noncommittal when Warren asked him to ensure that no HUD programs would benefit Trump.  “If there happens to be an extraordinarily good program that is working for millions of people and it turns that someone you’re targeting is going to gain $10 from it, am I going to say, ‘No, the rest of you Americans can’t have it?’” Carson responded. Carson is widely expected to be confirmed by the Republican-controlled Senate committee and full Senate. Much of Thursday’s hearing focused on Carson’s long-standing antipathy toward social-welfare programs, with some senators asking him whether he, in fact, supports HUD’s mission to provide housing assistance to millions of Americans. “I think the rental assistance program is essential,” Carson replied. He later, added, however that social programs have to operate within financial constraints.  Much of Carson’s remarks focused on his well-documented background growing up poor in inner-city Detroit, saying he learned young what “housing insecurity” means. He provided few details in terms of U.S. housing policy or how he would revamp the agency, suggesting only that he was interested in making it more efficient. If confirmed as HUD secretary, Carson would oversee an agency with a roughly $40 billion budget that not only helps the poor but underwrites mortgage loans for middle-class families. HUD’s Federal Housing Administration (FHA) stands behind roughly one in five home loans and at least $1.1 trillion in debt - a number that Carson said was troubling. Outgoing HUD Secretary Julian Castro this week slashed HUD fees for mortgage default insurance, which he said would save the average HUD-backed homeowner $500 a year. Carson suggested that he would consider reversing that surprise move. “Certainly, if confirmed, I am going to work with the FHA administrator and other financial experts to really examine that policy,” he said. ",  # 0
    "MOSCOW (Reuters) - Pyongyang does not plan to hold any talks with Washington about its nuclear program, a senior North Korean diplomat said on Friday, declaring that possessing nuclear weapons was a matter of life and death for North Korea, the RIA news agency reported. Tension has soared on the peninsula following a series of weapons tests by North Korea and a string of increasingly bellicose exchanges between U.S. President Donald Trump and North Korean leader Kim Jong Un. Trump, in a speech last month at the United Nations, threatened to  totally destroy  North Korea if necessary to defend itself and allies and called the North s leader a  rocket man  on a suicide mission. Choe Son-hui, director-general of the North American department of North Korea s foreign ministry, told a non-proliferation conference in Moscow Washington would  have to put up  with North Korea s nuclear status, RIA reported.  This is a matter of life and death for us. The current situation deepens our understanding that we need nuclear weapons to repel a potential attack.   We will respond to fire with fire.  Pyongyang would regard any attempts to strangle the country via U.N. Security Council sanctions as an attempt  to declare war . Russian Foreign Minister Sergei Lavrov, addressing the same conference earlier on Friday, urged world powers to get behind a joint Russian-Chinese roadmap for settling the crisis over North Korea s weapons program.  We are convinced that its implementation will promote the lessening of military activity and tension on the Korean peninsula and the forming in Northeastern Asia of a system of equal and indivisible security,  he said.  The plan proposes a moratorium on North Korea s missile and nuclear weapons tests, while South Korea and the United States suspend holding military exercises.  The main task at the current stage is to prevent a military conflict which will inevitably lead to a large-scale humanitarian, economic and ecological catastrophe,  Lavrov said.  All the sides involved should exercise restraint.     ",  # 0
    "There will be no immediate overhaul or reforms to the   foreign guest worker visa program, according to White House Press Secretary Sean Spicer. [In a media briefing, Spicer said President Donald Trump would be reviewing a number of immigration issues that include   visas, but said any reforms to the system before foreign workers could begin applying for the visas on April 1, 2017 would not occur.  “I think there is the legal part of immigration and then the illegal part of immigration,” Spicer said, according to The Hindu. “The President’s actions that he’s taken in terms of his executive order and other revamping of immigration policy have focused on our border security, keeping our country safe, our people safe. And then, obviously, whether it’s   visas or the other one  —   spousal visas  —   other areas of student visas, I think there is a natural desire to have a full look at  —   a comprehensive look at that. ” The open borders lobby has shifted much of their focus away from Trump’s initiatives on illegal immigration and are now in defensive mode on the issue of   visas, as Breitbart Texas reported.   media and open border groups are particularly worried of any kinds of changes to the   visa program, as the system largely favors young, male Indian workers who fill American   jobs. The Trump Administration’s inaction to move on the   visa program could see pushback from immigration hawks and voters, as only 30 percent of Americans see the program as necessary, Breitbart Texas reported. Trump does remain aware of issues regarding the current legal immigration system. In an interview with POLITICO, Sen. Tom Cotton ( ) said Trump seemed to endorse his and Sen. David Perdue’s ( ) “RAISE Act,” which will cut legal immigration by 50 percent over a period of time. The legislation would also reduce the number of green cards issued every year from about one million to 500, 000 end extended family chain migration to the U. S. eliminate the 50, 000 visas granted to foreigners under the diversity visa lottery and permanently cap U. S. refugees resettlement to 50, 000 per year. Some 85, 000   visas are allotted to foreign workers every year through the U. S. government’s visa lottery program, which has long been criticized by Trump’s Attorney General Jeff Sessions. John Binder is a contributor for Breitbart Texas. Follow him on Twitter at @JxhnBinder.",  # 0
    "  Carol Adl in News , UK // 0 Comments  A Tory councillor believes that homeless people in Bradford city centre should be ‘grabbed by scruff of the neck’ and ‘dealt with’.  The comments by David Heseltine, a Conservative Party Councillor in Bradford caused outrage during a meeting of the Council’s regeneration overview and scrutiny committee. He said :  “The tramps and drunks sleeping in doorways. What we need to do is get them by the scruff of the neck and deal with them. People have said they’d go into Bradford if they were eliminated.”  He claimed that while plans to redevelop the city centre were all well and good, the investment had been ruined by the homeless.  So what exactly does Mr Heseltine envisage when he suggests grabbing the homeless “by the scruff of the neck” and eliminating them?  EvolvePolitics reports:  Why Councillor Heseltine believes that perpetrating an assault on the homeless would rectify the problem is a mystery. He would be better advised to encourage his colleagues to work towards providing people with homes, opportunities for employment, improving mental health services and providing interventions for alcohol and drug addiction.  The ‘Nasty Party’ Councillor said that he had spoken to people who claimed that they would not visit Bradford city centre due to the number of drinkers and beggars.  Heseltine is correct in his assertion that Bradford council – like many other councils – places the town centre aesthetic above the needs of those that live in and around them. However, his suggestions for dealing with the problem of homelessness are utterly reprehensible. These are people, not dogs or vermin to be disposed of to make your town look just a little better.  Most people are never more than three paychecks away from being homeless. Our elected representatives would be well advised to remember that.",  # 1
    " There s no way to sample this extra 20 or 25% of new voters that are Trump supporters. Sid Miller, Texas Agriculture Commissioner, went on FOX News on Wednesday to discuss the record breaking voter turnout for Donald Trump in the Lone Star State.Sid Miller: We have a record number of people registered to vote in Texas. We re having record turnouts, the first day, the second day of voting. And it s not Bernie Sanders supporters coming out to support Hillary. It s not Barack Obama supporters coming out to support Hillary. It s a new surge of Trump voters, many who have never registered to vote. Many who have not voted in eight or ten elections so they re not reported in the polls I know for a fact that the polls are off because they oversample Democrats by eight sometimes up to sixteen percent oversampling Democrats. They re oversampling women by five to eight percent. So the Republican vote is underreported. Plus there is no way to sample this extra twenty to twenty-five percent of new voters that are Trump voters. They re not Republican, they re not Democrats, they re pragmatists. They re tired of the status quo and they want change.  GPYUGE RECORD BREAKING VOTES in TEXAS!!! & Its Not Bernie Fans or Obama Voters for Hillary Clinton!! ALLL Donald Trump Folks  #wednesdaywisdom pic.twitter.com/Sn79fMhwXG   STOCK MONSTER  (@StockMonsterUSA) October 26, 2016",  # 1
    "21st Century Wire says US college students continue to be emotionally traumatized and  threatened  by Donald Trump s surprise presidential victory this week, sparking a wave of nationally coordinated street protests, organized in part through the George Soros funded Democratic Party  community organizing  web portal MoveOn.org. Various protests and riots have been organized around a number university campuses including Berkeley, Portland, New York City and Los Angleses. When asked why she scared by the election result, one female university student claimed that the President Elect is a  rapist,  while another claim that,  He [Trump] is about to have a court case about a 13 years old girl soon, and I m not down for that. The alleged court case the student is referring was actually dropped, and has since been exposed as a likely politically motivated hoax according to reports.Here is last night s student protest, including multiple interviews from both sides of the debate, filmed last night in  Albuquerque, New Mexico, by Stuart J Hooper. Watch:  SUPPORT 21WIRE   SUBSCRIBE & BECOME A MEMBER @21WIRE.TV",  # 1
]

# Etiquetas esperadas (0 = falsa, 1 = verdadera)
true_labels = [
    1,
    0,
    0,
    1,
    0,
    0,
    1,
    1,
    0,
    1,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    0,
    1,
    1,
    1,
]



# Realizar las predicciones
predictions, probabilities = predict_news(news_to_test)

# Contadores para las verdaderas y falsas
correct_true = 0
correct_false = 0
incorrect_true = 0
incorrect_false = 0

# Mostrar los resultados con las etiquetas reales
for i, news in enumerate(news_to_test):
    pred_label = 'Verdadera' if predictions[i] == 1 else 'Falsa'
    true_label = 'Verdadera' if true_labels[i] == 1 else 'Falsa'

    # Comparar con las etiquetas reales
    if true_labels[i] == 1 and predictions[i] == 1:
        correct_true += 1
    elif true_labels[i] == 0 and predictions[i] == 0:
        correct_false += 1
    elif true_labels[i] == 1 and predictions[i] == 0:
        incorrect_true += 1
    elif true_labels[i] == 0 and predictions[i] == 1:
        incorrect_false += 1

    print(f"Noticia: {news}")
    print(f"Predicción IA: {pred_label}")
    print(f"Etiqueta Real: {true_label}")
    print(f"Probabilidad de ser falsa: {probabilities[i][0]:.4f}")
    print(f"Probabilidad de ser verdadera: {probabilities[i][1]:.4f}")
    print("")

# Calcular las métricas de evaluación
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
accuracy = accuracy_score(true_labels, predictions)

# Mostrar las métricas
print(f"Exactitud: {accuracy:.4f}")
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Contador final
print("\nContador final:")
print(f"Verdaderas correctas: {correct_true}")
print(f"Falsas correctas: {correct_false}")
print(f"Verdaderas incorrectas: {incorrect_true}")
print(f"Falsas incorrectas: {incorrect_false}")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Noticia: Home / Badge Abuse / Shots Fired, 117 Arrests Made as Militarized Police Remove Pipeline Protesters Shots Fired, 117 Arrests Made as Militarized Police Remove Pipeline Protesters The Free Thought Project October 28, 2016 1 Comment  Standing Rock, ND — (RT) Hundreds of police in riot gear and with heavy military equipment have moved Dakota Access Pipeline protesters from their encampment on private land in North Dakota. Police have reportedly arrested at least 117 demonstrators so far.  “117 protesters have been arrested [as of 8:15 pm Thursday]. Morton County will be utilizing other jails in this mass arrest operation,” said Donnell Hushka, a spokeswoman for the Morton County, North Dakota, Sheriff’s Department.  Police have entered the sacred ground camp and surrounded protesters there. Livestream videos showed more than one hundred protesters on the front line. Police have protesters more or less surrounded. #noDAPL pic.twitter.com/G4xGQuXpZM  — Jason Patinkin (@JasonPatinki